# 库

In [1]:
import re, sys, os, json
import ijson
import pandas as pd
from importlib import reload

In [2]:
from vVariableInspector import _getshapeof

In [3]:
from password import mysqlDb, mysqlIp, mysqlPassword, mysqlUser

In [4]:
from vMysql import MysqlProxy

In [75]:
from vUtil.vTqdm import tqdm, trange
from vUtil.vLog import frmt
from vUtil.debug import  debug

# 重新加载

In [175]:
import vMysql
import vMysql.mysqlProxy as mp# 库
reload(mp), reload(vMysql)
from vMysql import MysqlProxy
mp.MysqlProxy.__version__, MysqlProxy.__version__

(20210113, 20210113)

# 初始化

In [177]:
db = MysqlProxy(ip=mysqlIp, user=mysqlUser, password=mysqlPassword, db=mysqlDb)

In [10]:
path = r'C:\Users\ict\LymphV\data\cnd\who is who\na_v1'

# 函数定义

In [11]:
def getJsonKv(f):
    '''
    读取大json文件object的每个key-value
    '''
    parser = ijson.parse(f)
    for x, y, z in parser:
        if y == 'map_key':
            objects = ijson.items(parser, z)
            yield z, objects.__next__()

In [101]:
def getJsonKv(f):
    '''
    读取大json文件object的每个key-value
    '''
    parser = ijson.parse(f)
    pool = []
    k = None
    for x, y, z in parser:
        pool += [(x, y, z)]
        if not x and (y == 'map_key' or y == 'end_map'):
            if k is not None:
                objects = ijson.items(pool, k)
                yield k, objects.__next__()
            k = z
            pool = []

In [15]:
def insertAuthor (filePath, table):
    with open(filePath, encoding='utf-8') as f:
        with tqdm(enumerate(getJsonKv(f), 1), leave=False) as tkvs:
            for n, _ in tkvs: pass
    with open(filePath, encoding='utf-8') as f:
        with tqdm(getJsonKv(f), total=n) as tkvs:
            for name, aps in tkvs:
                frmt(name, tqdm=tkvs)
                
                names, authors, papers = [], [], []
                for author in aps:
                    for pub in aps[author]:
                        names += [name]
                        authors += [author]
                        papers += [pub]                
                db.insert(table, df=pd.DataFrame({'name':names, 'id':authors, 'paper_id':papers}), ignore=True)

In [126]:
def insertPub (filePath, table, tableAuthors, bulkSize=100, start=0, n=None):
    if n is None:
        with open(filePath, encoding='utf-8') as f:
            with tqdm(enumerate(getJsonKv(f), 1), leave=False) as tkvs:
                for n, _ in tkvs: 
                    pass
    with open(filePath, encoding='utf-8') as f:
        with tqdm(enumerate(getJsonKv(f)), total=n) as tkvs:
            cols = ['title', 'abstract', 'venue', 'year']
            colsAuthors = ['name', 'org']
            
            def init ():
                data = {col : [] for col in [*cols, 'keywords', 'id']}
                dataAuthors = {col : [] for col in [*colsAuthors, 'id' , 'no']}
                return data, dataAuthors
            
            data, dataAuthors = init()
            
            for now, (pid, pub) in tkvs:
                if now < start: continue
                frmt(pid, tqdm=tkvs)
                
                for col in cols: data[col] += [pub.get(col,None)]
                data['id'] += [pid]
                data['keywords'] += [';'.join(pub.get('keywords',None) or [])]
                
                for no, author in enumerate(pub['authors'], 1):
                    for col in colsAuthors: dataAuthors[col] += [author.get(col,None)]
                    dataAuthors['id'] += [pid]
                    dataAuthors['no'] += [no]
                
                if now == n - 1 or bulkSize <= len(data['id']):
#                     return data, dataAuthors
                    
                    db.insert(table, df=pd.DataFrame(data), ifCommit=False)
                    db.insert(tableAuthors, df=pd.DataFrame(dataAuthors))
                    data, dataAuthors = init()

# na_v1_pub/na_v1_author

In [99]:
na_v1_pub = (os.path.join(path, 'na_v1_pub.json'), 'na_v1_pub', 'na_v1_pub_authors')
na_v1_author = (os.path.join(path, 'na_v1_author.json'), 'na_v1_author')

In [104]:
with open(os.path.join(path, 'na_v1_author.json'), encoding='utf-8') as f:
    with tqdm(enumerate(getJsonKv(f), 1)) as tkvs:
        for n, (k, v) in tkvs:
            if sum(len(y) for x, y in v.items()) < 1:
                print (k, v)

j_yu {}
roger_williams {}



In [102]:
insertAuthor(*na_v1_author)

In [137]:
insertPub(*na_v1_pub, 1, 180035, n=388994)

# train_pub/train_author

In [179]:
train_pub = (os.path.join(path, 'train_pub.json'), 'train_pub', 'train_pub_authors')
train_author = (os.path.join(path, 'train_author.json'), 'train_author')

In [180]:
insertAuthor(*train_author)

In [181]:
insertPub(*train_pub, 1)

In [182]:
from xpinyin import pinyin

ModuleNotFoundError: No module named 'xpinyin'

In [188]:
from pypinyin import pinyin, lazy_pinyin, Style

In [193]:
pinyin('hi there 你 好')

[['hi there '], ['nǐ'], [' '], ['hǎo']]

In [190]:
lazy_pinyin('甄宓', heteronym=True)

TypeError: lazy_pinyin() got an unexpected keyword argument 'heteronym'

In [189]:
pinyin('甄宓', heteronym=True)

[['zhēn', 'zhèn', 'juàn'], ['mì', 'fú']]

# na_v1_pub/na_v1_author

In [122]:
pymysql.err.# na_v1_pub/na_v1_author

<module 'pymysql' from 'C:\\Users\\ict\\LymphV\\software\\python\\anaconda\\lib\\site-packages\\pymysql\\__init__.py'>

In [172]:
try:
#     db.insert('train_pub', id = '1' * 300)
    db2.db
except Exception as ex:
    e = ex
    raise e

OperationalError: (2003, "Can't connect to MySQL server on 'rds-inner.mysql.zhangbei.rds.aliyuncs.com' (timed out)")

In [141]:
pymysql.err.DataError, pymysql.err.ProgrammingError

(pymysql.err.DataError,)

In [ ]:
pymysql.err.MySQLError.

In [161]:
e.args[0]

1406

In [ ]:
OperationalError(1054, "Unknown column 'ppp' in 'field list'")

In [173]:
e

pymysql.err.OperationalError(2003,
                             "Can't connect to MySQL server on 'rds-inner.mysql.zhangbei.rds.aliyuncs.com' (timed out)")

In [152]:
e

pymysql.err.DataError(1406, "Data too long for column 'id' at row 1")

In [150]:
e

pymysql.err.ProgrammingError(1064,
                             "You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'ddd' at line 1")

In [149]:
e.__class__.__bases__[0].__bases__[0].__bases__[0].__bases__

(Exception,)

In [289]:
pd.DataFrame(x)

,title,abstract,venue,year,keywords,id
0,Study on the fabrication technology of SnO2 mi...,"In this paper, a microlens with 45μm radius an...",Jiguang Jishu/Laser Technology,1996,,56PyrvLA
1,Gas-phase ion-molecule reactions of neutral C6...,Gas-phase ion-molecule reactions of buckminste...,Rapid Communications in Mass Spectrometry,1998,,YlCMdSUx
2,Gas-phase ion-molecule reactions of C60 with m...,Gas-phase ion-molecule reactions of buckminste...,Kao Teng Hsueh Hsiao Hua Heush Hsueh Pao/ Chem...,1998,C60;Ion-molecule reaction;Methyl ethers,JIKd7Vd8
3,Multipole theory for 3D magnetic vector potent...,A MT (Multipole Theory) method for 3D magnetic...,Zhongguo Dianji Gongcheng Xuebao/Proceedings o...,1998,,QgReN4u4
4,Virtual corporation and enterprise group,Based on the respective characteristics of the...,Jisuanji Jicheng Zhizao Xitong/Computer Integr...,1999,,g7lzsICE
5,Electrospray ionization multiple-stage tandem ...,Electrospray ionization (ESI) combined with mu...,Rapid Communications in Mass Spectrometry,1999,,Nm9K7f15
6,Longitudinal distribution and evolution law of...,For studying the relationship between the dist...,Xi'an Shiyou Xueyuan Xuebao/Journal of Xi'an P...,1999,,mizzjKfH
7,Fracture character and collapse mechanism of t...,In order to seek the sloughing prevention meas...,Xi'an Shiyou Xueyuan Xuebao/Journal of Xi'an P...,1999,,9U5h8EPJ
8,Gas-phase ion-molecule reactions of neutral C6...,The gas-phase ion-molecule reactions of C,Journal of Physical Chemistry A,1999,,RKKSV7Rf
9,A study of the massive black hole in the cente...,We report the observation of a dramatic outbur...,Wuli Xuebao/Acta Physica Sinica,2000,,59hRFwAG


In [286]:
pd.DataFrame(y)

,name,org,id,no
0,Qingya Wang,Jilin Univ,56PyrvLA,1
1,Li Ma,Jilin Univ,56PyrvLA,2
2,Fanghai Zhao,Jilin Univ,56PyrvLA,3
3,Wei Zheng,Jilin Univ,56PyrvLA,4
4,Yushu Zhang,Jilin Univ,56PyrvLA,5
5,Dingsan Gao,Jilin Univ,56PyrvLA,6
6,Li Ma,Changchun Inst. of Applied Chemistry,YlCMdSUx,1
7,Ziyang Liu,Changchun Inst. of Applied Chemistry,YlCMdSUx,2
8,Yaping Xu,Changchun Inst. of Applied Chemistry,YlCMdSUx,3
9,Weijie Wang,Changchun Inst. of Applied Chemistry,YlCMdSUx,4


In [ ]:
db.insert()

In [88]:
i = 0

In [25]:
with open(path + '/na_v1_author.json') as f:
    objects = ijson.kvitems(f, 'li_ma.MRb2KOMw')
    print([*objects])
    for i in range(10):
        print(objects.__next__())

[]


StopIteration: 

In [44]:
[*enumerate(range(10),2)]

[(2, 0),
 (3, 1),
 (4, 2),
 (5, 3),
 (6, 4),
 (7, 5),
 (8, 6),
 (9, 7),
 (10, 8),
 (11, 9)]

In [26]:
with open(path + '/na_v1_author.json') as f:
    parser = ijson.parse(f)
    for x, y, z in parser:
        print(x,y,z)

 start_map None
 map_key li_ma
li_ma start_map None
li_ma map_key MRb2KOMw
li_ma.MRb2KOMw start_array None
li_ma.MRb2KOMw.item string 56PyrvLA
li_ma.MRb2KOMw.item string YlCMdSUx
li_ma.MRb2KOMw.item string JIKd7Vd8
li_ma.MRb2KOMw.item string QgReN4u4
li_ma.MRb2KOMw.item string g7lzsICE
li_ma.MRb2KOMw.item string Nm9K7f15
li_ma.MRb2KOMw.item string mizzjKfH
li_ma.MRb2KOMw.item string 9U5h8EPJ
li_ma.MRb2KOMw.item string RKKSV7Rf
li_ma.MRb2KOMw.item string 59hRFwAG
li_ma.MRb2KOMw.item string qRyU2VDx
li_ma.MRb2KOMw.item string 15GL4Qbv
li_ma.MRb2KOMw.item string NQgktZ8J
li_ma.MRb2KOMw.item string kEVdx8XQ
li_ma.MRb2KOMw.item string 2X5xipeW
li_ma.MRb2KOMw.item string TfMAXCIu
li_ma.MRb2KOMw.item string wefCfkpN
li_ma.MRb2KOMw.item string IfP6liBY
li_ma.MRb2KOMw.item string x9ekozOb
li_ma.MRb2KOMw.item string seUQKtEP
li_ma.MRb2KOMw.item string Su2YHtqZ
li_ma.MRb2KOMw.item string koKZgAaj
li_ma.MRb2KOMw.item string mkkP5HIK
li_ma.MRb2KOMw.item string IbSOY0yD
li_ma.MRb2KOMw.item string Ub

li_ma.3yshGMiy.item string aASyC6eV
li_ma.3yshGMiy.item string hcdcKwNW
li_ma.3yshGMiy.item string jrnVcvTh
li_ma.3yshGMiy.item string mxh9G1j4
li_ma.3yshGMiy.item string tBq5V0wn
li_ma.3yshGMiy.item string 8kKxSMmD
li_ma.3yshGMiy.item string bBdu7onM
li_ma.3yshGMiy.item string mSj5q37D
li_ma.3yshGMiy.item string QkubgWxC
li_ma.3yshGMiy.item string D1xCjfQE
li_ma.3yshGMiy.item string cKxiSITp
li_ma.3yshGMiy.item string xCOEzEXE
li_ma.3yshGMiy.item string nP6SiR3i
li_ma.3yshGMiy.item string gXJkuKRL
li_ma.3yshGMiy.item string LjWRYCCt
li_ma.3yshGMiy.item string DiRpUpY1
li_ma.3yshGMiy.item string qA44tR49
li_ma.3yshGMiy.item string UjYo2lLB
li_ma.3yshGMiy.item string NGVkRUrF
li_ma.3yshGMiy.item string 2MsiWfMM
li_ma.3yshGMiy.item string jfgN9Pv0
li_ma.3yshGMiy.item string 5QDoyXz6
li_ma.3yshGMiy.item string 605mcJWe
li_ma.3yshGMiy.item string ZX4R9VZ9
li_ma.3yshGMiy.item string OiTTWpkt
li_ma.3yshGMiy.item string JVJjmIvp
li_ma.3yshGMiy.item string 03V5HouG
li_ma.3yshGMiy.item string 0

li_ma.eLVUxW4O end_array None
li_ma map_key TVKXNVc7
li_ma.TVKXNVc7 start_array None
li_ma.TVKXNVc7.item string Wdgp3JiS
li_ma.TVKXNVc7.item string 03dDyJvR
li_ma.TVKXNVc7 end_array None
li_ma map_key KjxAtkws
li_ma.KjxAtkws start_array None
li_ma.KjxAtkws.item string iIn8DoZA
li_ma.KjxAtkws end_array None
li_ma map_key kNIZZ8Jz
li_ma.kNIZZ8Jz start_array None
li_ma.kNIZZ8Jz.item string 6FUFQ4ky
li_ma.kNIZZ8Jz end_array None
li_ma map_key 0MDfFh7a
li_ma.0MDfFh7a start_array None
li_ma.0MDfFh7a.item string O6UCSlDc
li_ma.0MDfFh7a end_array None
li_ma map_key fKiJiEho
li_ma.fKiJiEho start_array None
li_ma.fKiJiEho.item string sXsvGxll
li_ma.fKiJiEho end_array None
li_ma map_key 4L2saLA4
li_ma.4L2saLA4 start_array None
li_ma.4L2saLA4.item string qLLFeF3Y
li_ma.4L2saLA4 end_array None
li_ma map_key jz1LxNwF
li_ma.jz1LxNwF start_array None
li_ma.jz1LxNwF.item string ycUWRGyu
li_ma.jz1LxNwF end_array None
li_ma map_key 30zzrnMn
li_ma.30zzrnMn start_array None
li_ma.30zzrnMn.item string KZKg7

li_ma.lCCHGkzb.item string lMawDTCZ
li_ma.lCCHGkzb.item string h8ibHLX7
li_ma.lCCHGkzb.item string GP7snbGm
li_ma.lCCHGkzb.item string PuRJ5IOY
li_ma.lCCHGkzb.item string g9WLOMZG
li_ma.lCCHGkzb.item string zDyH0aZu
li_ma.lCCHGkzb.item string 0uBmzB7T
li_ma.lCCHGkzb.item string 2I0uNg2O
li_ma.lCCHGkzb end_array None
li_ma map_key s9COBLra
li_ma.s9COBLra start_array None
li_ma.s9COBLra.item string 9ksneN3w
li_ma.s9COBLra.item string LUXmrjQm
li_ma.s9COBLra end_array None
li_ma map_key nMGRTz9t
li_ma.nMGRTz9t start_array None
li_ma.nMGRTz9t.item string OU5IFubj
li_ma.nMGRTz9t end_array None
li_ma map_key gGrrSCOP
li_ma.gGrrSCOP start_array None
li_ma.gGrrSCOP.item string ak5wv1BU
li_ma.gGrrSCOP end_array None
li_ma map_key KIpIEewx
li_ma.KIpIEewx start_array None
li_ma.KIpIEewx.item string sXKzV0kR
li_ma.KIpIEewx.item string 3AVw4XGh
li_ma.KIpIEewx.item string 64SYvKSK
li_ma.KIpIEewx end_array None
li_ma map_key 75y6fqfT
li_ma.75y6fqfT start_array None
li_ma.75y6fqfT.item string FSLh82eR

li_ma.iErj6ZWK.item string 2WRV4Rwf
li_ma.iErj6ZWK end_array None
li_ma map_key bZzvIBX4
li_ma.bZzvIBX4 start_array None
li_ma.bZzvIBX4.item string CI7u9shW
li_ma.bZzvIBX4.item string UDhkvSWu
li_ma.bZzvIBX4.item string 91GrsymO
li_ma.bZzvIBX4 end_array None
li_ma map_key Iz6rNhkD
li_ma.Iz6rNhkD start_array None
li_ma.Iz6rNhkD.item string FnxPxXaS
li_ma.Iz6rNhkD end_array None
li_ma map_key fUPyWIEo
li_ma.fUPyWIEo start_array None
li_ma.fUPyWIEo.item string gJHXp2r3
li_ma.fUPyWIEo end_array None
li_ma map_key M0dYw6gD
li_ma.M0dYw6gD start_array None
li_ma.M0dYw6gD.item string FCriPa6t
li_ma.M0dYw6gD end_array None
li_ma map_key Ux4ft2hZ
li_ma.Ux4ft2hZ start_array None
li_ma.Ux4ft2hZ.item string JtIn6Kzf
li_ma.Ux4ft2hZ end_array None
li_ma map_key g0st02Ws
li_ma.g0st02Ws start_array None
li_ma.g0st02Ws.item string P8uNMCcn
li_ma.g0st02Ws.item string BfKVlFgh
li_ma.g0st02Ws.item string QkTH96xH
li_ma.g0st02Ws.item string jROqSkOq
li_ma.g0st02Ws.item string b9au0rcv
li_ma.g0st02Ws.item st

...

xi_zhang.pu9vsB65.item string 8piROJBX
xi_zhang.pu9vsB65 end_array None
xi_zhang map_key pK7E5I8H
xi_zhang.pK7E5I8H start_array None
xi_zhang.pK7E5I8H.item string kSFZb55s
xi_zhang.pK7E5I8H end_array None
xi_zhang map_key 1JDUSADq
xi_zhang.1JDUSADq start_array None
xi_zhang.1JDUSADq.item string adYVYAPw
xi_zhang.1JDUSADq end_array None
xi_zhang map_key 1WxP9NEg
xi_zhang.1WxP9NEg start_array None
xi_zhang.1WxP9NEg.item string DL06rgsI
xi_zhang.1WxP9NEg.item string XRoR9RvN
xi_zhang.1WxP9NEg.item string AJIoX0BD
xi_zhang.1WxP9NEg end_array None
xi_zhang map_key XrBmomwO
xi_zhang.XrBmomwO start_array None
xi_zhang.XrBmomwO.item string wZKXKTrL
xi_zhang.XrBmomwO end_array None
xi_zhang map_key nOgW2bhe
xi_zhang.nOgW2bhe start_array None
xi_zhang.nOgW2bhe.item string ZpKm2cMM
xi_zhang.nOgW2bhe end_array None
xi_zhang map_key 1oOolV4F
xi_zhang.1oOolV4F start_array None
xi_zhang.1oOolV4F.item string X6BTAJns
xi_zhang.1oOolV4F.item string xLaYLBPJ
xi_zhang.1oOolV4F.item string 3VmaOf2e
xi_zhan

xi_zhang.gC9VQ7vN end_array None
xi_zhang map_key FO3ag4gr
xi_zhang.FO3ag4gr start_array None
xi_zhang.FO3ag4gr.item string 313xkxnr
xi_zhang.FO3ag4gr end_array None
xi_zhang map_key KO7BIM0A
xi_zhang.KO7BIM0A start_array None
xi_zhang.KO7BIM0A.item string GOvhjUaa
xi_zhang.KO7BIM0A end_array None
xi_zhang map_key vRka164q
xi_zhang.vRka164q start_array None
xi_zhang.vRka164q.item string Se3mfB63
xi_zhang.vRka164q.item string Py5Lr0kg
xi_zhang.vRka164q.item string Ru1unbbx
xi_zhang.vRka164q.item string YKxTDpz9
xi_zhang.vRka164q end_array None
xi_zhang map_key Xrrm81tr
xi_zhang.Xrrm81tr start_array None
xi_zhang.Xrrm81tr.item string WH6C0wKU
xi_zhang.Xrrm81tr.item string I96mrQ4D
xi_zhang.Xrrm81tr.item string L8QJVp4Y
xi_zhang.Xrrm81tr end_array None
xi_zhang map_key RP4r7yW1
xi_zhang.RP4r7yW1 start_array None
xi_zhang.RP4r7yW1.item string pxqes7Gy
xi_zhang.RP4r7yW1 end_array None
xi_zhang map_key XDYI8nQE
xi_zhang.XDYI8nQE start_array None
xi_zhang.XDYI8nQE.item string jNuNzi74
xi_zhan

xi_zhang.rSFHeatm.item string NDNfucML
xi_zhang.rSFHeatm.item string lv4XAYap
xi_zhang.rSFHeatm end_array None
xi_zhang map_key 71ko9evX
xi_zhang.71ko9evX start_array None
xi_zhang.71ko9evX.item string RT2NuJDq
xi_zhang.71ko9evX end_array None
xi_zhang map_key wFtHMcCk
xi_zhang.wFtHMcCk start_array None
xi_zhang.wFtHMcCk.item string rcwNKD0i
xi_zhang.wFtHMcCk.item string 3P37hbys
xi_zhang.wFtHMcCk end_array None
xi_zhang map_key rh30XQzC
xi_zhang.rh30XQzC start_array None
xi_zhang.rh30XQzC.item string xil7FHsP
xi_zhang.rh30XQzC end_array None
xi_zhang map_key jJKDgS2g
xi_zhang.jJKDgS2g start_array None
xi_zhang.jJKDgS2g.item string 9W6jevqt
xi_zhang.jJKDgS2g end_array None
xi_zhang map_key ICaYl31d
xi_zhang.ICaYl31d start_array None
xi_zhang.ICaYl31d.item string H38SPvAU
xi_zhang.ICaYl31d end_array None
xi_zhang map_key xb0lhsWh
xi_zhang.xb0lhsWh start_array None
xi_zhang.xb0lhsWh.item string MBjwiko3
xi_zhang.xb0lhsWh end_array None
xi_zhang map_key FuvCGSxJ
xi_zhang.FuvCGSxJ start_ar

jie_zhou.GMiab1uG.item string Fqix2Yva
jie_zhou.GMiab1uG.item string sr7ecF6M
jie_zhou.GMiab1uG.item string FLT9qozm
jie_zhou.GMiab1uG.item string OFdPMQO0
jie_zhou.GMiab1uG.item string XYasJvDs
jie_zhou.GMiab1uG.item string MWa9kTYs
jie_zhou.GMiab1uG.item string u9aAn3V3
jie_zhou.GMiab1uG.item string rBA6GTat
jie_zhou.GMiab1uG.item string 3wkFJbKW
jie_zhou.GMiab1uG.item string 8KWiKic4
jie_zhou.GMiab1uG.item string YhCymhys
jie_zhou.GMiab1uG.item string OBhVz6NV
jie_zhou.GMiab1uG.item string KXqfmm2l
jie_zhou.GMiab1uG.item string bYmhyuUi
jie_zhou.GMiab1uG.item string aT8M7ciG
jie_zhou.GMiab1uG.item string KFYeiGfn
jie_zhou.GMiab1uG.item string nVwSAhly
jie_zhou.GMiab1uG.item string mTgHZJSN
jie_zhou.GMiab1uG.item string pShRrfSg
jie_zhou.GMiab1uG.item string M2KT1z5P
jie_zhou.GMiab1uG.item string TCfpAG61
jie_zhou.GMiab1uG.item string tOKw2LRY
jie_zhou.GMiab1uG.item string C128BHK4
jie_zhou.GMiab1uG.item string VmQ5Uss2
jie_zhou.GMiab1uG.item string xGNV4t2Z
jie_zhou.GMiab1uG.item st

KeyboardInterrupt: 

In [ ]:
json.loads

In [45]:
with open(path + '/na_v1_pub.json') as f:
    parser = ijson.parse(f)
    w = []
    k = None
    for x, y, z in parser:
        if k is not None:
            w += [(x, y, z)]
        if y == 'map_key' and k is None: k = z
        if y == 'end_map' and x == k: break
        
    
    objects = ijson.items(w, k)
    c = {k : objects.__next__()}
    print (x,y,z,parser)

56PyrvLA end_map None <_yajl2.parse object at 0x00000236AACDDC10>


In [46]:
c

{'56PyrvLA': {'authors': [{'name': 'Qingya Wang', 'org': 'Jilin Univ'},
   {'name': 'Li Ma', 'org': 'Jilin Univ'},
   {'name': 'Fanghai Zhao', 'org': 'Jilin Univ'},
   {'name': 'Wei Zheng', 'org': 'Jilin Univ'},
   {'name': 'Yushu Zhang', 'org': 'Jilin Univ'},
   {'name': 'Dingsan Gao', 'org': 'Jilin Univ'}],
  'title': 'Study on the fabrication technology of SnO2 microlens and its array by using excimer laser-CVD technique',
  'abstract': 'In this paper, a microlens with 45μm radius and its array were fabricated using liquid SnCl',
  'keywords': [''],
  'venue': 'Jiguang Jishu/Laser Technology',
  'year': 1996,
  'id': '56PyrvLA'}}

In [43]:
w[-10]

('hong_yang.mcyNZ9bQ.item', 'string', 'ihymAbYT')

In [40]:
w, k

([('li_ma', 'start_map', None),
  ('li_ma', 'map_key', 'MRb2KOMw'),
  ('li_ma.MRb2KOMw', 'start_array', None),
  ('li_ma.MRb2KOMw.item', 'string', '56PyrvLA'),
  ('li_ma.MRb2KOMw.item', 'string', 'YlCMdSUx'),
  ('li_ma.MRb2KOMw.item', 'string', 'JIKd7Vd8'),
  ('li_ma.MRb2KOMw.item', 'string', 'QgReN4u4'),
  ('li_ma.MRb2KOMw.item', 'string', 'g7lzsICE'),
  ('li_ma.MRb2KOMw.item', 'string', 'Nm9K7f15'),
  ('li_ma.MRb2KOMw.item', 'string', 'mizzjKfH'),
  ('li_ma.MRb2KOMw.item', 'string', '9U5h8EPJ'),
  ('li_ma.MRb2KOMw.item', 'string', 'RKKSV7Rf'),
  ('li_ma.MRb2KOMw.item', 'string', '59hRFwAG'),
  ('li_ma.MRb2KOMw.item', 'string', 'qRyU2VDx'),
  ('li_ma.MRb2KOMw.item', 'string', '15GL4Qbv'),
  ('li_ma.MRb2KOMw.item', 'string', 'NQgktZ8J'),
  ('li_ma.MRb2KOMw.item', 'string', 'kEVdx8XQ'),
  ('li_ma.MRb2KOMw.item', 'string', '2X5xipeW'),
  ('li_ma.MRb2KOMw.item', 'string', 'TfMAXCIu'),
  ('li_ma.MRb2KOMw.item', 'string', 'wefCfkpN'),
  ('li_ma.MRb2KOMw.item', 'string', 'IfP6liBY'),
  ('li_

# 测试kv

In [97]:
import io
ioo = io.BytesIO(b'{"a": 1, "a": 2, "c": 3}')

In [91]:
[*ijson.parse(ioo)]

[('', 'start_map', None),
 ('', 'map_key', 'a'),
 ('a', 'number', 1),
 ('', 'map_key', 'a'),
 ('a', 'number', 2),
 ('', 'map_key', 'c'),
 ('c', 'number', 3),
 ('', 'end_map', None)]

In [98]:
[*getJsonKv(ioo)]

<ipython-input-95-edb210164063> line 9 : x = '', y = 'start_map', z = None, k = None, pool = []
<ipython-input-95-edb210164063> line 9 : x = '', y = 'map_key', z = 'a', k = None, pool = [('', 'start_map', None)]
<ipython-input-95-edb210164063> line 12 : k = None, pool = [('', 'start_map', None), ('', 'map_key', 'a')]
<ipython-input-95-edb210164063> line 9 : x = 'a', y = 'number', z = 1, k = 'a', pool = []
<ipython-input-95-edb210164063> line 9 : x = '', y = 'map_key', z = 'a', k = 'a', pool = [('a', 'number', 1)]
<ipython-input-95-edb210164063> line 12 : k = 'a', pool = [('a', 'number', 1), ('', 'map_key', 'a')]
<ipython-input-95-edb210164063> line 9 : x = 'a', y = 'number', z = 2, k = 'a', pool = []
<ipython-input-95-edb210164063> line 9 : x = '', y = 'map_key', z = 'c', k = 'a', pool = [('a', 'number', 2)]
<ipython-input-95-edb210164063> line 12 : k = 'a', pool = [('a', 'number', 2), ('', 'map_key', 'c')]
<ipython-input-95-edb210164063> line 9 : x = 'c', y = 'number', z = 3, k = 'c',

[('a', 1), ('a', 2), ('c', 3)]

In [96]:
ioo.close()

In [66]:
import io
parse_events = ijson.parse(io.BytesIO(b'["skip", {"a": 1}, {"": 2}, {"c": 3}]'))


In [67]:
for x, y, z in parse_events:
    print (x,y,z)
    break
else:
    print ('hh')

 start_array None


In [42]:
c

{'li_ma': {'MRb2KOMw': ['56PyrvLA',
   'YlCMdSUx',
   'JIKd7Vd8',
   'QgReN4u4',
   'g7lzsICE',
   'Nm9K7f15',
   'mizzjKfH',
   '9U5h8EPJ',
   'RKKSV7Rf',
   '59hRFwAG',
   'qRyU2VDx',
   '15GL4Qbv',
   'NQgktZ8J',
   'kEVdx8XQ',
   '2X5xipeW',
   'TfMAXCIu',
   'wefCfkpN',
   'IfP6liBY',
   'x9ekozOb',
   'seUQKtEP',
   'Su2YHtqZ',
   'koKZgAaj',
   'mkkP5HIK',
   'IbSOY0yD',
   'UbFVzsoH',
   'GTZcD4s3',
   'M2Y7YXr7',
   'zUy6a6s2',
   'Mo0yq9ZL',
   'zUXRjM2E',
   'gMyY8mYt',
   'VTGFhfwY',
   'ZS46wH19',
   'kxrCUXnI',
   '5PFftOwC',
   'wF4UUjVz',
   'mmHJj8b5',
   'fZkacfTA',
   'zHnQ3NJT',
   'mMd5d0Ok',
   'Rw2xyY1C',
   'kLVlBFCp',
   'FHQ77pHb',
   '1ZFwXBKG',
   'xZCFOQIx',
   'Euqy7Lz7',
   'WXMkw3NL',
   'QxHatJYG',
   'XDV6VUnI',
   '0NZ9fW8j',
   'r0gKQPee',
   'vWzchd9A',
   '8MM4OxJ6',
   '6hAnwjG1',
   'e1n3Keye',
   'Llrt9nh3',
   'moPh8gbz',
   'VBQJtST3',
   'gShNFy98',
   'lVkjz8kp',
   'EkZ2n5EI',
   'kv1wFDW9',
   'Pc8FXAUm',
   '1CLfFz5c',
   'z145XorH',
   '

In [135]:
len("The Khanka Massif forms the easternmost segment of the Central Asian Orogenic Belt (CAOB) in NE China and its tectonic setting is highly controversial. Metasedimentary rocks from the Hulin Complex and the Mt. Huoshi sequence were selected to address this issue. For the Hulin Complex, SHRIMP detrital zircon U–Pb dating of a two-mica schist (sample HL-011) reveals a unimodal population with a weighted mean 206 Pb/ 238 U age of 257 ± 3 Ma, suggesting local derivation from nearby Permian granitoids. In contrast, a quartz schist (sample 04H-138) contains five zircon populations with ages of 233–270 Ma, 350–400 Ma, 473–570 Ma, 720–760 Ma and 930–970 Ma, and peaks at 252, 350, 520, 758 and 970 Ma, indicating a much more diverse provenance. Two samples of fine-grained gneiss were selected from Mt Houshi, with sample 06H-15 containing four zircon populations, with groupings at 260–340 Ma, 470–560 Ma, 710–940 Ma and a single grain at 2090 Ma, whereas zircons in sample 06H-16 define two populations with ages of 727–854 Ma, with a peak at 776 Ma, and a single grain with an age of 404 ± 13 Ma. The youngest detrital zircon was obtained from sample 04H-138 and has a 206 Pb/ 238 U age of 233 ± 9 Ma, limiting deposition of the Hulin Complex to after ∼233 Ma. The late Pan-African and Paleoproterozoic to Neoproterozoic ages, together with the ∼500 Ma age for granulite facies metamorphism previously determined for the Hutou Complex in the Khanka Massif, indicate that the Khanka and Jiamusi massifs (together with the Bureya Massif in Russia) formed a contiguous crustal unit. Detrital zircon data suggest that this combined block was not part of either the North China or the South China cratons. Instead, it was either derived from a peri-Gondwana position, a rifted portion of the Songliao Massif in the CAOB, an exotic block of unknown affinity, or from the Siberia Craton. We evaluate the possibility that the Khanka/Jiamusi/Bureya block may have originated in Siberia and once formed part of the Sayan-Baikal orogen at ∼500 Ma. Whatever its origin, the block drifted westward to collide with the Songliao Massif of NE China in the latest Triassic-Early Jurassic as a result of the onset of Pacific-plate subduction. It thus forms part of the evolving supercontinent of Eurasia. Keywords Central Asian Orogenic Belt Khanka Massif NE China Russian Far East Gondwana Late Pan-African SHRIMP U–Pb dating of detrital zircon 1 Introduction One of the main problems in reconstructing the Neoproterozoic to Phanerozoic history of the Central Asian Orogenic Belt (CAOB) is the paucity of reliable paleomagnetic data, especially from the older components of several micro-continental blocks now located between the North China Craton to the South and the Siberia Craton to the north (see Fig. 1 and Wilde et al., 1999 ). This is particularly evident for the extreme eastern margin of the CAOB, where the ongoing effects of Pacific plate subduction since the Mesozoic complicate the situation. This lack of constraint has led to widely divergent views as to the origin and initial location of these blocks. In recent years, an ever-growing geochronological data base, combined with paleontological, geochemical and petrological information, has enabled first-order global comparisons to be made, but they have done little to resolve the fundamental issue of where the blocks were located at various stages during their history. Indeed, the new data have commonly led to an even wider range of possible connections. Three micro-continental blocks make up the extreme eastern part of the CAOB: the Bureya, Jiamusi and Khanka massifs ( Fig. 1 ). They have variously been considered to form separate entities or else to form one coherent block, albeit dissected by younger NE-trending faults ( Natal’in and Borukayev, 1991; Natal’in, 1991, 1993; Cao et al., 1992; Sengör et al., 1993; Sengör and Natal’in, 1996; Wilde et al., 1997, 1999, 2000, 2003, in press; Wu et al., 2007 ). The Jiamusi Massif was the first to be investigated in any detail ( Wilde et al., 1997, 1999, 2000, 2003 ) and, with preliminary geochronological data now available for the southern part of the Bureya Massif ( Wilde et al., 2008 ) and recent detailed studies undertaken on the Khanka Massif ( Wilde et al., in press; Zhou et al., in press ), it appears likely that they indeed form one contiguous crustal unit. They are separated from the Songliao Massif to the west by the Yanji-Heilongjiang Fault (also referred to as the Mudanjiang Fault— Wu et al., 2007 ) and from Jurassic and Cretaceous accretionary complexes to the east by extensive areas of no exposure. Along the contact between the Jiamusi and Songliao blocks lies the Heilongjiang Complex, a blueschist-facies accretionary complex recently interpreted as the suture zone along which the two blocks came together in the Late Triassic-Early Jurassic ( Wu et al., 2007; Zhou et al., 2009 ). As an illustration of the diversity of opinion regarding the nature, origin and tectonic significance of units in the eastern CAOB, the Heilongjiang Complex has also been interpreted as an extension of the Triassic Qingling-Dabie-Sulu HP-UHP suture zone that marks the collision between the North China Craton (NCC) and South China Craton (SCC). It was renamed the Qingling-Dabie-Tanlu-Sulu-Imjiangang-Yanji zone ( Zhang, 1997, 2004; Zhang et al., 2006 ) and this interpretation was supported by Oh (2006) and Oh et al. (2006, 2009) based on an evaluation of the Korean data. Ishiwatari and Tsujimori (2001, 2003) further suggested that the Yanji zone could be linked with the Heilongjiang blueschist belt (Yanji-Heilongjiang HP belt; see Figs. 1 and 2 a) . The implication of such a scenario is that the Khanka, Jiamusi and Bureya massifs would have initially formed part of the SCC. Interestingly, Shao and Tang (1995) , Shao et al. (1995) and Ren et al. (1999) had previously argued that the Khanka Massif be considered exotic to NE China, since it contains mixed Permian faunas, characterized by both cool-temperate elements and paleo-tropical Cathaysian taxa ( Tang, 1990; Tang et al., 1995; Shao and Tang, 1995; Shao et al., 1995; Ren et al., 1999 ). It thus becomes imperative to determine which, if any, of these scenarios is correct. One approach is to examine the provenance data provided by detrital zircons within the sedimentary sequences that form a component of each of these crustal blocks. This is particularly pertinent in this case, since it is well established that there are significant differences between the North and South China cratons. Indeed, the basement rocks of the NCC and SCC underwent very different thermo-magmatic histories before they finally came together in the Early Mesozoic (e.g., Zheng et al., 2003, 2005; Grimmer et al., 2003; Hacker et al., 2006; Ratschbacher et al., 2006; Zhou et al., 2008a ). In the North China Craton, Precambrian basement rocks are dominated by Neoarchean granitoids and orthogneisses (mostly tonalite-trondhjemite-granodiorite—TTG) and Paleoproterozoic metasedimentary sequences (e.g., Zhao et al., 2000, 2002, 2005; Wilde and Zhao, 2005; Zhai et al., 2005; Wan et al., 2006; Santosh et al., 2007a; Zhou et al., 2008b ). Protolith ages for the North China Craton orthogneisses and granitoids are mostly in the range of 2900–2500 Ma ( Zhao et al., 2000, 2002, 2005; Zhai et al., 2005; Wilde and Zhao, 2005; Zhou et al., 2008b ), and they underwent regional amphibolite- to granulite-facies metamorphism between 1900 and 1800 Ma ( Zhao et al., 2000; Santosh et al., 2006, 2007b; Zhou et al., 2008b ). In contrast, the SCC is predominantly characterized by young basement of Meso- to Neoproterozoic age that was subjected to major thermal events during the Siboan (1100–1000 Ma) and the Jinningian-Chengjiangian (850–700 Ma) ( Li et al., 2002, 2003; Zheng et al., 2003, 2005; Zhou et al., 2008c ). Although recent ion-microprobe U–Pb zircon studies of orthogneisses and granitoids in the Khanka Massif have established its similarity to the Jiamusi block ( Wilde et al., in press; Zhou et al., in press ), provenance data for the metasedimentary rocks have only been obtained from the Hutou Complex ( Wilde et al., in press; Zhou et al., in press ) and no data are currently available from the main outcrops at Hulin ( Fig. 2 ). In this paper, we undertake the first specific U–Pb detrital zircon study on the metasedimentary rocks of the Hulin Complex. This will enable us to compare the data not only with the nearby Hutou Complex and adjacent Jiamusi Massif, but to place the results within a broader regional context, based on recently published studies in the CAOB, including the Songliao Massif to the west and the southern margin of the Siberia Craton to the north. This will allow us to evaluate whether the combined Khanka-Jiamusi-Bureya block has an affinity to the NCC or SCC, whether it is related to other components of the CAOB, if it is an exotic block possibly derived from Gondwana, or whether it was derived from Siberia. 2 Geological setting NE China and adjacent regions, including the Russian Far East, have traditionally been considered as the eastern part of the Central Asian Orogenic Belt (CAOB), located between the Siberia and North China cratons ( Natal’in, 1991, 1993; Natal’in and Borukayev, 1991; Sengör et al., 1993; Sengör and Natal’in, 1996; Ren et al., 1999; Jahn et al., 2000; 2004; Xiao et al., 2003, 2004a,b; Li, 2006 ). The area has been interpreted as a collage of several micro-continental blocks ( Tang, 1990; Tang et al., 1995; Li et al., 1999; Li, 2006; Xiao et al., 2003, 2004a,b; Wu et al., 2007; Wilde et al., in press ), including the Khanka, Jiamusi and Bureya massifs, the Nadanhada terrane to the northeast and the Songliao Massif to the west ( Figs. 1 and 2 a). The Nadanhada terrane is located to the northeast of the Khanka Massif ( Fig. 2 a) and forms part of the Late Jurassic-Early Cretaceous Paleo-Pacific orogenic belt, being composed of Jurassic clastic rocks containing tectonic lenses of Middle Jurassic shale, Late Triassic chert and mafic igneous rocks, all intruded by Cretaceous granitoids ( Kojima, 1989; Cheng et al., 2006 ). The granites were emplaced between 131 and 115 Ma ( Cheng et al., 2006 ), whereas gabbro in the Raohe Complex was emplaced at 166 ± 1 Ma. The Nadanhada terrane was thus assembled between 165 and 130 Ma in the Late Jurassic-Early Cretaceous ( Kojima, 1989; Cheng et al., 2006 ), and in this respect is similar to the Sikhote-Alin complex that lies farther to the east in the Russian Far East ( Fig. 2 a) ( Kojima, 1989; Zyabrev and Matsuoka, 1999 ) and the Tamba-Mino-Ashio terrane in Japan ( Kojima, 1989 ): these terranes all being related to Pacific plate subduction. The Songliao Massif is located to the west of the Khanka-Jiamusi-Bureya massifs ( Fig. 2 a) and is poorly exposed, being mostly covered by the Songliao sedimentary basin. Petrographic examination of several hundred drill cores indicates that the basement rocks are dominantly Paleozoic granitic and sedimentary rocks ( Wu et al., 2000 ). Detrital zircons from these metasedimentary rocks reveal a range in U–Pb and Pb–Pb ages from 406 to 2208 Ma ( Wang et al., 2007 ). In particular, a sample of quartz schist from a drillhole to the southeast of the Songliao Basin reveals several detrital zircon populations with ages of ∼520 Ma, 1000–1100 Ma, 1400–1500 Ma and 1700–1800 Ma ( Wang et al., 2007 ). These indicate that the basement of the Songliao Basin has a complex provenance, with the ∼520 Ma zircons possibly derived from the Jiamusi Massif ( Wang et al., 2007 ). The eastern and northern part of the Songliao Massif is defined by the Zhangguangcai and Lesser Xing’an Ranges, respectively, which are characterized by voluminous syn-collisional Mesozoic granitoids. These have been interpreted to have resulted from westward subduction of the Jiamusi Massif beneath the Songliao block during the Mesozoic at between ∼215 and 185 Ma ( Wu et al., 2003, 2007 ). The Jiamusi Massif, forming the northern extension of the Khanka Massif ( Wilde et al., in press; Zhou et al., 2009, in press ), is dominated by three main rock associations: the granulite-facies Mashan Complex, which was metamorphosed in the Early Paleozoic at ∼500 Ma ( Wilde et al., 1997, 1999, 2000 ), Early and Late Paleozoic granitoids and volcanics ( Li et al., 1999; Li, 2006; Wu et al., 2001, 2007; Wilde et al., 2003 ) and the Heilongjiang Complex, an accretionary wedge located along the western margin of the Jiamusi Massif (see Wu et al., 2007; Zhou et al., 2009 ). The Mashan Complex is a khondalitic sequence characterized by a tight clockwise P/T path ( Jiang, 1992; Lennon et al., 1997 ), with peak temperatures up to 850 °C and pressures of 7.4 kb ( Jiang, 1992 ). Amphibolite facies rocks predominate in the northern outcrops and reached peak temperatures of ∼650 °C (garnet-biotite) ( Cao et al., 1992 ). Two stages of granitoids have been identified; the metamorphosed and deformed granites associated with the Mashan Complex have emplacement ages of ∼520 Ma ( Wilde et al., 1999, 2003 ), whereas the younger granitoids are undeformed and of Permian age ( Wilde et al., 1997 ). The Khanka Massif is located to the south of the Jiamusi Massif from which it is separated by the Dunhua–Mishan Fault, whereas it is separated from the Songliao Massif to the west by the Yanji Suture ( Jia et al., 2004 ) ( Figs. 1 and 2 a). It is composed of Precambrian metamorphosed basement rocks, with a cover of Paleozoic to Mesozoic strata that includes carbonates, clastic sediments, and volcanic rocks ( Natal’in, 1993; Shao and Tang, 1995; Shao et al., 1995; Jia et al., 2004; Shi, 2006; Shi and Zhan, 1996 ). The Chinese segment of the Khanka Massif crops out in only three small areas ( Fig. 2 b), with exposures concentrated in the Hutou Complex in the east and the Hulin Complex in the west, with rocks exposed between these at Mt. Huoshi ( Fig. 2 b) considered to be more closely related to the Hutou Complex ( HBGMR, 1993 ). During regional mapping, the Hutou Complex was interpreted to be related to the Mashan ‘group’ in the Jiamusi Massif and believed to be of late Archean age, whereas the Hulin Complex was thought to be related to the Heilongjiang ‘group’ of the Jiamusi Massif and considered to be Mesoproterozoic ( HBGMR, 1993 ). A westward decrease in metamorphic grade from granulite to amphibolite facies was also recognized. The granulite facies rocks are found mostly near Hutou and consist of a khondalitic sequence of graphite-, sillimanite- and cordierite-bearing gneisses, carbonates and felsic paragneisses, with more than 70% of the outcrops composed of sillimanite- and garnet-bearing granite. Amphibolite facies rocks are predominant at Mt. Huoshi ( Fig. 2 b) and consist of felsic paragneiss and hornblende schist. The rocks in the Hulin area ( Fig. 2 b) are predominantly at greenschist facies and consist of quartz schist, quartz- muscovite schist and biotite-muscovite (two-mica) schist. In this study, we focused on obtaining precise SHRIMP U–Pb detrital zircon data for metasedimentary rocks of the Hulin Complex, since no previous studies had been undertaken. In addition, we also obtained samples from Mt. Huoshi in order to test its affinity with the Hutou Complex, as suggested from previous studies ( HBGMR, 1993; Zhao et al., 1995; Wilde et al., in press; Zhou et al., in press ). 3 Sample locations and descriptions Two samples from the Hulin Complex and two samples from Mt. Huoshi, considered to be related to the Hutou Complex, were chosen for analysis ( Fig. 2 b). 3.1 Hulin Complex 3.1.1 Two-mica schist (HL011) A sample of two-mica schist (HL011) was collected from the metasedimentary sequence of the Hulin Complex, about 5 km northeast of Hulin (Lat. 45°46′30.4″N; Long. 133°2′3.3″E; Fig. 2 b). It consists of recrystallized quartz grains (70–75%) with serrated margins, plagioclase (<10–15%), biotite (8–10%), muscovite (5–8%) and trace amounts of opaque minerals ( Fig. 3 a) . The rock is foliated, defined by aligned plagioclase, biotite and muscovite. Muscovite flakes are poikiloblastic, with inclusions of quartz and plagioclase ( Fig. 3 a). Accessory tourmaline and zircon are also present. 3.1.2 Quartz schist (04H138) A sample of quartz schist (04H138) was obtained from the metasedimentary sequence immediately east of Hulin (Lat. 45°46′15.7″N, Long. 133°1′5.0″E; Fig. 2 b). It is composed of quartz (85–90%), plagioclase (5–10%) and muscovite (1–10%), with minor amounts of biotite, magnetite, monazite and zircon. The quartz is recrystallized to a granular mosaic ( Fig. 3 b) and, together with plagioclase, ranges from 1 to 5 mm in diameter. The muscovite is generally subidiomorphic and ranges from 0.5 to 1 mm in length. It is present at grain boundaries and defines the schistosity. 3.2 Hutou Complex 3.2.1 Fine-grained gneiss (06H-15) Sample 06H-15 is a fine-grained gneiss collected at Mt. Huoshi (Lat. 45°52′36.5″N, Long. 133°16′55.9″E; Fig. 2 b), approximately 35 km west-southwest of Hutou from an area mapped as Liumao “formation” of the Mashan “group” ( HBGMR, 1993 ). The sequence is strongly deformed and composed of marble and fine-grained gneiss, cut by later granitic veins. Sample 06H-15 has a mineral assemblage of clinopyroxene (10–15%), hornblende (5–8%), quartz (45–50%), K-feldspar (35–40%) ( Fig. 3 c), with accessory apatite, titanite and zircon. The conversion of clinopyroxene to hornblende is common in rocks from this outcrop and has been interpreted to result from later metamorphism ( HBGMR, 1993 ). A sample collected from an adjacent outcrop (FW04-210) was analyzed by Wilde et al. (in press) . 3.2.2 Fine-grained gneiss (06H-16) Another sample of fine-grained gneiss (06H-16) was collected at Mt. Huoshi, about 2 km west of sample 06H-15 (Lat. 45°52′38.8″N, Long. 133°16′49.9″E; Fig. 2 b), where it is associated with quartzite and fine-grained biotite-rich gneiss. Similar to sample 06H-15, it has a mineral assemblage of clinopyroxene (10–15%), hornblende (5–8%), quartz (35–40%) and K-feldspar (40–45%) ( Fig. 3 d), with accessory apatite, titanite and zircon. 4 SHRIMP U–Pb analytical techniques The four samples were processed by crushing in a tema ring mill, followed by initial heavy liquid and subsequent magnetic separation. Samples were divided into different size and magnetic fractions using a Frantz isodynamic separator. Zircons from the non-magnetic fractions were handpicked and mounted, along with pieces of the CZ3 zircon standard, onto adhesive tape, enclosed in epoxy resin and then polished to about half their thickness. The mount was then cleaned and gold-coated and photographed in reflected and transmitted light. Cathodoluminescence (CL) imaging of zircon grains was carried out using a Philips XL30 scanning electron microscope (SEM) at Curtin University of Technology. The U–Th–Pb analyses were conducted using the WA Consortium SHRIMP IIA ion microprobe housed at Curtin University of Technology. Detailed analytical procedures are described by Nelson (1997) and Williams (1998) . Isotopic ratios were monitored by reference to Sri Lankan gem zircon standard (CZ3) with a 206 Pb/ 238 U ratio of 0.0914 that is equivalent to an age of 564 Ma. Pb/U ratios in the unknown zircons were corrected using the ln (Pb/U)/ln (UO/U) relationship as measured on CZ3. Spot size ranged between 25 and 30 μm during analysis and each spot was rastered over 140 μm for 3 min prior to analysis to remove common Pb on the surface or contamination from the gold coating. A total of 7 scans through the mass stations were used for sample HL011, whereas for the remaining three samples, 5 scans were utilized. One standard was run for every 5 unknowns during the analytical sessions. The uncertainty associated with measurement of Pb/U isotopic ratios for the standard at 1 σ was ∼2% during all sessions. All ages were calculated using the U and Th decay constants recommended by Steiger and Jäger (1977) . Reported ages >1000 Ma represent the 207 Pb/ 206 Pb data and ages <1000 Ma are quoted as 206 Pb/ 238 U ages, based on the fact that young grains provide low count rates on the 207 Pb peak, thus increasing the uncertainties ( Black and Jagodzinski, 2003; Black et al., 2003 ); both data sets were corrected using the measured 204 Pb ( Compston et al., 1992 ). The analytical data were reduced, calculated and plotted using the Squid (1.0) and IsoplotEx 2.46 programs ( Ludwig, 2001a,b ), with the uncertainties on the standard included in the calculations. Individual analyses in the data table and concordia plots are presented with 1 σ error and uncertainties in weighted mean ages are quoted at the 95% confidence level (2 σ ), unless otherwise indicated. 5 SHRIMP U–Pb zircon data 5.1 Hulin Complex 5.1.1 Two-mica schist (sample HL-011) Zircons from sample HL-011 are colorless, transparent, and subhedral to euhedral in shape, although some are slightly rounded. They range from ∼80 to 120 μm in length, with length:width ratios of 2:1 to 3:1. Cathodoluminescence imaging reveals that most grains have weak oscillatory or planar zoning ( Fig. 4 a) , indicating a magmatic origin. A total of 12 analyses were made on 12 zircons ( Table 1 ) and they have U and Th contents and Th/U ratios ranging from 65 to 284 ppm, 19 to 406 ppm and 0.18 to 0.47 ppm, respectively. All but one of the analyses are concordant ( Fig. 4 b) and define a weighted mean 206 Pb/ 238 U age of 257 ± 3 Ma (MSWD = 0.95). The oscillatory zoning with moderate Th/U ratios ( Table 1 ) suggest an igneous origin and the age of 257 ± 3 Ma is identical to that of Permian granitoids previously analyzed from the southern part of the Jiamusi Massif (264 ± 11 Ma) by Wilde et al. (1997) . 5.1.2 Quartz schist (sample 04H-138) Sample 04H-138 yielded abundant small zircon grains, ranging from 50 to 200 μm in size, some of which are rounded to subhedral in form. Cathodoluminescence imaging reveals that two kinds of zircon domains are present: oscillatory zoned portions and darker or lighter rims that contain higher U concentrations than the relatively lighter grey cores ( Table 1 ; Fig. 5 a) . A total of 50 analyses were made on 50 rounded grains and 17 analyses show discordance less than 10%, but with most analyses plotting along concordia ( Fig. 5 b; Table 1 ). The grains show great variation in U and Th contents and Th/U ratios, with respective ranges of 32–1814 ppm, 1–1397 ppm and 0.00–3.20. Broadly, the grains fall into five age populations with 206 Pb/ 238 U ages ranging from 233 to 270 Ma, 350 to 400 Ma, 473 to 570 Ma, 720 to 760 Ma and 930 to 970 Ma, with peaks at 252, 350, 520, 758 and 970 Ma, respectively ( Fig. 5 b). The majority of zircons in the age populations of 350–400, 473–570, 720–760 and 930–970 Ma are oscillatory zoned and record the igneous ages of their source rocks. The exception to this is within the 473–570 Ma group, where one of the sites (138–41.1; Table 1 ) was located on a rim and the U and Th contents and Th/U ratios are 301 ppm, 1 ppm and 0.003, respectively. The rim has a 206 Pb/ 238 U age of 520 ± 7 Ma which is therefore interpreted as the time of high-grade metamorphism of the source rocks, prior to incorporation in the quartz schist protolith. The youngest group of ages from 233 to 270 Ma are from zircons with high Th/U ratios and most likely record the igneous ages from a younger suite of source rocks. 5.2 Hutou Complex 5.2.1 Fine-grained gneiss (06H-15) Sample 06H-15 yielded abundant small zircon grains, ranging from 80 to 200 μm in size, most of which are rounded and anhedral. Cathodoluminescence imaging reveals that the grains contain two zircon domains: oscillatory zoned cores and darker rims that contain higher U concentrations ( Fig. 6 a) . A total of 57 analyses were made on 56 rounded grains and 28 analyses show discordance less than 10%, although the majority of analyses plot along concordia ( Fig. 6 b). The grains show great variation in U and Th contents and Th/U ratios, with respective ranges of 60–4288 ppm, 12–1938 ppm and 0.01–1.24. The 28 concordant analyses yield 206 Pb/ 238 U (when <1000 Ma) and 207 Pb/ 206 Pb (when >1000 Ma) ages ranging from 259 to 2090 Ma. Broadly, the grains fall into four age populations, with groupings at 260–340 Ma, 470–560 Ma, 710–940 Ma and a single grain at 2090 Ma, respectively ( Table 1 ; Fig. 6 b). The single grain with a 207 Pb/ 206 Pb >2000 Ma indicates the presence of Paleoproterozoic rocks in the source area of the Hulin Complex. The age population of 710–940 Ma is from oscillatory zoned zircons and indicates the igneous ages of the source rocks, as does the 470–560 Ma group, with a peak at 494 Ma. However, within the 470–560 Ma group, one of the ages (15-4; Table 1 ) was obtained from a rim and the U and Th contents and Th/U ratios were 854 ppm, 12 ppm and 0.01, respectively. The rim 206 Pb/ 238 U age of 474 ± 15 Ma is thus interpreted to reflect the time of high-grade metamorphism of the source rocks, prior to incorporation in the gneiss protolith. The youngest age group of 260–340 Ma is from oscillatory zoned zircons with high Th/U ratios and thus records the igneous ages of the source rocks, which limits the maximum possible deposition age to later than 259 ± 8 Ma. 5.2.2 Fine-grained gneiss (06H-16) This sample yielded abundant small zircon grains, ranging from 80 to 200 μm in size, most of which are subhedral and weakly elongate. Cathodoluminescence imaging reveals that most grains show dark rims that contain higher U concentrations than the relatively lighter cores that only rarely show distinct oscillatory zoning ( Fig. 7 a) . A total of 25 analyses were made on 25 grains and only 5 analyses show discordance of less than 10%, although most grains lie on or near concordia ( Fig. 7 b). The total grain population shows great variation in U and Th contents and Th/U ratios, with respective ranges of 108–1872 ppm, 58–1279 ppm and 0.13–1.18. The 5 concordant analyses yield 206 Pb/ 238 U ages ranging from 404 to 854 Ma that can be subdivided into two groups. The population between 727 and 854 Ma, with a peak at 776 Ma, includes oscillatory zoned zircon and therefore records the igneous ages of the source rocks. The youngest concordant age of 404 ± 13 Ma is from a zircon with a high Th/U ratio and planar zoning ( Fig. 7 a) and thus records the igneous age of the youngest source rocks. 6 Discussion and tectonic implications 6.1 Age of the Hulin Complex The Hulin Complex was previously assigned to the Heilongjiang ‘group’ in the Chinese literature ( HBGMR, 1993 ), based on similarities to the high-pressure (HP) rocks of the Jiamusi Massif (the Heilongjiang Complex; Wilde et al., 1997 ) that lie at the contact of the Jiamusi and Songliao-Zhangguangcai blocks ( Fig. 1 ; Fig. 2 a) ( Zhang, 1991; Li et al., 1999; Li, 2006; Wu et al., 2000, 2001, 2003, 2007; Wilde et al., 1997, 1999, 2000, 2003 ). However, as indicated earlier, the timing of formation, metamorphism and the tectonic setting of the Heilongjiang Complex has long been controversial, with interpretations ranging from an Archean to Proterozoic granite-greenstone terrane ( Liu, 1988; Zhao et al., 1995 ), Neoproterozoic metamorphic belt (e.g., Dang and Li, 1993 ), early Paleozoic mélange along the eastern segment of the Paleozoic Central Asian Orogenic Belt ( Liou et al., 1989a,b; Zhang, 1991; Cao et al., 1992; Li et al., 1999; Li, 2006 ), Triassic HP metamorphic belt representing the northeast extension of the Dabie-Sulu-Yanji HP-UHP belt ( Zhang, 1997, 2004; Ishiwatari and Tsujimori, 2003; Oh, 2006; Oh et al., 2006; Sajeev et al., 2010 ), and a Jurassic ophiolite of the Paleo-Pacific orogenic belt ( Wu et al., 2007 ). The zircons in sample HL-011, a two-mica schist from the metasedimentary sequence at Hulin, appear to be derived from a single source, with a weighted mean age of 257 ± 3 Ma. The oscillatory zoning and moderate Th/U ratios (Th/U = 0.26) of the zircons suggest that this records the age of the protolith of the detrital grains and is consistent with the age of Permian granitoids in the Jiamusi Massif ( Wilde et al., 1997 ). The age is thus interpreted as recording the time of emplacement of a granitoid protolith, indicating weathering and transportation at some later date. The youngest concordant 206 Pb/ 238 U zircon age of 255 ± 3 Ma ( Table 1 ) limits the deposition age to younger than 255 Ma, but does not provide any constraint on the actual time of sedimentation. The youngest detrital zircon from the quartz schist at Hulin (sample 04H-138) has a 206 Pb/ 238 U age of 233 ± 9 Ma, limiting deposition to after ∼230 Ma. The detrital zircon results thus indicate that these sediments must be younger than the Late Permian/Early Triassic. These results are also in agreement with recent work in the Heilongjiang Complex, where the presence of detrital zircons with ages of 210–250 Ma have been recorded ( Wu et al., 2007; Zhou et al., 2009 ). The results would suggest that the Hulin Complex, located at the western margin of the Khanka Massif is, indeed, equivalent to the Heilongjiang Complex that occupies a similar position in the Jiamusi Massif ( Zhou et al., 2009 ) 6.2 Age of the Hutou Complex Two fine-grained gneiss samples from an area mapped as part of the Hutou Complex ( HBGMR, 1993 ) at Mt. Huoshi have an amphibolite facies mineral assemblage of clinopyroxene + hornblende + quartz + K-feldspar. Zircons from sample 06H-15 show four age populations at: 260–340 Ma, 470–560 Ma, 710–940 Ma and 2090 Ma. The younger ages of 260–340 Ma are from zircons with high Th/U ratios and likely record the igneous ages of the source rocks, which limits the maximum possible depositional age to younger than 259 ± 8 Ma. The results are quite similar to the data obtained from a nearby rock of similar composition by Wilde et al. (in press) . Their results yielded clusters at 258 ± 5 and 504 ± 8 Ma, the latter specifically targeting the metamorphic rims. In addition, single detrital grains with ages of 609 ± 11, 771 ± 12 and 942 ± 19 Ma were present, broadly similar to the populations identified in the present study. However, one notable difference is in the interpretation of the age of the rock. Wilde et al. (in press) considered that the gneiss was similar to high-grade metamorphic rocks of the Mashan Complex in the Jiamusi Massif and that the age of 504 ± 8 Ma recorded the time of in situ metamorphism of the sedimentary protolith. They explained the 258 ± 5 Ma age population as the result of re-heating by Permian granites in the area, since the ages were only recorded from zircon rims ( Wilde et al., in press ). However, the present study has revealed several oscillatory zoned magmatic zircons with core ages ranging from 260 to 340 Ma: in view of this, we consider that the depositional age of these gneissic rocks is younger than 260 Ma. Concordant zircons from sample 06H-16 show two age populations, and the youngest age of 404 ± 13 Ma is from a zircon with a high Th/U ratio and records the igneous age of the source rock, thereby limiting the maximum possible deposition age to younger than Late Silurian. The Hutou Complex has previously been equated with the Mashan ‘group’ of the Jiamusi massif on the regional geological map and was considered to be late Archean in age ( HBGMR, 1993; Zhao et al., 1995 ). The granulite-facies rocks around Hutou are certainly similar to the typical outcrops of Mashan Complex near Mudanjiang ( Wilde et al., 1997, 1999 ), being composed of a khondalitic sequence of graphite-, sillimanite- and cordierite-bearing gneisses, interleaved tectonically with carbonates and felsic paragneisses. SHRIMP zircon U–Pb dating of a sillimanite gneiss and two sillimanite- and garnet-bearing granites from the Hutou Complex, indicate that these rocks underwent metamorphism ∼500 Ma ago ( Zhou et al., in press ). The SHRIMP 206 Pb/ 238 U age of 490 ± 4 Ma for metamorphism in the paragneiss and the magmatic ages of 511 ± 6 and 515 ± 8 Ma for emplacement of the granitoids ( Zhou et al., 2009 ) are not only similar to the dates of 491–501 Ma for the high-grade metamorphism and 518 ± 7 Ma for granitoid emplacement in the Hutou Complex obtained by Wilde et al. (in press) , but remarkably similar to dates obtained from similar rocks of the Mashan Complex in the Jiamusi Massif ( Wilde et al., 1997, 2000, 2003 ). These data clearly establish the affinity of the Hutou Complex with the Mashan Complex in the Jiamusi Massif. However, the amphibolite-facies rocks predominant at Mt. Huoshi ( Fig. 2 b) are felsic paragneisses and hornblende schists with SHRIMP U–Pb ages indicating deposition of the former after 260 Ma (sample 06H-15) and ∼400 Ma (sample 06H-16), based on this study. This indicates that the Mt. Huoshi sequence is not part of the Hutou Complex. Further work is required to determine its affinity. 6.3 Provenance of the metasedimentary rocks of the Khanka Massif A total of 55 of 144 analyses with discordance <10% from four samples in the Khanka Massif yield apparent ages ranging from 2038 to 233 Ma ( Fig. 8 a) . Broadly, the grains fall into four age populations according to their apparent 207 Pb/ 206 Pb ages (for >1000 Ma) and 206 Pb/ 238 U ages (for <1000 Ma) at: >2000 Ma, with the oldest age of 2090 Ma; 980–740 Ma, with a peak at 900 Ma; 570–335 Ma, with a peak at 360 Ma; and 280–233 Ma, with a peak at 274 Ma. The results show that overall deposition of these rocks was later than ∼233 Ma and that most of the detrital material came from early Paleozoic and Neoproterozoic source rocks. Ages in the range of 280–233 Ma partially match ages of 257–340 Ma recorded from mafic lavas, mafic and felsic dykes and gabbros in the Hegenshan ophiolitic complex in Inner Mongolia ( Fig. 1 ) ( Miao et al., 2008 ). However, the Hulin and Mt Huoshi samples also contain inherited zircon with ages of ∼335–400 Ma. Slightly younger ages in the range of 312–250 Ma ( Song et al., 1994 ) and detrital zircons in mica schist with an age of 321 ± 1 Ma ( Wu et al., 2007 ) have been recorded from Mudanjiang, in the adjacent Jiamusi Massif. The source of zircons with ages of 570–430 Ma is more problematical and, perhaps significantly, these were recorded in samples from both the Hulin and Mt. Huoshi areas. Peak ages of 480 Ma closely match those of zircons recorded from the nearby Mashan Complex. Wilde et al. (1997, 2000) reported SHRIMP zircon U–Pb ages of 505 ± 4 and 497 ± 5 Ma for sillimanite gneiss and 515 ± 8 and 523 ± 8 Ma for deformed granitoid from the Mashan Complex, but ages as young as 430 Ma have not previously been recorded from the area. However, Jian et al. (2008) reported more than 10 samples with SHRIMP zircon U–Pb ages ranging from 416 ± 1 to 452 ± 3 Ma at Barer and 425 ± 2 to 480 ± 2 Ma at Tulinkai in Inner Mongolia within the CAOB ( Fig. 1 ). Also, Salnikova et al. (1998) have reported ages of 488 ± 0.5 Ma for the emplacement of TTG and 478 ± 2 Ma for subsequent granulite-facies metamorphism, respectively, in the Slyudyanka complex in the southwestern Baikal region, at the southern margin of the Siberia Craton ( Fig. 1 ). This was followed by post-tectonic emplacement of quartz syenite at 471 ± 2 Ma. The age population of 980–740 Ma is also recorded from both the Hulin and Mt. Huoshi areas. Similarly, zircons of this age have been reported as detrital grains in granulite-facies metasedimentary rocks of the Mashan Complex in the Jiamusi Massif ( Wilde et al., 1997, 2000 ). Stern (2008) has summarized the Neoproterozoic history of the CAOB, including evidence of Neoproterozoic igneous activity found in ophiolites, fragments of continental-margin and rift-related magmatic belts, bimodal volcanic associations, mafic intrusions, and granites (850–700 Ma). Neoproterozoic igneous rocks are also associated with rifts along the southern and southwestern margins of the Siberia Craton. Vernikovsky et al. (2003) have also identified three main Neoproterozoic tectonic events along the western margin of the Siberia Craton in forming the Yenisey Ridge fold-and-thrust belt: at 880–860 Ma, 760–720 Ma and 700–630 Ma. Khain et al. (2002) and Kuzmichev et al. (2005) proposed that Mesoproterozoic to Neoproterozoic crust (1.1–0.8 Ga) was produced by subduction-related magmatism. These data establish that many of the blocks in the CAOB record Neoproterozoic tectonic events. The detrital zircon age of ∼2090 Ma in sample 06H-15 from Mt. Huoshi is older than any previously recorded from either the Khanka or Jiamusi blocks; the previous oldest being ca. 1800 Ma in the Mashan Complex ( Wilde et al., 2000 ). 6.4 Relations between the Khanka (-Jiamusi) block and the NCC and SCC The data from detrital zircons are important for constraining the relationships between the massifs in NE China and placing greater certainty when reconstructing their affinity. The age spectra also give valuable information about zircon-forming events in the source area. As indicated above, some models suggest that the Khanka (and by association, the Jiamusi and Bureya) Massif could have formed part of the SCC, implying that the HP Heilongjiang Complex is a continuation of the Dabie-Sulu belt ( Zhang, 1997, 2004; Zhang et al., 2006 ). Such a scenario would also mean that the Hutou Complex has an SCC affinity. As discussed earlier, the basement rocks of the NCC and SCC underwent very different thermo-magmatic histories before the Mesozoic ( Zheng et al., 2003, 2005; Grimmer et al., 2003; Hacker et al., 2006; Ratschbacher et al., 2006; Zhou et al., 2008a ). Although granulite-facies rocks are present at the Jiaobei area (Jiangshan Group) in the Dabie-Sulu belt ( Zhou et al., 2008b,d ), their metamorphic ages of 1864–1803 Ma (zircon U–Pb age, Zhou et al., 2008d ) are quite different from the ∼500 Ma age of the Hutou Complex ( Wilde et al., in press; Zhou et al., in press ). Precambrian basement in the SCC is further characterized by the widespread occurrence of 820–740 Ma bimodal igneous rocks formed in rift settings ( Li et al., 2002, 2003; Zheng et al., 2003, 2005; Zhou et al., 2008c ). However, the ∼500 Ma granulite-facies metamorphism is lacking in both the NCC and SCC. Thus the Khanka massif is not related to either the NCC or SCC. One outstanding issue that needs to be considered is the mixed Permian faunas recorded from the Yanji area, which are characterized by an admixture of both paleo-tropical Cathaysian taxa and cool-temperate elements ( Shao and Tang, 1995; Shao et al., 1995; Ren et al., 1999 ). This was previously used to support the view that the Khanka massif has an affinity to the SCC ( Shao and Tang, 1995; Shao et al., 1995; Zhang, 1997, 2004; Zhang et al., 2006; Ren et al., 1999 ). However, Shi and Zhan (1996) , Shi (2006) and Li (2006) have pointed out that such mixed Permian faunas are not only found in the Yanji area, but also in other parts of the eastern CAOB. Specifically, since the Khanka massif is located in the easternmost part of the CAOB, it was at the boundary with the Paleo-Pacific Ocean during the Permian (Shi, 1996). Therefore, it seems quite likely that there could have been some transfer of material with paleo-tropical Cathaysian taxa across the Paleo-Pacific Ocean into the Paleo-Asian Ocean ( Shi, 2006 ). 6.5 Relations between the Khanka (-Jiamusi) block and Gondwana and/or the CAOB Another suggestion is that the Jiamusi/Khanka Massif was a crustal fragment derived from a peri-Gondwana position that drifted northward and collided with the CAOB collage in the Late Permian ( Wilde et al., 2003 ). Wilde et al. (1997, 2000, 2003) presented evidence of ∼500 Ma granulite-facies metamorphism in the Mashan Complex of the Jiamusi Massif and they related this to global late Pan-African events. Results of recent studies by Wilde et al. (in press) and Zhou et al. (in press) indicate that the Hutou Complex in the Khanka Massif also underwent a major ∼500 Ma granulite-facies metamorphic event (compare Fig. 9 a and b) . This not only supports the link between the Jiamusi and Khanka massifs but also the more widespread distribution of late Pan-African events in NE China. In terms of global setting, Wilde et al. (1999) undertook a review, based on Harley (1992) , of the nature and location of other late Pan-African granulite-facies terranes described in the literature and which showed broadly similar metamorphic P/T conditions; many also contained khondalitic metasediments. All such rocks were grouped closely together within a reconstructed Gondwana and were postulated to have once formed part of a late Pan-African orogenic belt that included parts of East Antarctica, Western Australia, Sri Lanka and India, and possibly also portions of Madagascar, Tanzania and Kenya. The Pan-African (or Brasiliano) orogeny is now considered to span almost the entire Neoproterozoic and most of the Cambrian from ca. 1000 to 500 Ma ( Rogers and Santosh, 2004 ). Late Pan-African events showing low-pressure granulite metamorphism at ∼500 Ma have also been recorded in the Delamerian Orogen in South Australia, with a southern extension into the Ross Orogen in Antarctica ( Foden et al., 1999 ). This close similarity with events in various parts of Australia, including the enigmatic high-grade events in the Halls Reward Belt in Queensland, led Wilde et al. (2003) to further develop the Gondwana connection and suggest that, along with the Tarim, North and South China blocks, the Jiamusi (and Khanka) Massif was located off Australia at ∼500 Ma. However, the lack of reliable paleomagnetic data for the Khanka and Jiamusi blocks means that it is currently impossible to test whether they were located in a similar peri-Gondwana position as the other Chinese blocks identified above ( Li et al., 1996 ). Another question is the route the Khanka/Jiamusi (Bureya) block took to reach its present position; did it simply drift northward, possibly from Gondwana, or did it somehow follow a separate path from the other blocks? Irrespective of its provenance, it now appears unlikely that the Jiamusi/Khanka/Bureya block represents a micro-continental block trapped between the North China and Siberia cratons during the main assembly of the CAOB. The identification of ∼180 Ma blueschist/greenschist metamorphism in the accretionary Heilongjiang Complex along the western margin of the Jiamusi Massif indicates accretion during the Jurassic ( Wu et al., 2007 ), much later than the other blocks. The location of the Jiamusi/Khanka (Bureya) block prior to this event is unknown. It may possibly have been located farther south along the Gondwana margin than the other Chinese blocks, thus taking longer to drift northward. Recent work on the Heilongjiang Complex has led to the possibility of a more local origin for the Khanka/Jiamusi (Bureya) blocks. Zhou et al. (2009) obtained magmatic zircons from blueschist-facies basaltic rocks and constrained the time of eruption as between 255 and 210 Ma. Based on their E-MORB to OIB signature and the fact they also contained inherited zircons with populations at 290–330 Ma, 420–530 Ma, 670–910 Ma and >1065 Ma, i.e. similar to zircon ages recorded from the Songliao Massif, Zhou et al. (2009) considered that the mafic rocks formed in a rift similar to the Red Sea that split the Jiamusi/Khanka/Bureya block from the Songliao Massif in the Permian. Subsequent closure of this ocean during the Late Triassic, with the development of extensive syn-collisional Triassic volcanic rocks ( Xu et al., 2009 ) and I-type granites in the Zhangguangcai-Lesser Xing’an ranges ( Wu et al., 2000, 2003, 2007 ) occurred later than ∼210 Ma. Thus collision and exhumation terminated in the Early Jurassic, based on the fact that peak metamorphism in the Heilongjiang HP belt occurred between 185 and 175 Ma ( Li et al., 1999; Wu et al., 2007 ). These latter events that resulted in closure of the ocean between the Songliao and Jiamusi/Khanka massifs were also postulated by Wu et al. (2007) , although they considered the Jiamusi/Khanka massif to be an exotic block of unknown affinity that collided with the CAOB in the Early Jurassic as a result of Pacific-Plate subduction. 6.6 Relations between the Khanka (-Jiamusi) block and the Siberia Craton It was earlier proposed ( Wilde et al., 1997 ) that the ∼500 Ma high-grade metamorphic event recognized in the Mashan Complex of the Jiamusi Massif was the result of collision between Siberia and the North China Craton. However, Siberia is generally considered to have been located much farther to the east in the initial Rodinia configuration ( Dalziel, 1991 ). It was more closely associated with Laurentia and, by ∼540 Ma, it is considered to have lain much further to the southeast; direct collision with Siberia therefore appears unlikely. However, late Pan-African metamorphism has now been reported along the southern margin of the Siberia Craton in several recent studies ( Salnikova et al., 1998, 2001; Donskaya et al., 2000; Khain et al., 2003; Gladkochub et al., 2008 ). This early Paleozoic collision is named the Sayang-Baikal Orogen ( Salnikova et al., 1998; Donskaya et al., 2000; Gladkochub et al., 2008 ) and includes the Olkhon, Kitoykin, Derba and Slyudyanka massifs (OKDS massifs) ( Fig. 1 ), developed by accretion of arcs and massifs relatively early during the evolution of the CAOB. This event was responsible for substantial continental growth in southern Siberia ( Salnikova et al., 1998; Gladkochub et al., 2008 ). Salnikova et al. (1998) obtained single-grain zircon 206 Pb/ 238 U ages of 488–478 Ma for TTG rocks interpreted to have been emplaced during granulite-facies metamorphism of the Slyudyanka Complex in the southwestern Baikal region at the southern margin of the Siberia Craton ( Fig. 1 ). Peak metamorphism was determined at 478 ± 2 Ma, whereas a post-metamorphic pyroxene-bearing quartz syenite, with a zircon multi-grain 206 Pb/ 238 U age of 471 ± 2 Ma, indicated the metamorphic episode was short-lived, lasting for only ∼20 Ma. The ages reported by Salnikova et al. (1998) are only slightly younger than the results presented here from the Khanka Massif and it is possible that they may be related to the same overall event. Indeed, there are marked similarities when the data for the Sayang-Baikal belt are plotted against the Jiamusi and Khanka data ( Fig. 9 c), which includes 498 ± 5 Ma ages from the Derba Massif ( Nozhkin et al., 2004 ) and 473 ± 3 Ma data from the Kioykin Massif ( Donskaya et al., 2000 ). More recently, Gladkochub et al. (2008) have reported metamorphic zircons from two mafic granulites of the Olkhon massif ( Fig. 1 ), which recorded 238 U/ 206 Pb ages of 507 ± 8 to 498 ± 7 Ma, with the main metamorphic event occurring at ∼500 Ma. Ages from 535 to 2750 Ma from detrital zircon cores in early Paleozoic metasediments of the Olhkon massif were also obtained by Gladkochub et al. (2008) , showing a marked similarity to results reported from the Jiamusi and Khanka massifs in NE China. In summary, based on the fact that the zircon populations of the Hutou Complex in the Khanka Massif are similar to those of the Mashan Complex in the Jiamusi Massif and also to zircon populations in the CAOB and the Sayang-Baikal belt along the southern margin of the Siberia Craton ( Figs. 8 and 9 ), we postulate that the most likely source area of the Khanka and Jiamusi massifs is a ‘local’ one within the CAOB. This is further supported by recent studies on the Heilongjiang Complex discussed above ( Zhou et al., 2009 ), where evidence from the blueschist-facies rocks favours rifting and closure of an ocean between the Songliao and Jiamusi/Khanka (Bureya) blocks. However, the emerging data from the southern margin of the Siberia Craton leads to another possibility: that the combined Jiamusi/Khanka/Bureya block was derived from Siberia and that it rifted away from here and then later collided with the Songliao Massif. This possible scenario is examined further in the following section. 6.7 Possible location of the Khanka Massif from the Neoproterozoic to the Mesozoic Based on the tectonic model of Gladkochub et al. (2008) , we propose a tentative model whereby the Jiamusi/Khanka/Bureya block originated as part of the late Pan-African (∼500 Ma) orogenic belt developed around the southern margin of the Siberia craton. The precursor of the OKDS massifs have a Siberia Craton (Aldan Province) affinity ( Fig. 10 ; Salnikova et al., 1998; Gladkochub et al., 2008 ), and may have been detached from here in the latest Mesoproterozoic ( Khain et al., 2002; Kuzmichev et al., 2005; Gladkochub et al., 2008 ), These detached fragments, possibly including the Jiamusi/Khanka/Bureya massifs, would have drifted to the south and west (in present coordinates) across the Paleo-Asian Ocean to the southern part of the Siberia Craton (Angara–Anabar Province), where Neoproterozoic magmatism was the result of subduction ( Fig. 10 a; Khain et al., 2002; Kuzmichev et al., 2005; Gladkochub et al., 2008 ). During oblique collision with the SE part of the Angara–Anabar Province at about 500 Ma, the rocks were deformed and metamorphosed to granulite facies. The collision was accompanied by emplacement of mantle-derived magmas with ages at 475–450 Ma ( Gladkochub et al., 2008 ), which caused melting of the thickened lithosphere and intrusion of syn-tectonic granitoids, However, at sometime between 450 and 180 Ma, the combined Jiamusi/Khanka/Bureya block rifted away and moved farther south into what is now NE China/Far East Russia. Based on data from the blueschist-facies metabasalts of the Heilongjiang Complex ( Zhou et al., 2009 ), it is likely that this rifting took place around 260 Ma in the Late Permian, The final docking with the Songliao Massif occurred between 210 and 180 Ma, based on mica cooling ages recorded from the Heilongjiang Complex ( Wu et al., 2007 ). The westward drift of the Jiamusi/Khanka/Bureya block was the result of the onset of Pacific-plate subduction ( Wu et al., 2007 ), that has been constrained to the latest Triassic-Early Jurassic ( Zhou et al., 2009 ). The scenario presented above is somewhat speculative at this time and requires additional data to either substantiate or refute the model. If correct, then the implication would be that other massifs/blocks in the eastern CAOB (e.g., Songliao, Great Hinggan, Ergun massifs in NE China; see Fig. 1 ), may have had a similar history, since they appear to record the same Pan-African events with two peaks in zircon ages at ∼500 and 600–900 Ma (e.g., Wilde et al., 1997, 2003; Jahn et al., 2000; Jahn, 2004; Kelty et al., 2008; Xiao et al., 2009; Qian et al., 2009; Zhou et al., 2009 ). So, whilst this is admittedly a speculative model until there are new paleomagnetic and geochronological data to provide further constraints, the scenario is not inconsistent with the available evidence. 7 Conclusions (1) The Khanka Massif straddles the boundary between NE China and Far East Russia and, in China, consists of the Hulin and Hutou complexes and the Mt. Huoshi sequence. The Hulin Complex is composed of sedimentary rocks that were metamorphosed to greenschist facies and are now represented by quartz schist, muscovite-quartz schist and two-mica schist. The Hutou Complex is a khondalitic sequence with intrusive granitoids, remarkably similar to the Mashan Complex in the Jiamusi Massif. The amphibolite-facies metasedimentary sequences at Mt. Huoshi were originally grouped with the Hutou Complex ( HBGMR, 1993 ), but our results suggest they may either form part of the Hulin Complex, or else be a separate unrelated unit. (2) Samples of two-mica schist and quartz schist (samples HL-011 and 04H-138, respectively) were selected for SHRIMP detrital zircon U–Pb dating from the Hulin Complex. Sample HL-011 yields a unimodal population of grains with a weighted mean 238 U/ 206 Pb age of 257 ± 3 Ma, suggesting derivation from nearby Permian granitoids (see Wilde et al., 1997, in press ). Sample 04H-138 contains five age populations: at 233–270 Ma, 350–400 Ma, 473–570 Ma, 720–760 Ma and 930–970 Ma, with peaks at 252, 350, 520, 758 and 970 Ma. The youngest detrital zircon has a 206 Pb/ 238 U age of 233 ± 9 Ma, limiting deposition to after ∼230 Ma. (3) SHRIMP detrital zircon U–Pb dating was undertaken on two samples of fine-grained gneiss (samples 06H-15 and 06H-16) from the Mt. Huoshi sequence. In sample 06H-15, the grains fall into four age populations, with groupings at 260–340 Ma, 470–560 Ma, 710–940 Ma and a single grain at 2090 Ma. Besides establishing the presence of a range of Paleoproterozoic to Neoproterozoic rocks in the source region, the youngest zircon with an age of 259 ± 8 Ma limits the maximum possible age of deposition, similar to that of sample HL-011 from the Hulin Complex. The data from sample 06H-16 do not constrain the time of deposition. (4) When the detrital zircon results from this study are taken together with the ∼500 Ma age previously obtained for the granulite-facies metamorphic event in the Hutou Complex of the Khanka Massif ( Wilde et al., in press; Zhou et al., in press ), it suggests that this complex is equivalent to the Mashan Complex in the Jiamusi Massif and that the Khanka and Jiamusi massifs form part of a contiguous continental block, along with the Bureya Massif in Russia. (5) The evolution of the combined Khanka/Jiamusi/Bureya block is difficult to constrain on the basis of presently available data. The detrital zircon data establish that it was not part of either the North China Craton or the South China Craton, and available paleomagnetic data preclude a collision with the Siberia Craton to explain the ∼500 Ma high-grade metamorphic event. We propose that it might have initially formed part of the Aldan Province of the Siberia Craton that broke away and collided with the southern margin of the Angara-Anabar Province of Siberia, forming part of the Sayang-Baikal orogen, before moving southward between the 450 and 210 Ma. What is evident is that the Khanka Massif, together with the Jiamusi-Bureya massifs, collided with the Songliao Massif in the latest Triassic to early Jurassic as a result of Pacific-plate subduction. They thus form part of the accretion of continental blocks to Eurasia that will, in time, form the next supercontinent. Acknowledgements This study was funded by grants from the National Natural Science Foundation of China ( 40872121 , 40672148 and 40739905 ), the Ministry of Science and Technology of the P.R. China ( 2009CB825008 ), and the Ph.D. Programs Foundation of the Ministry of Education of China ( 20090061110048 ). We thank Professor Santosh (Guest Editor) and the Reviewers (Alfred Kroner and ‘anonymous’) for their comments that helped to improve the manuscript. This is The Institute for Geoscience Research (TIGeR) publication No. 216. References Black and Jagodzinski, 2003 L.P. Black E.A. Jagodzinski Importance of establishing sources of uncertainty for the derivation of reliable SHRIMP ages Australian Journal of Earth Sciences 50 2003 503 512 Black et al., 2003 L.P. Black S.L. Kamo I.S. Williams R. Mundil D.W. Davis R.J. Korsch C. Foudoulis The application of SHRIMP to Phanerozoic geochronology: a critical appraisal of four zircon standards Chemical Geology 200 2003 171 188 Cao et al., 1992 X. Cao Z.X. Dang X.Z. Zhang J.S. Jiang H.D. Wang Jiamusi Composite Terranes 1992 Jilin Publishing House of Science and Technology Changchun 224 pp (in Chinese, with English and Russian abstracts) Cheng et al., 2006 R.Y. Cheng F.Y. Wu W.C. Ge D.Y. Sun X.M. Liu J.H. Yang Emplacement age of the Raohe Complex in eastern Heilongjiang Province and the tectonic evolution of the eastern part of Northeastern China Acta Petrologica Sinica 22 2006 353 376 Compston et al., 1992 W. Compston I.S. Williams J.L. Kirschvink Z. Zhang G. Ma Zircon U–Pb ages for the early Cambrian time-scale Journal of the Geological Society of London 149 1992 171 184 Dalziel, 1991 I.W.D. Dalziel Pacific margins of Laurentia and East Antarctica–Australia as a conjugate rift pair: evidence and implications for an Eocambrian supercontinent Geology 19 1991 598 601 Dang and Li, 1993 Y. Dang D. Li Discussion on isotope geochronology of Precambrian Jiamusi Block Journal of Changchun University of Earth Sciences 23 1993 318 332 (in Chinese with English abstract) Donskaya et al., 2000 T.V. Donskaya E.V. Sklyarov D.P. Gladkochub The Baikal collisional metamorphic belt Doklady Earth Sciences 374 2000 1075 1079 Foden et al., 1999 J. Foden M. Sandiford P.J. Dougherty I. Willams Geochemistry and geochronology of the Rathjen Gneiss: Implications for the early tectonic evolution of the Delamerian Orogen Australian Journal of Earth Sciences 46 1999 377 389 Gladkochub et al., 2008 D.P. Gladkochub T.V. Donskaya M.T.D. Wingate U. Poller A. Kröner V.S. Fedorovsky A.M. Mazukabov W. Todt S.A. Pisarevsky Petrology, geochronology, and tectonic implications of c. 500 Ma metamorphic and igneous rocks along the northern margin of the Central Asian Orogen (Olkhon terrane, Lake Baikal, Siberia) Journal of the Geological Society of London 165 2008 235 246 Grimmer et al., 2003 J.C. Grimmer L. Ratschbacher M. McWilliams L. Franz I. Gaitzsch M. Tichomirowa B.R. Hacker Y. Zhang When did the ultrahigh pressure rocks reach the surface? A 207Pb/206Pb zircon, 40Ar/39Ar white mica, Si-in-white mica, single-grain provenance study of Dabie Shan synorogenic foreland sediments Chemical Geology 197 2003 87 110 Hacker et al., 2006 B.R. Hacker W.C. McClelland J.G. Liou Ultrahigh-pressure metamorphism: deep continental subduction The Geological Society of America, Special Paper 403 2006 1 206 Harley, 1992 S.L. Harley Proterozoic granulite terranes K.C. Condie Developments in Precambrian Geology, Proterozoic crustal evolution vol. 10 1992 Elsevier Amsterdam, The Netherlands 301 359 HBGMR, 1993 HBGMR (Heilongjiang Bureau of Geology and Mineral Resources) Regional Geology of Heilongjiang Province 1993 Geological Publishing House pp. 1–734 (in Chinese with English abstract) Ishiwatari and Tsujimori, 2001 A. Ishiwatari T. Tsujimori Late Paleozoic High-pressure Metamorphic Belts in Japan and Sikhote-Alin: Possible Oceanic Extension of the Chinese Dabie-Su-Lu Suture Detouring Korea Gondwana Research 4 2001 636 638 Ishiwatari and Tsujimori, 2003 A. Ishiwatari T. Tsujimori Paleozoic ophiolites and blueschists in Japan and Russian Primorye in the tectonic framework of East Asia: a synthesis The Island Arc 12 2003 190 206 Jahn, 2004 B.M. Jahn The Central Asian Orogenic Belt and growth of the continental crust in the Phanerozoic J. Malpas C.J.N. Fletcher J.R. Ali J.C. Aitchison Aspects of the Tectonic Evolution of China vol. 226 2004 Geological Society of London, Special Publication 73 100 Jahn et al., 2000 B.M. Jahn F.Y. Wu B. Chen Massive granitoid generation in central Asia: Nd isotopic evidence and implication for continental growth in the Phanerozoic Episodes 23 2000 82 92 Jia et al., 2004 D.C. Jia R.Z. Hu Y. Lu X.L. Qiu Collision belt between the Khanka block and the North China block in the Yanbian region, northeast China Journal of Asian Earth Sciences 23 2004 211 219 Jian et al., 2008 P. Jian D.Y. Liu X.M. Sun SHRIMP dating of the Permo-Carboniferous Jinshajiang ophiolite, southwestern China: Geochronological constraints for the evolution of Paleo-Tethys Journal of Asian Earth Sciences 32 2008 371 384 Jiang, 1992 J. Jiang Peak regional metamorphism of the khondalite series of Mashan Group and its evolution Acta Petrologica et Mineralogica 11 1992 97 108 (in Chinese with English abstract) Khain et al., 2003 E.V. Khain E.V. Bibikova E.B. Salnikova A. Kröner A.S. Gibsher A.N. Didenko K.E. Degtyarev A.A. Fedotova The Palaeo-Asian Ocean in the Neoproterozoic and early Palaeozoic: new geochronologic data and palaeotectonic reconstructions Precambrian Research 122 2003 329 358 Khain et al., 2002 E.V. Khain E.V. Bibikova A. Kröner D.Z. Zhuravlev E.V. Sklyarov A.A. Fedotova I.R. Kravchenko-Berezhnoy The most ancient ophiolite of the central Asian fold belt: U–Pb and Pb–Pb zircon ages for the Dunzhugur Complex, Eastern Sayan, Siberia, and geodynamic implications Earth and Planetary Science Letters 202 2002 1 16 Kuzmichev et al., 2005 A. Kuzmichev A. Kröner E. Hegner D.Y. Liu Y.S. Wan The Shishkhid ophiolite, northern Mongolia: a key to the reconstruction of a Neoproterozoic island-arc system in central Asia Precambrian Research 138 2005 125 150 Kojima, 1989 S. Kojima Mesozoic terrane accretion in northeast China, Sikhote-Alin and Japan regions Palaeogeography, Palaeoclimatology, Palaeoecology 69 1989 213 232 Kelty et al., 2008 T.K. Kelty A. Yin B. Dash G.E. Gehrels A.E. Ribeiro Detrital-zircon geochronology of Paleozoic sedimentary rocks in the Hangay—Hentey basin, north-central Mongolia: Implications for the tectonic evolution of the Mongol—Okhotsk Ocean in central Asia Tectonophysics 451 2008 290 311 Lennon et al., 1997 Lennon, R.G., Wilde S.A., Yang T., 1997. The Mashan Group: a 500 Ma granulite facies terrain within the Jiamusi massif, Heilongjiang Province, North-eastern China. In: Cox, R., Ashwal, L.D., (Eds.), Proterozoic Geology of Madagascar, Proceedings of UNESCO-IUGS-IGCP 348/368 International Field Workshop, Antananarivo, Madagascar, 1997. Gondwana Research Group, Publication 5, pp. 45–46. Li, 2006 J.Y. Li Permian geodynamic setting of Northeast China and adjacent regions: Closure of the Paleo-Asian Ocean and subduction of the Paleo-Pacific Plate Journal of Asian Earth Sciences 26 2006 207 224 Li et al., 1999 J.Y. Li B.G. Niu B. Song W.X. Xu Y.H. Zhang Z.R. Zhao Crustal Formation and Evolution of Northern Changbai Mountains, Northeast China 1999 Geological Publishing House Beijing pp. 32–50 (in Chinese with English abstract) Li et al., 2003 Z.X. Li X.H. Li P.D. Kinny J. Wang S. Zhang H. Zhou Geochronology of Neoproterozoic syn-rift magmatism in the Yangtze Craton, South China and correlations with other continents: evidence for a mantle superplume that broke up Rodinia Precambrian Research 122 2003 85 109 Li et al., 2002 Z.X. Li X.H. Li H. Zhou P.D. Kinny Grenvillian continental collision in south China: new SHRIMP U–Pb zircon results and implications for the configuration of Rodinia Geology 30 2002 163 166 Li et al., 1996 X.Z. Li L. Zhang C.Mc. Powell Positions of the East Asian cratons in the Neoproterozoic supercontinent Rodinia Australian Journal of Earth Sciences 43 1996 593 604 Liou et al., 1989a J.G. Liou S.A. Graham S. Maruyama X. Wang X. Xiao A.R. Carroll J. Chu Y. Feng M.S. Hendrix Y.H. Liang C.L. McKnight Y. Tang Z.X. Wang M. Zhao B. Zhu Proterozoic blueschist belt in western China: best documented Precambrian blueschists in the world Geology 17 1989 1127 1131 Liou et al., 1989b J.G. Liou X. Wang R.G. Coleman Z.M. Zhang S. Maruyama Blueschists in major suture zones of China Tectonics 8 1989 609 619 Liu, 1988 J. Liu Precambrian Geology of the Jiamusi Massif Heilongjiang Geology 1 1988 148 156 (in Chinese with English abstract) Ludwig, 2001a Ludwig, K.R., 2001a. Squid 1.02: a user's manual. Berkeley Geochronology Center, Special Publication No 2, Berkeley, CA, 19 pp. Ludwig, 2001b Ludwig, K.R., 2001b. Users Manual for Isoplot/Ex (ver. 2.49): A Geochronological Toolkit for Microsoft Excel. Berkeley Geochronology Center, Special Publication No. 1a, Berkeley, CA, 55 pp. Miao et al., 2008 L.C. Miao W.M. Fan D.Y. Liu F.Q. Zhang Y.R. Shi F. Guo Geochronology and geochemistry of the Hegenshan ophiolitic complex: Implications for late-stage tectonic evolution of the Inner Mongolia-Daxinganling Orogenic Belt, China Journal of Asian Earth Sciences 32 2008 348 370 Natal’in, 1991 B.A. Natal’in Mesozoic accretion and collision tectonics of southern USSR Far East Pacific Geology 10 1991 3 23 Natal’in, 1993 B.A. Natal’in History and modes of Mesozoic accretion in southeastern Russia The Island Arc 2 1993 15 34 Natal’in and Borukayev, 1991 B.A. Natal’in C.B. Borukayev Mesozoic sutures in the southern Far East of USSR Geotectonics 25 1991 64 74 Nelson, 1997 Nelson, D.R., 1997. Compilation of SHRIMP U–Pb zircon geochronology data, 1996, Geological Survey of Western Australia, Record 1997/2, 189 pp. Nozhkin et al., 2004 A.D. Nozhkin O.M. Turkina T.B. Bayanova A.V. Travin The granitoids of the south-western frame of the Siberian craton as indicators of the Riphean juvenile crust forming and following accretion–collisional processes E.V. Sklyarov Geodynamic Evolution of the Lithosphere of the Central Asian Mobile Belt 2004 IGSBRAS Irkutsk 49 52 Oh, 2006 C.W. Oh A new concept on tectonic correlation between Korea, China and Japan: Histories from the late Proterozoic to Cretaceous Gondwana Research 9 2006 47 61 Oh et al., 2006 C.W. Oh S.W. Kim I.S. Williams Spinel granulite in Odesan area, South Korea: Tectonic implications for the collision between the North and South China blocks Lithos 92 2006 557 575 Oh et al., 2009 C.W. Oh S.-G. Choi J. Seo V.J. Rajesh J.H. Lee M. Zhai P. Peng Neoproterozoic tectonic evolution of the Hongseong area, southwestern Gyeonggi Massif, South Korea: implications for the tectonic evolution of Northeast Asia Gondwana Research 16 2009 272 284 Qian et al., 2009 Q. Qian J. Gao R. Klemd G.Q. He B. Song D.Y. Liu R.-H. Xu Early Paleozoic tectonic evolution of the Chinese South Tianshan Orogen: constraints from SHRIMP zircon U–Pb geochronology and geochemistry of basaltic and dioritic rocks from Xiate, NW China International Journal of Earth Sciences 98 2009 551 569 Ratschbacher et al., 2006 L. Ratschbacher L. Franz E. Enkelmann R. Jonckheere A. Porschke B.R. Hacker S. Dong Y. Zhang The Sino-Korean-Yangtze suture, the Huwan detachment, and the Paleozoic-Tertiary exhumation of (ultra) high-pressure rocks along the Tongbai-Xinxian-Dabie Mountains B.R. Hacker W.C. McClelland J.G. Liou Ultrahigh-pressure Metamorphism: Deep Continental Subduction vol. 403 2006 The Geological Society of America, Special Paper 45 77 Ren et al., 1999 J.S. Ren Z.X. Wang B.W. Chen C.F. Jiang B.G. Niu J.Y. Li G.L. Xie Z.G. He Z.G. Liu The tectonics of China from a global view. A guide to the tectonic map of China and adjacent regions 1999 Geological Publishing House Beijing pp. 4–32 Rogers and Santosh, 2004 J.J.W. Rogers M. Santosh Continents and Supercontinents 2004 Oxford University Press USA 289 pp Sajeev et al., 2010 K. Sajeev J. Joeng S. Kwon W.-S. Kee S.W. Kim T. Komiya T. Itaya H.-S. Jung Y. Park High P-T granulite relicts from the Imjingang belt, South Korea: Tectonic significance Gondwana Research 17 2010 75 86 Salnikova et al., 2001 E.B. Salnikova I.K. Kozakov A.B. Kotov A. Kröner W. Todt E.V. Bibikova A. Nutman S.Z. Yakovleva V.P. Kovach Age of Palaeozoic granites and metamorphism in the Tuvino-Mongolian massif of the Central Asian Mobile Belt: loss of a Precambrian microcontinent Precambrian Research 110 2001 143 164 Salnikova et al., 1998 E.B. Salnikova S.A. Sergeev A.B. Kotov S.Z. Yakovleva L.Z. Reznitskii E.P. Vasil’ev U–Pb zircon dating of granulite metamorphism in the Slyudyanskiy complex, Eastern Siberia Gondwana Research 1 1998 195 205 Santosh et al., 2006 M. Santosh K. Sajeev J.H. Li Extreme crustal metamorphism during Columbia supercontinent assembly: evidence from North China craton Gondwana Research 10 2006 256 266 Santosh et al., 2007a M. Santosh T. Tsunogae J.H. Li S.J. Liu Discovery of sapphirine-bearing Mg-Al granulites in the North China Craton: implications for Paleoproterozoic ultrahigh temperature metamorphism Gondwana Research 11 2007 263 285 Santosh et al., 2007b M. Santosh S.A. Wilde J.H. Li Timing of Paleoproterozoic ultrahigh-temperature metamorphism in the North China Craton: evidence from SHRIMP U–Pb zircon geochronology Precambrian Research 159 2007 178 196 Sengör and Natal’in, 1996 A.M.C. Sengör B.A. Natal’in Paleotectonics of Asia: fragments of a synthesis A. Yin T.M. Harrison The Tectonic Evolution of Asia 1996 Cambridge University Press 486 640 Sengör et al., 1993 A.M.C. Sengör B.A. Natal’in V.S. Burtman Evolution of the Altaid tectonic collage and Palaeozoic crustal growth in Eurasia Nature 364 1993 299 307 Shao and Tang, 1995 J.A. Shao K.D. Tang Terranes in Northeast China and Evolution of Northeast Asia Continental Margin 1995 Seismology Publishing House Beijing 185 pp (in Chinese) Shao et al., 1995 J.A. Shao K.D. Tang L.P. Zhan Reconstruction of an ancient continental margin and its implication: new progress on the study of geology of Yanbian region, northeast China Science in China (Series D) 25 1995 548 555 Shi, 2006 G.R. Shi Marine Permian in East and NE Asia: an overview of biostratigraphy, palaeobiogeography and palaeogeographical implications Journal of Asian Earth Sciences 26 2006 175 206 Shi and Zhan, 1996 G.R. Shi L.P. Zhan A mixed mid-Permian marine fauna from the Yanji area, northeastern China: a paleobiogeographical reinterpretation The Island Arc 5 1996 385 395 Song et al., 1994 B. Song B.G. Niu J.Y. Li W.X. Xu Isotope geochronology of granitoids in Mudanjiang-Jixi area Acta Petrologica et Mineralogica 13 1994 204 213 (in Chinese with English abstract) Steiger and Jäger, 1977 R.H. Steiger E. Jäger Subcommission on geochronology: convention on the use of decay constants in geo- and cosmochronology Earth and Planetary Science Letters 36 1977 359 362 Stern, 2008 J.R. Stern Neoproterozoic crustal growth: The solid Earth system during a critical episode of Earth history Gondwana Research 14 2008 33 50 Tang, 1990 K.D. Tang Tectonic development of Paleozoic foldbelts at the northern margin of the Sino–Korean craton Tectonics 9 1990 249 260 Tang et al., 1995 K.D. Tang Y. Wang G.Q. He J.A. Shao Continental-margin structure of Northeast China and its adjacent areas Acta Geologica Sinica 69 1995 16 30 Vernikovsky et al., 2003 V.A. Vernikovsky A.E. Vernikovskaya A.B. Kotov E.B. Salnikova V.P. Kovach Neoproterozoic accretionary and collisional events on the western margin of the Siberian craton: new geological and geochronological evidence from the Yenisey Ridge Tectonophysics 375 2003 147 168 Wan et al., 2006 Y.S. Wan B. Song D.Y. Liu S.A. Wilde J.S. Wu Y.R. Shi X.Y. Yin H.Y. Zhou SHRIMP U–Pb zircon geochronology of Palaeoproterozoic metasedimentary rocks in the North China Craton: Evidence for a major Late Palaeoproterozoic tectonothermal event Precambrian Research 149 2006 249 271 Wang et al., 2007 L.W. Wang Y. Wang J. Yang G.Q. Wu G.Y. Li L. Sheng Pre-Mesozoic basement provenance tracing of the Songliao Basin by means of detrital zircon SHRIMP chronology Earth Science Frontiers 14 2007 151 158 Wilde et al., 1999 S.A. Wilde H.L. Dorsett-Bain R.G. Lennon Geological setting and controls on the development of graphite, sillimanite and phosphate mineralisation within the Jiamusi massif: An exotic fragment of Gondwanaland located in North-Eastern China? Gondwana Research 2 1999 21 46 Wilde et al., 1997 S.A. Wilde H.L. Dorsett-Bain J.L. Liu The identification of a Late Pan-African granulite facies event in Northeastern China: SHRIMP U–Pb zircon dating of the Mashan Group at Liu Mao, Heilongjiang Province, China Proceedings of the 30th IGC: Precambrian Geology and Metamorphic Petrology 17, VSP International 1997 Science Publishers Amsterdam 59 74 Wilde et al., 2003 S.A. Wilde F.Y. Wu X.Z. Zhang Late Pan-African magmatism in Northeastern China: SHRIMP U–Pb zircon evidence for igneous ages from the Mashan Complex Precambrian Research 122 2003 311 327 Wilde et al., 2000 S.A. Wilde X.Z. Zhang F.Y. Wu Extension of a newly-identified 500 Ma metamorphic terrain in Northeast China: further U–Pb SHRIMP dating of the Mashan Complex, Heilongjiang Province, China Tectonophysics 328 2000 115 130 Wilde et al., in press Wilde, S.A., Wu, F.Y., Zhao, G.C., in press. The Khanka Block, NE China, and its significance to the evolution of the Central Asian Orogenic Belt and continental accretion. In: Kusky, T.M., Zhai, M.G., Xiao, W.J. (Eds.). The Evolved Continents: Understanding the Processes of Continental Growth, Geological Society of London, Special Publication. Wilde et al., 2008 S.A. Wilde F.Y. Wu G.C. Zhao E. Sklyarov The Jiamusi, Khanka and Bureya blocks; a contiguous crustal entity accreted to the Central Asian Orogenic Belt in the Early Jurassic? 33rd International Geological Congress Oslo 2008 (Abstract CD-ROM) Wilde and Zhao, 2005 S.A. Wilde G.C. Zhao Archean to Paleoproterozoic evolution of the North China Craton Journal of Asian Earth Sciences 24 2005 519 522 Williams, 1998 I.S. Williams U-Th-Pb geochronology by ion microprobe M.A. McKibben W.C. Shanks III W.I. Ridley Applications of Microanalytical Techniques to Understanding Mineralizing Processes vol. 7 1998 Reviews in Economic Geology 1 35 Wu et al., 2003 F.Y. Wu B.M. Jahn S.A. Wilde C.H. Lo T.F. Yui Q. Lin W.C. Ge S.Y. Sun Highly fractionated I-type granites in NE China (I): Geochronology and petrogenesis Lithos 66 2003 241 273 Wu et al., 2000 F.Y. Wu B.M. Jahn S.A. Wilde D.Y. Sun Phanerozoic continental crustal growth: Sr-Nd isotopic evidence from the granites in northeastern China Tectonophysics 328 2000 87 113 Wu et al., 2001 F.Y. Wu D.Y. Sun H.M. Li X.L. Wang The nature of basement beneath the Songliao Basin in NE China: Geochemical and isotopic constraints Physics and Chemistry of the Earth (Part A) 26 2001 793 803 Wu et al., 2007 F.Y. Wu J.H. Yang C.H. Lo S.A. Wilde D.Y. Sun B.M. Jahn Jiamusi massif in China: a Jurassic accrationary terrane in the western Pacific The Island Arc 16 2007 156 172 Xiao et al., 2004b W.J. Xiao L.C. Zhang K.Z. Qin S. Sun J. Li Paleozoic accretionary and collisional tectonics of the eastern Tienshan (China): implications for the continental growth of Central Asia American Journal of Science 304 2004 370 395 Xiao et al., 2004a W.J. Xiao B.F. Windley G. Badarch S. Sun J. Li K. Qin Z. Wang Palaeozoic accretionary and convergent tectonics of the southern Altaids: implications for the growth of Central Asia Journal of the Geological Society of London 161 2004 339 342 Xiao et al., 2003 W.J. Xiao B. Windley J. Hao M.G. Zhai Accretion leading to collision and the Permian Solonker suture, Inner Mongolia, China: Termination of the Central Asian Orogenic Belt Tectonics 22 2003 2002TC001484 Xiao et al., 2009 W.J. Xiao B.F. Windley B.C. Huang C.M. Han C. Yuan H.L. Chen M. Sun S. Sun J.L. Li End-Permian to mid-Triassic termination of the accretionary processes of the southern Altaids: implications fro the geodynamic evolution, Phanerozoic continental growth and metallogeny of Central Asia International Journal of the Earth Sciences 2009 10.1007/s00531-008-0407-z Xu et al., 2009 W.L. Xu W.Q. Ji F.P. Pei E. Meng Y. Yu D.B. Yang X.Z. Zhang Triassic volcanism in eastern Heilongjiang and Jilin provinces, NE China: chronology, geochemistry, and tectonic implications Journal of Asian Earth Sciences 34 2009 392 402 Zhai et al., 2005 M.G. Zhai J.H. Guo W.J. Liu Neoarchean to Paleoproterozoic continental evolution and tectonic history of the North China Craton: a review Journal of Asian Earth Sciences 24 2005 547 561 Zhang, 1997 K.J. Zhang North and South China collision along the eastern and southern North China margins Tectonophysics 270 1997 145 156 Zhang, 2004 K.J. Zhang Granulite xenoliths from Cenozoic basalts in SE China provide geochemical fingerprints to distinguish lower crust terranes from the North and South China tectonic blocks: comment Lithos 73 2004 127 134 Zhang et al., 2006 K.J. Zhang J.X. Cai J.X. Zhu North China and South China collision: insights from analogue modeling Journal of Geodynamics 42 2006 38 51 Zhang, 1991 X.Z. Zhang Coexistence of blueschists and greenschists: a new evidence for the tectonic evolution of the Heilongjiang rock series Journal Changchun University (Earth Science) 21 1991 277 282 (in Chinese with English abstract) Zhao et al., 2000 G.C. Zhao P.A. Cawood S.A. Wilde M. Sun L.Z. Lu Metamorphism of basement rocks in the Central Zone of the North China Craton: implications for Paleoproterozoic tectonic evolution Precambrian Research 103 2000 55 88 Zhao et al., 2002 G.C. Zhao P.A. Cawood S.A. Wilde M. Sun Review of global 2.1-1.8 Ga orogens: implications for a pre-Rodinia supercontinent Earth Science Reviews 59 2002 125 162 Zhao et al., 2005 G.C. Zhao M. Sun S.M. Wilde S.Z. Li Late Archean to Paleoproterozoic evolution of the North China Craton: key issues revisited Precambrian Research 136 2005 177 202 Zhao et al., 1995 C.J. Zhao Y.J. Peng Z.X. Dang The Formation and Evolution of Crust in Eastern Jilin and Heilongjiang Provinces 1995 Liaoning University Press Shenyang pp. 1–226 (in Chinese with English abstract) Zheng et al., 2003 Y.F. Zheng B. Fu B. Gong L. Li Stable isotope geochemistry of ultrahigh pressure metamorphic rocks from the Dabie-Sulu orogen in China: Implications for geodynamics and fluid regime Earth Science Reviews 62 2003 105 161 Zheng et al., 2005 Y.F. Zheng J.B. Zhou Y.B. Wu Z. Xie Low-grade metamorphic rocks in the Dabie-Sulu orogenic belt: a passive-margin accretionary wedge deformed during continent subduction International Geology Review 47 2005 851 871 Zhou et al., 2008a J.B. Zhou S.A. Wilde G.C. Zhao C.Q. Zheng W. Jin X.Z. Zhang H. Cheng Detrital zircon U–Pb dating of low-grade metamorphic rocks in the Sulu UHP belt: evidence for overthrusting of the North China Craton onto the South China Craton during continental subduction Journal of the Geological Society of London 165 2008 423 433 Zhou et al., 2008c J.B. Zhou S.A. Wilde G.C. Zhao C.Q. Zheng W. Jin X.Z. Zhang H. Cheng SHRIMP U–Pb zircon dating of the Wulian Complex: defining the boundary between the North and South China Cratons in the Sulu Orogenic Belt, China Precambrian Research 162 2008 559 576 Zhou et al., 2008b J.B. Zhou S.A. Wilde G.C. Zhao C.Q. Zheng W. Jin X.Z. Zhang H. Cheng SHRIMP U–Pb zircon dating of the Neoproterozoic Penglai Group and Archean gneisses from the JiaobeiTerrane, North China Craton, and their tectonic implications Precambrian Research 160 2008 323 340 Zhou et al., 2008d X.W. Zhou G.C. Zhao C.J. Wei Y.S. Geng M. Sun EPMA, U-Th-Pb monazite and SHRIMP U–Pb zircon geochronology of high-pressure pelitic granulites in the Jiaobei massif of the North China Craton American Journal of Science 308 2008 328 350 Zhou et al., 2009 J.B. Zhou S.A. Wilde X.Z. Zhang G.C. Zhao C.Q. Zheng Y.J. Wang X.H. Zhang The onset of Pacific margin accretion in NE China: evidence from the Heilongjiang high-pressure metamorphic belt Tectonophysics 478 2009 230 246 Zhou et al., in press Zhou, J.B., Wilde, S.A., Zhao, G.C., Zhang, X.Z., Zheng, C.Q., Wang H. Pan-African metamorphic and magmatic rocks of the Khanka Massif, NE China: further evidence regarding their affinity. Geological Magazine. doi:10.1017/S0016756810000063 , in press. Zyabrev and Matsuoka, 1999 S. Zyabrev A. Matsuoka Late Jurassic (Tithonian) radiolarians from a clastic unit of the Khabarovsk complex (Russian Far East): significance for subduction accretion timing and terrane correlation The Island Arc 1999 830 837")

79051

In [133]:
with open(path + '/na_v1_pub.json') as f:
    objects = ijson.items(f, 'zgpFhiSS')
    print ([*objects])

[{'id': 'zgpFhiSS', 'title': 'Was the easternmost segment of the Central Asian Orogenic Belt derived from Gondwana or Siberia: An intriguing dilemma?', 'abstract': "The Khanka Massif forms the easternmost segment of the Central Asian Orogenic Belt (CAOB) in NE China and its tectonic setting is highly controversial. Metasedimentary rocks from the Hulin Complex and the Mt. Huoshi sequence were selected to address this issue. For the Hulin Complex, SHRIMP detrital zircon U–Pb dating of a two-mica schist (sample HL-011) reveals a unimodal population with a weighted mean 206 Pb/ 238 U age of 257 ± 3 Ma, suggesting local derivation from nearby Permian granitoids. In contrast, a quartz schist (sample 04H-138) contains five zircon populations with ages of 233–270 Ma, 350–400 Ma, 473–570 Ma, 720–760 Ma and 930–970 Ma, and peaks at 252, 350, 520, 758 and 970 Ma, indicating a much more diverse provenance. Two samples of fine-grained gneiss were selected from Mt Houshi, with sample 06H-15 containi

In [110]:
with open(path + '/na_v1_pub.json') as f:
    objects = ijson.items(f, 'KDgsDUXr')
    print ([*objects])

[{'id': 'KDgsDUXr', 'title': 'Genetic influences on schizophrenia and subcortical brain volumes: large-scale proof of concept', 'authors': [{'name': 'Barbara Franke Kimm J E van Hulzen Alejandro Arias-Vasquez Janita Bralten Martine Hoogman Marieke Klein Marjolein M J van Donkelaar Marina M H Hakobjan Angelien J G A M Heister Remco R R Makkinje Marlies A M Naber Saskia S L van der Marel Jeanette C Mostert Han G Brunner  Hans van Bokhoven', 'org': 'Department of Human Genetics, Radboud University Medical Center, Nijmegen, the Netherlands.'}, {'name': 'Barbara Franke Kimm J E van Hulzen Alejandro Arias-Vasquez Janita Bralten Martine Hoogman Marieke Klein Marjolein M J van Donkelaar Marina M H Hakobjan Angelien J G A M Heister Remco R R Makkinje Marlies A M Naber Saskia S L van der Marel Jeanette C Mostert Han G Brunner  Hans van Bokhoven', 'org': 'Department of Human Genetics, Radboud University Medical Center, Nijmegen, the Netherlands.'}, {'name': 'Barbara Franke Kimm J E van Hulzen Ale

In [111]:
[{'id': 'KDgsDUXr', 'title': 'Genetic influences on schizophrenia and subcortical brain volumes: large-scale proof of concept', 'authors': [{'name': 'Barbara Franke Kimm J E van Hulzen Alejandro Arias-Vasquez Janita Bralten Martine Hoogman Marieke Klein Marjolein M J van Donkelaar Marina M H Hakobjan Angelien J G A M Heister Remco R R Makkinje Marlies A M Naber Saskia S L van der Marel Jeanette C Mostert Han G Brunner  Hans van Bokhoven', 'org': 'Department of Human Genetics, Radboud University Medical Center, Nijmegen, the Netherlands.'}, {'name': 'Barbara Franke Kimm J E van Hulzen Alejandro Arias-Vasquez Janita Bralten Martine Hoogman Marieke Klein Marjolein M J van Donkelaar Marina M H Hakobjan Angelien J G A M Heister Remco R R Makkinje Marlies A M Naber Saskia S L van der Marel Jeanette C Mostert Han G Brunner  Hans van Bokhoven', 'org': 'Department of Human Genetics, Radboud University Medical Center, Nijmegen, the Netherlands.'}, {'name': 'Barbara Franke Kimm J E van Hulzen Alejandro Arias-Vasquez Janita Bralten Martine Hoogman Marieke Klein Marjolein M J van Donkelaar Marina M H Hakobjan Angelien J G A M Heister Remco R R Makkinje Marlies A M Naber Saskia S L van der Marel Jeanette C Mostert Han G Brunner  Hans van Bokhoven', 'org': 'Department of Human Genetics, Radboud University Medical Center, Nijmegen, the Netherlands.'}, {'name': 'Barbara Franke Kimm J E van Hulzen Alejandro Arias-Vasquez Janita Bralten Martine Hoogman Marieke Klein Marjolein M J van Donkelaar Marina M H Hakobjan Angelien J G A M Heister Remco R R Makkinje Marlies A M Naber Saskia S L van der Marel Jeanette C Mostert Han G Brunner  Hans van Bokhoven', 'org': 'Department of Human Genetics, Radboud University Medical Center, Nijmegen, the Netherlands.'}, {'name': 'Barbara Franke Kimm J E van Hulzen Alejandro Arias-Vasquez Janita Bralten Martine Hoogman Marieke Klein Marjolein M J van Donkelaar Marina M H Hakobjan Angelien J G A M Heister Remco R R Makkinje Marlies A M Naber Saskia S L van der Marel Jeanette C Mostert Han G Brunner  Hans van Bokhoven', 'org': 'Department of Human Genetics, Radboud University Medical Center, Nijmegen, the Netherlands.'}, {'name': 'Barbara Franke Kimm J E van Hulzen Alejandro Arias-Vasquez Janita Bralten Martine Hoogman Marieke Klein Marjolein M J van Donkelaar Marina M H Hakobjan Angelien J G A M Heister Remco R R Makkinje Marlies A M Naber Saskia S L van der Marel Jeanette C Mostert Han G Brunner  Hans van Bokhoven', 'org': 'Department of Human Genetics, Radboud University Medical Center, Nijmegen, the Netherlands.'}, {'name': 'Barbara Franke Kimm J E van Hulzen Alejandro Arias-Vasquez Janita Bralten Martine Hoogman Marieke Klein Marjolein M J van Donkelaar Marina M H Hakobjan Angelien J G A M Heister Remco R R Makkinje Marlies A M Naber Saskia S L van der Marel Jeanette C Mostert Han G Brunner  Hans van Bokhoven', 'org': 'Department of Human Genetics, Radboud University Medical Center, Nijmegen, the Netherlands.'}, {'name': 'Barbara Franke Kimm J E van Hulzen Alejandro Arias-Vasquez Janita Bralten Martine Hoogman Marieke Klein Marjolein M J van Donkelaar Marina M H Hakobjan Angelien J G A M Heister Remco R R Makkinje Marlies A M Naber Saskia S L van der Marel Jeanette C Mostert Han G Brunner  Hans van Bokhoven', 'org': 'Department of Human Genetics, Radboud University Medical Center, Nijmegen, the Netherlands.'}, {'name': 'Barbara Franke Kimm J E van Hulzen Alejandro Arias-Vasquez Janita Bralten Martine Hoogman Marieke Klein Marjolein M J van Donkelaar Marina M H Hakobjan Angelien J G A M Heister Remco R R Makkinje Marlies A M Naber Saskia S L van der Marel Jeanette C Mostert Han G Brunner  Hans van Bokhoven', 'org': 'Department of Human Genetics, Radboud University Medical Center, Nijmegen, the Netherlands.'}, {'name': 'Barbara Franke Kimm J E van Hulzen Alejandro Arias-Vasquez Janita Bralten Martine Hoogman Marieke Klein Marjolein M J van Donkelaar Marina M H Hakobjan Angelien J G A M Heister Remco R R Makkinje Marlies A M Naber Saskia S L van der Marel Jeanette C Mostert Han G Brunner  Hans van Bokhoven', 'org': 'Department of Human Genetics, Radboud University Medical Center, Nijmegen, the Netherlands.'}, {'name': 'Barbara Franke Kimm J E van Hulzen Alejandro Arias-Vasquez Janita Bralten Martine Hoogman Marieke Klein Marjolein M J van Donkelaar Marina M H Hakobjan Angelien J G A M Heister Remco R R Makkinje Marlies A M Naber Saskia S L van der Marel Jeanette C Mostert Han G Brunner  Hans van Bokhoven', 'org': 'Department of Human Genetics, Radboud University Medical Center, Nijmegen, the Netherlands.'}, {'name': 'Barbara Franke Kimm J E van Hulzen Alejandro Arias-Vasquez Janita Bralten Martine Hoogman Marieke Klein Marjolein M J van Donkelaar Marina M H Hakobjan Angelien J G A M Heister Remco R R Makkinje Marlies A M Naber Saskia S L van der Marel Jeanette C Mostert Han G Brunner  Hans van Bokhoven', 'org': 'Department of Human Genetics, Radboud University Medical Center, Nijmegen, the Netherlands.'}, {'name': 'Barbara Franke Kimm J E van Hulzen Alejandro Arias-Vasquez Janita Bralten Martine Hoogman Marieke Klein Marjolein M J van Donkelaar Marina M H Hakobjan Angelien J G A M Heister Remco R R Makkinje Marlies A M Naber Saskia S L van der Marel Jeanette C Mostert Han G Brunner  Hans van Bokhoven', 'org': 'Department of Human Genetics, Radboud University Medical Center, Nijmegen, the Netherlands.'}, {'name': 'Barbara Franke Kimm J E van Hulzen Alejandro Arias-Vasquez Janita Bralten Martine Hoogman Marieke Klein Marjolein M J van Donkelaar Marina M H Hakobjan Angelien J G A M Heister Remco R R Makkinje Marlies A M Naber Saskia S L van der Marel Jeanette C Mostert Han G Brunner  Hans van Bokhoven', 'org': 'Department of Human Genetics, Radboud University Medical Center, Nijmegen, the Netherlands.'}, {'name': 'Barbara Franke Kimm J E van Hulzen Alejandro Arias-Vasquez Janita Bralten Martine Hoogman Marieke Klein Marjolein M J van Donkelaar Marina M H Hakobjan Angelien J G A M Heister Remco R R Makkinje Marlies A M Naber Saskia S L van der Marel Jeanette C Mostert Han G Brunner  Hans van Bokhoven', 'org': 'Department of Human Genetics, Radboud University Medical Center, Nijmegen, the Netherlands.'}, {'name': 'Barbara Franke  Alejandro Arias-Vasquez', 'org': 'Department of Psychiatry, Radboud University Medical Center, Nijmegen, the Netherlands.'}, {'name': 'Barbara Franke  Alejandro Arias-Vasquez', 'org': 'Department of Psychiatry, Radboud University Medical Center, Nijmegen, the Netherlands.'}, {'name': 'Barbara Franke Kimm J E van Hulzen Alejandro Arias-Vasquez Janita Bralten Martine Hoogman Marieke Klein Marjolein M J van Donkelaar Marcel P Zwiers Marina M H Hakobjan Angelien J G A M Heister Remco R R Makkinje Marlies A M Naber Saskia S L van der Marel Jeanette C Mostert Han G Brunner Jan K Buitelaar Hans van Bokhoven Guillén Fernández Simon E Fisher  Clyde Francks', 'org': 'Donders Institute for Brain, Cognition and Behaviour, Raboud University, Nijmegen, the Netherlands.'}, {'name': 'Barbara Franke Kimm J E van Hulzen Alejandro Arias-Vasquez Janita Bralten Martine Hoogman Marieke Klein Marjolein M J van Donkelaar Marcel P Zwiers Marina M H Hakobjan Angelien J G A M Heister Remco R R Makkinje Marlies A M Naber Saskia S L van der Marel Jeanette C Mostert Han G Brunner Jan K Buitelaar Hans van Bokhoven Guillén Fernández Simon E Fisher  Clyde Francks', 'org': 'Donders Institute for Brain, Cognition and Behaviour, Raboud University, Nijmegen, the Netherlands.'}, {'name': 'Barbara Franke Kimm J E van Hulzen Alejandro Arias-Vasquez Janita Bralten Martine Hoogman Marieke Klein Marjolein M J van Donkelaar Marcel P Zwiers Marina M H Hakobjan Angelien J G A M Heister Remco R R Makkinje Marlies A M Naber Saskia S L van der Marel Jeanette C Mostert Han G Brunner Jan K Buitelaar Hans van Bokhoven Guillén Fernández Simon E Fisher  Clyde Francks', 'org': 'Donders Institute for Brain, Cognition and Behaviour, Raboud University, Nijmegen, the Netherlands.'}, {'name': 'Barbara Franke Kimm J E van Hulzen Alejandro Arias-Vasquez Janita Bralten Martine Hoogman Marieke Klein Marjolein M J van Donkelaar Marcel P Zwiers Marina M H Hakobjan Angelien J G A M Heister Remco R R Makkinje Marlies A M Naber Saskia S L van der Marel Jeanette C Mostert Han G Brunner Jan K Buitelaar Hans van Bokhoven Guillén Fernández Simon E Fisher  Clyde Francks', 'org': 'Donders Institute for Brain, Cognition and Behaviour, Raboud University, Nijmegen, the Netherlands.'}, {'name': 'Barbara Franke Kimm J E van Hulzen Alejandro Arias-Vasquez Janita Bralten Martine Hoogman Marieke Klein Marjolein M J van Donkelaar Marcel P Zwiers Marina M H Hakobjan Angelien J G A M Heister Remco R R Makkinje Marlies A M Naber Saskia S L van der Marel Jeanette C Mostert Han G Brunner Jan K Buitelaar Hans van Bokhoven Guillén Fernández Simon E Fisher  Clyde Francks', 'org': 'Donders Institute for Brain, Cognition and Behaviour, Raboud University, Nijmegen, the Netherlands.'}, {'name': 'Barbara Franke Kimm J E van Hulzen Alejandro Arias-Vasquez Janita Bralten Martine Hoogman Marieke Klein Marjolein M J van Donkelaar Marcel P Zwiers Marina M H Hakobjan Angelien J G A M Heister Remco R R Makkinje Marlies A M Naber Saskia S L van der Marel Jeanette C Mostert Han G Brunner Jan K Buitelaar Hans van Bokhoven Guillén Fernández Simon E Fisher  Clyde Francks', 'org': 'Donders Institute for Brain, Cognition and Behaviour, Raboud University, Nijmegen, the Netherlands.'}, {'name': 'Barbara Franke Kimm J E van Hulzen Alejandro Arias-Vasquez Janita Bralten Martine Hoogman Marieke Klein Marjolein M J van Donkelaar Marcel P Zwiers Marina M H Hakobjan Angelien J G A M Heister Remco R R Makkinje Marlies A M Naber Saskia S L van der Marel Jeanette C Mostert Han G Brunner Jan K Buitelaar Hans van Bokhoven Guillén Fernández Simon E Fisher  Clyde Francks', 'org': 'Donders Institute for Brain, Cognition and Behaviour, Raboud University, Nijmegen, the Netherlands.'}, {'name': 'Barbara Franke Kimm J E van Hulzen Alejandro Arias-Vasquez Janita Bralten Martine Hoogman Marieke Klein Marjolein M J van Donkelaar Marcel P Zwiers Marina M H Hakobjan Angelien J G A M Heister Remco R R Makkinje Marlies A M Naber Saskia S L van der Marel Jeanette C Mostert Han G Brunner Jan K Buitelaar Hans van Bokhoven Guillén Fernández Simon E Fisher  Clyde Francks', 'org': 'Donders Institute for Brain, Cognition and Behaviour, Raboud University, Nijmegen, the Netherlands.'}, {'name': 'Barbara Franke Kimm J E van Hulzen Alejandro Arias-Vasquez Janita Bralten Martine Hoogman Marieke Klein Marjolein M J van Donkelaar Marcel P Zwiers Marina M H Hakobjan Angelien J G A M Heister Remco R R Makkinje Marlies A M Naber Saskia S L van der Marel Jeanette C Mostert Han G Brunner Jan K Buitelaar Hans van Bokhoven Guillén Fernández Simon E Fisher  Clyde Francks', 'org': 'Donders Institute for Brain, Cognition and Behaviour, Raboud University, Nijmegen, the Netherlands.'}, {'name': 'Barbara Franke Kimm J E van Hulzen Alejandro Arias-Vasquez Janita Bralten Martine Hoogman Marieke Klein Marjolein M J van Donkelaar Marcel P Zwiers Marina M H Hakobjan Angelien J G A M Heister Remco R R Makkinje Marlies A M Naber Saskia S L van der Marel Jeanette C Mostert Han G Brunner Jan K Buitelaar Hans van Bokhoven Guillén Fernández Simon E Fisher  Clyde Francks', 'org': 'Donders Institute for Brain, Cognition and Behaviour, Raboud University, Nijmegen, the Netherlands.'}, {'name': 'Barbara Franke Kimm J E van Hulzen Alejandro Arias-Vasquez Janita Bralten Martine Hoogman Marieke Klein Marjolein M J van Donkelaar Marcel P Zwiers Marina M H Hakobjan Angelien J G A M Heister Remco R R Makkinje Marlies A M Naber Saskia S L van der Marel Jeanette C Mostert Han G Brunner Jan K Buitelaar Hans van Bokhoven Guillén Fernández Simon E Fisher  Clyde Francks', 'org': 'Donders Institute for Brain, Cognition and Behaviour, Raboud University, Nijmegen, the Netherlands.'}, {'name': 'Barbara Franke Kimm J E van Hulzen Alejandro Arias-Vasquez Janita Bralten Martine Hoogman Marieke Klein Marjolein M J van Donkelaar Marcel P Zwiers Marina M H Hakobjan Angelien J G A M Heister Remco R R Makkinje Marlies A M Naber Saskia S L van der Marel Jeanette C Mostert Han G Brunner Jan K Buitelaar Hans van Bokhoven Guillén Fernández Simon E Fisher  Clyde Francks', 'org': 'Donders Institute for Brain, Cognition and Behaviour, Raboud University, Nijmegen, the Netherlands.'}, {'name': 'Barbara Franke Kimm J E van Hulzen Alejandro Arias-Vasquez Janita Bralten Martine Hoogman Marieke Klein Marjolein M J van Donkelaar Marcel P Zwiers Marina M H Hakobjan Angelien J G A M Heister Remco R R Makkinje Marlies A M Naber Saskia S L van der Marel Jeanette C Mostert Han G Brunner Jan K Buitelaar Hans van Bokhoven Guillén Fernández Simon E Fisher  Clyde Francks', 'org': 'Donders Institute for Brain, Cognition and Behaviour, Raboud University, Nijmegen, the Netherlands.'}, {'name': 'Barbara Franke Kimm J E van Hulzen Alejandro Arias-Vasquez Janita Bralten Martine Hoogman Marieke Klein Marjolein M J van Donkelaar Marcel P Zwiers Marina M H Hakobjan Angelien J G A M Heister Remco R R Makkinje Marlies A M Naber Saskia S L van der Marel Jeanette C Mostert Han G Brunner Jan K Buitelaar Hans van Bokhoven Guillén Fernández Simon E Fisher  Clyde Francks', 'org': 'Donders Institute for Brain, Cognition and Behaviour, Raboud University, Nijmegen, the Netherlands.'}, {'name': 'Barbara Franke Kimm J E van Hulzen Alejandro Arias-Vasquez Janita Bralten Martine Hoogman Marieke Klein Marjolein M J van Donkelaar Marcel P Zwiers Marina M H Hakobjan Angelien J G A M Heister Remco R R Makkinje Marlies A M Naber Saskia S L van der Marel Jeanette C Mostert Han G Brunner Jan K Buitelaar Hans van Bokhoven Guillén Fernández Simon E Fisher  Clyde Francks', 'org': 'Donders Institute for Brain, Cognition and Behaviour, Raboud University, Nijmegen, the Netherlands.'}, {'name': 'Barbara Franke Kimm J E van Hulzen Alejandro Arias-Vasquez Janita Bralten Martine Hoogman Marieke Klein Marjolein M J van Donkelaar Marcel P Zwiers Marina M H Hakobjan Angelien J G A M Heister Remco R R Makkinje Marlies A M Naber Saskia S L van der Marel Jeanette C Mostert Han G Brunner Jan K Buitelaar Hans van Bokhoven Guillén Fernández Simon E Fisher  Clyde Francks', 'org': 'Donders Institute for Brain, Cognition and Behaviour, Raboud University, Nijmegen, the Netherlands.'}, {'name': 'Barbara Franke Kimm J E van Hulzen Alejandro Arias-Vasquez Janita Bralten Martine Hoogman Marieke Klein Marjolein M J van Donkelaar Marcel P Zwiers Marina M H Hakobjan Angelien J G A M Heister Remco R R Makkinje Marlies A M Naber Saskia S L van der Marel Jeanette C Mostert Han G Brunner Jan K Buitelaar Hans van Bokhoven Guillén Fernández Simon E Fisher  Clyde Francks', 'org': 'Donders Institute for Brain, Cognition and Behaviour, Raboud University, Nijmegen, the Netherlands.'}, {'name': 'Barbara Franke Kimm J E van Hulzen Alejandro Arias-Vasquez Janita Bralten Martine Hoogman Marieke Klein Marjolein M J van Donkelaar Marcel P Zwiers Marina M H Hakobjan Angelien J G A M Heister Remco R R Makkinje Marlies A M Naber Saskia S L van der Marel Jeanette C Mostert Han G Brunner Jan K Buitelaar Hans van Bokhoven Guillén Fernández Simon E Fisher  Clyde Francks', 'org': 'Donders Institute for Brain, Cognition and Behaviour, Raboud University, Nijmegen, the Netherlands.'}, {'name': 'Barbara Franke Kimm J E van Hulzen Alejandro Arias-Vasquez Janita Bralten Martine Hoogman Marieke Klein Marjolein M J van Donkelaar Marcel P Zwiers Marina M H Hakobjan Angelien J G A M Heister Remco R R Makkinje Marlies A M Naber Saskia S L van der Marel Jeanette C Mostert Han G Brunner Jan K Buitelaar Hans van Bokhoven Guillén Fernández Simon E Fisher  Clyde Francks', 'org': 'Donders Institute for Brain, Cognition and Behaviour, Raboud University, Nijmegen, the Netherlands.'}, {'name': 'Barbara Franke Kimm J E van Hulzen Alejandro Arias-Vasquez Janita Bralten Martine Hoogman Marieke Klein Marjolein M J van Donkelaar Marcel P Zwiers Marina M H Hakobjan Angelien J G A M Heister Remco R R Makkinje Marlies A M Naber Saskia S L van der Marel Jeanette C Mostert Han G Brunner Jan K Buitelaar Hans van Bokhoven Guillén Fernández Simon E Fisher  Clyde Francks', 'org': 'Donders Institute for Brain, Cognition and Behaviour, Raboud University, Nijmegen, the Netherlands.'}, {'name': 'Jason L Stein Derrek P Hibar  Paul M Thompson', 'org': 'Imaging Genetics Center, Mark and Mary Stevens Neuroimaging & Informatics Institute, Keck School of Medicine of the University of Southern California, Marina del Rey, California, USA.'}, {'name': 'Jason L Stein Derrek P Hibar  Paul M Thompson', 'org': 'Imaging Genetics Center, Mark and Mary Stevens Neuroimaging & Informatics Institute, Keck School of Medicine of the University of Southern California, Marina del Rey, California, USA.'}, {'name': 'Jason L Stein Derrek P Hibar  Paul M Thompson', 'org': 'Imaging Genetics Center, Mark and Mary Stevens Neuroimaging & Informatics Institute, Keck School of Medicine of the University of Southern California, Marina del Rey, California, USA.'}, {'name': 'Jason L Stein', 'org': 'Neurogenetics Program, Department of Neurology, UCLA School of Medicine, Los Angeles, California, USA.'}, {'name': 'Stephan Ripke Verneri Anttila Benjamin M Neale Kai-How Farh Brendan Bulik-Sullivan Hailiang Huang Menachem Fromer Jacqueline I Goldstein Mark J Daly  Raymond K Walters', 'org': 'Analytic and Translational Genetics Unit, Massachusetts General Hospital, Boston, Massachusetts, USA.'}, {'name': 'Stephan Ripke Verneri Anttila Benjamin M Neale Kai-How Farh Brendan Bulik-Sullivan Hailiang Huang Menachem Fromer Jacqueline I Goldstein Mark J Daly  Raymond K Walters', 'org': 'Analytic and Translational Genetics Unit, Massachusetts General Hospital, Boston, Massachusetts, USA.'}, {'name': 'Stephan Ripke Verneri Anttila Benjamin M Neale Kai-How Farh Brendan Bulik-Sullivan Hailiang Huang Menachem Fromer Jacqueline I Goldstein Mark J Daly  Raymond K Walters', 'org': 'Analytic and Translational Genetics Unit, Massachusetts General Hospital, Boston, Massachusetts, USA.'}, {'name': 'Stephan Ripke Verneri Anttila Benjamin M Neale Kai-How Farh Brendan Bulik-Sullivan Hailiang Huang Menachem Fromer Jacqueline I Goldstein Mark J Daly  Raymond K Walters', 'org': 'Analytic and Translational Genetics Unit, Massachusetts General Hospital, Boston, Massachusetts, USA.'}, {'name': 'Stephan Ripke Verneri Anttila Benjamin M Neale Kai-How Farh Brendan Bulik-Sullivan Hailiang Huang Menachem Fromer Jacqueline I Goldstein Mark J Daly  Raymond K Walters', 'org': 'Analytic and Translational Genetics Unit, Massachusetts General Hospital, Boston, Massachusetts, USA.'}, {'name': 'Stephan Ripke Verneri Anttila Benjamin M Neale Kai-How Farh Brendan Bulik-Sullivan Hailiang Huang Menachem Fromer Jacqueline I Goldstein Mark J Daly  Raymond K Walters', 'org': 'Analytic and Translational Genetics Unit, Massachusetts General Hospital, Boston, Massachusetts, USA.'}, {'name': 'Stephan Ripke Verneri Anttila Benjamin M Neale Kai-How Farh Brendan Bulik-Sullivan Hailiang Huang Menachem Fromer Jacqueline I Goldstein Mark J Daly  Raymond K Walters', 'org': 'Analytic and Translational Genetics Unit, Massachusetts General Hospital, Boston, Massachusetts, USA.'}, {'name': 'Stephan Ripke Verneri Anttila Benjamin M Neale Kai-How Farh Brendan Bulik-Sullivan Hailiang Huang Menachem Fromer Jacqueline I Goldstein Mark J Daly  Raymond K Walters', 'org': 'Analytic and Translational Genetics Unit, Massachusetts General Hospital, Boston, Massachusetts, USA.'}, {'name': 'Stephan Ripke Verneri Anttila Benjamin M Neale Kai-How Farh Brendan Bulik-Sullivan Hailiang Huang Menachem Fromer Jacqueline I Goldstein Mark J Daly  Raymond K Walters', 'org': 'Analytic and Translational Genetics Unit, Massachusetts General Hospital, Boston, Massachusetts, USA.'}, {'name': 'Stephan Ripke Verneri Anttila Benjamin M Neale Kai-How Farh Brendan Bulik-Sullivan Hailiang Huang Menachem Fromer Jacqueline I Goldstein Mark J Daly  Raymond K Walters', 'org': 'Analytic and Translational Genetics Unit, Massachusetts General Hospital, Boston, Massachusetts, USA.'}, {'name': "Stephan Ripke Verneri Anttila Jordan W Smoller Phil Lee Benjamin M Neale Brendan Bulik-Sullivan Richard A Belliveau Jr Sarah E Bergen Elizabeth Bevilacqua Kimberley D Chambert Menachem Fromer Giulio Genovese Colm O'Dushlaine Edward M Scolnick Steven A McCarroll Jennifer L Moran Aarno Palotie Tracey L Petryshen  Mark J Daly", 'org': 'Stanley Center for Psychiatric Research, Broad Institute of MIT and Harvard, Cambridge, Massachusetts, USA.'}, {'name': "Stephan Ripke Verneri Anttila Jordan W Smoller Phil Lee Benjamin M Neale Brendan Bulik-Sullivan Richard A Belliveau Jr Sarah E Bergen Elizabeth Bevilacqua Kimberley D Chambert Menachem Fromer Giulio Genovese Colm O'Dushlaine Edward M Scolnick Steven A McCarroll Jennifer L Moran Aarno Palotie Tracey L Petryshen  Mark J Daly", 'org': 'Stanley Center for Psychiatric Research, Broad Institute of MIT and Harvard, Cambridge, Massachusetts, USA.'}, {'name': "Stephan Ripke Verneri Anttila Jordan W Smoller Phil Lee Benjamin M Neale Brendan Bulik-Sullivan Richard A Belliveau Jr Sarah E Bergen Elizabeth Bevilacqua Kimberley D Chambert Menachem Fromer Giulio Genovese Colm O'Dushlaine Edward M Scolnick Steven A McCarroll Jennifer L Moran Aarno Palotie Tracey L Petryshen  Mark J Daly", 'org': 'Stanley Center for Psychiatric Research, Broad Institute of MIT and Harvard, Cambridge, Massachusetts, USA.'}, {'name': "Stephan Ripke Verneri Anttila Jordan W Smoller Phil Lee Benjamin M Neale Brendan Bulik-Sullivan Richard A Belliveau Jr Sarah E Bergen Elizabeth Bevilacqua Kimberley D Chambert Menachem Fromer Giulio Genovese Colm O'Dushlaine Edward M Scolnick Steven A McCarroll Jennifer L Moran Aarno Palotie Tracey L Petryshen  Mark J Daly", 'org': 'Stanley Center for Psychiatric Research, Broad Institute of MIT and Harvard, Cambridge, Massachusetts, USA.'}, {'name': "Stephan Ripke Verneri Anttila Jordan W Smoller Phil Lee Benjamin M Neale Brendan Bulik-Sullivan Richard A Belliveau Jr Sarah E Bergen Elizabeth Bevilacqua Kimberley D Chambert Menachem Fromer Giulio Genovese Colm O'Dushlaine Edward M Scolnick Steven A McCarroll Jennifer L Moran Aarno Palotie Tracey L Petryshen  Mark J Daly", 'org': 'Stanley Center for Psychiatric Research, Broad Institute of MIT and Harvard, Cambridge, Massachusetts, USA.'}, {'name': "Stephan Ripke Verneri Anttila Jordan W Smoller Phil Lee Benjamin M Neale Brendan Bulik-Sullivan Richard A Belliveau Jr Sarah E Bergen Elizabeth Bevilacqua Kimberley D Chambert Menachem Fromer Giulio Genovese Colm O'Dushlaine Edward M Scolnick Steven A McCarroll Jennifer L Moran Aarno Palotie Tracey L Petryshen  Mark J Daly", 'org': 'Stanley Center for Psychiatric Research, Broad Institute of MIT and Harvard, Cambridge, Massachusetts, USA.'}, {'name': "Stephan Ripke Verneri Anttila Jordan W Smoller Phil Lee Benjamin M Neale Brendan Bulik-Sullivan Richard A Belliveau Jr Sarah E Bergen Elizabeth Bevilacqua Kimberley D Chambert Menachem Fromer Giulio Genovese Colm O'Dushlaine Edward M Scolnick Steven A McCarroll Jennifer L Moran Aarno Palotie Tracey L Petryshen  Mark J Daly", 'org': 'Stanley Center for Psychiatric Research, Broad Institute of MIT and Harvard, Cambridge, Massachusetts, USA.'}, {'name': "Stephan Ripke Verneri Anttila Jordan W Smoller Phil Lee Benjamin M Neale Brendan Bulik-Sullivan Richard A Belliveau Jr Sarah E Bergen Elizabeth Bevilacqua Kimberley D Chambert Menachem Fromer Giulio Genovese Colm O'Dushlaine Edward M Scolnick Steven A McCarroll Jennifer L Moran Aarno Palotie Tracey L Petryshen  Mark J Daly", 'org': 'Stanley Center for Psychiatric Research, Broad Institute of MIT and Harvard, Cambridge, Massachusetts, USA.'}, {'name': "Stephan Ripke Verneri Anttila Jordan W Smoller Phil Lee Benjamin M Neale Brendan Bulik-Sullivan Richard A Belliveau Jr Sarah E Bergen Elizabeth Bevilacqua Kimberley D Chambert Menachem Fromer Giulio Genovese Colm O'Dushlaine Edward M Scolnick Steven A McCarroll Jennifer L Moran Aarno Palotie Tracey L Petryshen  Mark J Daly", 'org': 'Stanley Center for Psychiatric Research, Broad Institute of MIT and Harvard, Cambridge, Massachusetts, USA.'}, {'name': "Stephan Ripke Verneri Anttila Jordan W Smoller Phil Lee Benjamin M Neale Brendan Bulik-Sullivan Richard A Belliveau Jr Sarah E Bergen Elizabeth Bevilacqua Kimberley D Chambert Menachem Fromer Giulio Genovese Colm O'Dushlaine Edward M Scolnick Steven A McCarroll Jennifer L Moran Aarno Palotie Tracey L Petryshen  Mark J Daly", 'org': 'Stanley Center for Psychiatric Research, Broad Institute of MIT and Harvard, Cambridge, Massachusetts, USA.'}, {'name': "Stephan Ripke Verneri Anttila Jordan W Smoller Phil Lee Benjamin M Neale Brendan Bulik-Sullivan Richard A Belliveau Jr Sarah E Bergen Elizabeth Bevilacqua Kimberley D Chambert Menachem Fromer Giulio Genovese Colm O'Dushlaine Edward M Scolnick Steven A McCarroll Jennifer L Moran Aarno Palotie Tracey L Petryshen  Mark J Daly", 'org': 'Stanley Center for Psychiatric Research, Broad Institute of MIT and Harvard, Cambridge, Massachusetts, USA.'}, {'name': "Stephan Ripke Verneri Anttila Jordan W Smoller Phil Lee Benjamin M Neale Brendan Bulik-Sullivan Richard A Belliveau Jr Sarah E Bergen Elizabeth Bevilacqua Kimberley D Chambert Menachem Fromer Giulio Genovese Colm O'Dushlaine Edward M Scolnick Steven A McCarroll Jennifer L Moran Aarno Palotie Tracey L Petryshen  Mark J Daly", 'org': 'Stanley Center for Psychiatric Research, Broad Institute of MIT and Harvard, Cambridge, Massachusetts, USA.'}, {'name': "Stephan Ripke Verneri Anttila Jordan W Smoller Phil Lee Benjamin M Neale Brendan Bulik-Sullivan Richard A Belliveau Jr Sarah E Bergen Elizabeth Bevilacqua Kimberley D Chambert Menachem Fromer Giulio Genovese Colm O'Dushlaine Edward M Scolnick Steven A McCarroll Jennifer L Moran Aarno Palotie Tracey L Petryshen  Mark J Daly", 'org': 'Stanley Center for Psychiatric Research, Broad Institute of MIT and Harvard, Cambridge, Massachusetts, USA.'}, {'name': "Stephan Ripke Verneri Anttila Jordan W Smoller Phil Lee Benjamin M Neale Brendan Bulik-Sullivan Richard A Belliveau Jr Sarah E Bergen Elizabeth Bevilacqua Kimberley D Chambert Menachem Fromer Giulio Genovese Colm O'Dushlaine Edward M Scolnick Steven A McCarroll Jennifer L Moran Aarno Palotie Tracey L Petryshen  Mark J Daly", 'org': 'Stanley Center for Psychiatric Research, Broad Institute of MIT and Harvard, Cambridge, Massachusetts, USA.'}, {'name': "Stephan Ripke Verneri Anttila Jordan W Smoller Phil Lee Benjamin M Neale Brendan Bulik-Sullivan Richard A Belliveau Jr Sarah E Bergen Elizabeth Bevilacqua Kimberley D Chambert Menachem Fromer Giulio Genovese Colm O'Dushlaine Edward M Scolnick Steven A McCarroll Jennifer L Moran Aarno Palotie Tracey L Petryshen  Mark J Daly", 'org': 'Stanley Center for Psychiatric Research, Broad Institute of MIT and Harvard, Cambridge, Massachusetts, USA.'}, {'name': "Stephan Ripke Verneri Anttila Jordan W Smoller Phil Lee Benjamin M Neale Brendan Bulik-Sullivan Richard A Belliveau Jr Sarah E Bergen Elizabeth Bevilacqua Kimberley D Chambert Menachem Fromer Giulio Genovese Colm O'Dushlaine Edward M Scolnick Steven A McCarroll Jennifer L Moran Aarno Palotie Tracey L Petryshen  Mark J Daly", 'org': 'Stanley Center for Psychiatric Research, Broad Institute of MIT and Harvard, Cambridge, Massachusetts, USA.'}, {'name': "Stephan Ripke Verneri Anttila Jordan W Smoller Phil Lee Benjamin M Neale Brendan Bulik-Sullivan Richard A Belliveau Jr Sarah E Bergen Elizabeth Bevilacqua Kimberley D Chambert Menachem Fromer Giulio Genovese Colm O'Dushlaine Edward M Scolnick Steven A McCarroll Jennifer L Moran Aarno Palotie Tracey L Petryshen  Mark J Daly", 'org': 'Stanley Center for Psychiatric Research, Broad Institute of MIT and Harvard, Cambridge, Massachusetts, USA.'}, {'name': "Stephan Ripke Verneri Anttila Jordan W Smoller Phil Lee Benjamin M Neale Brendan Bulik-Sullivan Richard A Belliveau Jr Sarah E Bergen Elizabeth Bevilacqua Kimberley D Chambert Menachem Fromer Giulio Genovese Colm O'Dushlaine Edward M Scolnick Steven A McCarroll Jennifer L Moran Aarno Palotie Tracey L Petryshen  Mark J Daly", 'org': 'Stanley Center for Psychiatric Research, Broad Institute of MIT and Harvard, Cambridge, Massachusetts, USA.'}, {'name': "Stephan Ripke Verneri Anttila Jordan W Smoller Phil Lee Benjamin M Neale Brendan Bulik-Sullivan Richard A Belliveau Jr Sarah E Bergen Elizabeth Bevilacqua Kimberley D Chambert Menachem Fromer Giulio Genovese Colm O'Dushlaine Edward M Scolnick Steven A McCarroll Jennifer L Moran Aarno Palotie Tracey L Petryshen  Mark J Daly", 'org': 'Stanley Center for Psychiatric Research, Broad Institute of MIT and Harvard, Cambridge, Massachusetts, USA.'}, {'name': 'Stephan Ripke Susanne Erk Andreas Heinz Sebastian Mohnke Nina Romanczuk-Seiferth  Henrik Walter', 'org': 'Department of Psychiatry and Psychotherapy, Charité Universitätsmedizin Berlin, Campus Charité Mitte, Berlin, Germany.'}, {'name': 'Stephan Ripke Susanne Erk Andreas Heinz Sebastian Mohnke Nina Romanczuk-Seiferth  Henrik Walter', 'org': 'Department of Psychiatry and Psychotherapy, Charité Universitätsmedizin Berlin, Campus Charité Mitte, Berlin, Germany.'}, {'name': 'Stephan Ripke Susanne Erk Andreas Heinz Sebastian Mohnke Nina Romanczuk-Seiferth  Henrik Walter', 'org': 'Department of Psychiatry and Psychotherapy, Charité Universitätsmedizin Berlin, Campus Charité Mitte, Berlin, Germany.'}, {'name': 'Stephan Ripke Susanne Erk Andreas Heinz Sebastian Mohnke Nina Romanczuk-Seiferth  Henrik Walter', 'org': 'Department of Psychiatry and Psychotherapy, Charité Universitätsmedizin Berlin, Campus Charité Mitte, Berlin, Germany.'}, {'name': 'Stephan Ripke Susanne Erk Andreas Heinz Sebastian Mohnke Nina Romanczuk-Seiferth  Henrik Walter', 'org': 'Department of Psychiatry and Psychotherapy, Charité Universitätsmedizin Berlin, Campus Charité Mitte, Berlin, Germany.'}, {'name': 'Stephan Ripke Susanne Erk Andreas Heinz Sebastian Mohnke Nina Romanczuk-Seiferth  Henrik Walter', 'org': 'Department of Psychiatry and Psychotherapy, Charité Universitätsmedizin Berlin, Campus Charité Mitte, Berlin, Germany.'}, {'name': 'Alejandro Arias-Vasquez Janita Bralten Jan K Buitelaar  Guillén Fernández', 'org': 'Department of Cognitive Neuroscience, Radboud University Medical Center, Nijmegen, the Netherlands.'}, {'name': 'Alejandro Arias-Vasquez Janita Bralten Jan K Buitelaar  Guillén Fernández', 'org': 'Department of Cognitive Neuroscience, Radboud University Medical Center, Nijmegen, the Netherlands.'}, {'name': 'Alejandro Arias-Vasquez Janita Bralten Jan K Buitelaar  Guillén Fernández', 'org': 'Department of Cognitive Neuroscience, Radboud University Medical Center, Nijmegen, the Netherlands.'}, {'name': 'Alejandro Arias-Vasquez Janita Bralten Jan K Buitelaar  Guillén Fernández', 'org': 'Department of Cognitive Neuroscience, Radboud University Medical Center, Nijmegen, the Netherlands.'}, {'name': 'Jordan W Smoller Phil Lee Benjamin M Neale Menachem Fromer  Aarno Palotie', 'org': 'Psychiatric and Neurodevelopmental Genetics Unit, Massachusetts General Hospital, Boston, Massachusetts, USA.'}, {'name': 'Jordan W Smoller Phil Lee Benjamin M Neale Menachem Fromer  Aarno Palotie', 'org': 'Psychiatric and Neurodevelopmental Genetics Unit, Massachusetts General Hospital, Boston, Massachusetts, USA.'}, {'name': 'Jordan W Smoller Phil Lee Benjamin M Neale Menachem Fromer  Aarno Palotie', 'org': 'Psychiatric and Neurodevelopmental Genetics Unit, Massachusetts General Hospital, Boston, Massachusetts, USA.'}, {'name': 'Jordan W Smoller Phil Lee Benjamin M Neale Menachem Fromer  Aarno Palotie', 'org': 'Psychiatric and Neurodevelopmental Genetics Unit, Massachusetts General Hospital, Boston, Massachusetts, USA.'}, {'name': 'Jordan W Smoller Phil Lee Benjamin M Neale Menachem Fromer  Aarno Palotie', 'org': 'Psychiatric and Neurodevelopmental Genetics Unit, Massachusetts General Hospital, Boston, Massachusetts, USA.'}, {'name': 'Jordan W Smoller Phil Lee Lynn E DeLisi Robert W McCarley Raquelle I Mesholam-Gately Larry J Seidman  Tracey L Petryshen', 'org': 'Department of Psychiatry, Harvard Medical School, Boston, Massachusetts, USA.'}, {'name': 'Jordan W Smoller Phil Lee Lynn E DeLisi Robert W McCarley Raquelle I Mesholam-Gately Larry J Seidman  Tracey L Petryshen', 'org': 'Department of Psychiatry, Harvard Medical School, Boston, Massachusetts, USA.'}, {'name': 'Jordan W Smoller Phil Lee Lynn E DeLisi Robert W McCarley Raquelle I Mesholam-Gately Larry J Seidman  Tracey L Petryshen', 'org': 'Department of Psychiatry, Harvard Medical School, Boston, Massachusetts, USA.'}, {'name': 'Jordan W Smoller Phil Lee Lynn E DeLisi Robert W McCarley Raquelle I Mesholam-Gately Larry J Seidman  Tracey L Petryshen', 'org': 'Department of Psychiatry, Harvard Medical School, Boston, Massachusetts, USA.'}, {'name': 'Jordan W Smoller Phil Lee Lynn E DeLisi Robert W McCarley Raquelle I Mesholam-Gately Larry J Seidman  Tracey L Petryshen', 'org': 'Department of Psychiatry, Harvard Medical School, Boston, Massachusetts, USA.'}, {'name': 'Jordan W Smoller Phil Lee Lynn E DeLisi Robert W McCarley Raquelle I Mesholam-Gately Larry J Seidman  Tracey L Petryshen', 'org': 'Department of Psychiatry, Harvard Medical School, Boston, Massachusetts, USA.'}, {'name': 'Jordan W Smoller Phil Lee Lynn E DeLisi Robert W McCarley Raquelle I Mesholam-Gately Larry J Seidman  Tracey L Petryshen', 'org': 'Department of Psychiatry, Harvard Medical School, Boston, Massachusetts, USA.'}, {'name': 'Thomas E Nichols', 'org': 'FMRIB Centre, University of Oxford, Oxford, UK.'}, {'name': 'Thomas E Nichols', 'org': 'Department of Statistics & Warwick Manufacturing Group, University of Warwick, Coventry, UK.'}, {'name': 'Michael C Neale', 'org': 'Departments of Psychiatry & Human Genetics, Virginia Commonwealth University, Richmond, Virginia, USA.'}, {'name': 'Andrew M McIntosh Martina Papmeyer Emma Sprooten Stephen M Lawrie  Jessika E Sussmann', 'org': 'Division of Psychiatry, Royal Edinburgh Hospital, Centre for Cognitive Ageing and Cognitive Epidemiology, University of Edinburgh, Edinburgh, UK.'}, {'name': 'Andrew M McIntosh Martina Papmeyer Emma Sprooten Stephen M Lawrie  Jessika E Sussmann', 'org': 'Division of Psychiatry, Royal Edinburgh Hospital, Centre for Cognitive Ageing and Cognitive Epidemiology, University of Edinburgh, Edinburgh, UK.'}, {'name': 'Andrew M McIntosh Martina Papmeyer Emma Sprooten Stephen M Lawrie  Jessika E Sussmann', 'org': 'Division of Psychiatry, Royal Edinburgh Hospital, Centre for Cognitive Ageing and Cognitive Epidemiology, University of Edinburgh, Edinburgh, UK.'}, {'name': 'Andrew M McIntosh Martina Papmeyer Emma Sprooten Stephen M Lawrie  Jessika E Sussmann', 'org': 'Division of Psychiatry, Royal Edinburgh Hospital, Centre for Cognitive Ageing and Cognitive Epidemiology, University of Edinburgh, Edinburgh, UK.'}, {'name': 'Andrew M McIntosh Martina Papmeyer Emma Sprooten Stephen M Lawrie  Jessika E Sussmann', 'org': 'Division of Psychiatry, Royal Edinburgh Hospital, Centre for Cognitive Ageing and Cognitive Epidemiology, University of Edinburgh, Edinburgh, UK.'}, {'name': 'Francis J McMahon  Yin Yao', 'org': 'Intramural Research Program, National Institutes of Health, US Department of Health & Human Services, Bethesda, Maryland, USA.'}, {'name': 'Francis J McMahon  Yin Yao', 'org': 'Intramural Research Program, National Institutes of Health, US Department of Health & Human Services, Bethesda, Maryland, USA.'}, {'name': 'Andreas Meyer-Lindenberg Emanuel Schwarz  Oliver Grimm', 'org': 'Central Institute of Mental Health, Medical Faculty Mannheim, University Heidelberg, Mannheim, Germany.'}, {'name': 'Andreas Meyer-Lindenberg Emanuel Schwarz  Oliver Grimm', 'org': 'Central Institute of Mental Health, Medical Faculty Mannheim, University Heidelberg, Mannheim, Germany.'}, {'name': 'Andreas Meyer-Lindenberg Emanuel Schwarz  Oliver Grimm', 'org': 'Central Institute of Mental Health, Medical Faculty Mannheim, University Heidelberg, Mannheim, Germany.'}, {'name': 'Manuel Mattheisen', 'org': 'Department of Biomedicine, Aarhus University, Aarhus, Denmark.'}, {'name': 'Esben Agerbo Ditte Demontis Thomas Hansen Manuel Mattheisen Ole Mors Line Olsen Henrik B Rasmussen Anders D Børglum Preben B Mortensen  Thomas Werge', 'org': 'The Lundbeck Foundation Initiative for Integrative Psychiatric Research, iPSYCH, Aarhus and Copenhagen, Denmark.'}, {'name': 'Esben Agerbo Ditte Demontis Thomas Hansen Manuel Mattheisen Ole Mors Line Olsen Henrik B Rasmussen Anders D Børglum Preben B Mortensen  Thomas Werge', 'org': 'The Lundbeck Foundation Initiative for Integrative Psychiatric Research, iPSYCH, Aarhus and Copenhagen, Denmark.'}, {'name': 'Esben Agerbo Ditte Demontis Thomas Hansen Manuel Mattheisen Ole Mors Line Olsen Henrik B Rasmussen Anders D Børglum Preben B Mortensen  Thomas Werge', 'org': 'The Lundbeck Foundation Initiative for Integrative Psychiatric Research, iPSYCH, Aarhus and Copenhagen, Denmark.'}, {'name': 'Esben Agerbo Ditte Demontis Thomas Hansen Manuel Mattheisen Ole Mors Line Olsen Henrik B Rasmussen Anders D Børglum Preben B Mortensen  Thomas Werge', 'org': 'The Lundbeck Foundation Initiative for Integrative Psychiatric Research, iPSYCH, Aarhus and Copenhagen, Denmark.'}, {'name': 'Esben Agerbo Ditte Demontis Thomas Hansen Manuel Mattheisen Ole Mors Line Olsen Henrik B Rasmussen Anders D Børglum Preben B Mortensen  Thomas Werge', 'org': 'The Lundbeck Foundation Initiative for Integrative Psychiatric Research, iPSYCH, Aarhus and Copenhagen, Denmark.'}, {'name': 'Esben Agerbo Ditte Demontis Thomas Hansen Manuel Mattheisen Ole Mors Line Olsen Henrik B Rasmussen Anders D Børglum Preben B Mortensen  Thomas Werge', 'org': 'The Lundbeck Foundation Initiative for Integrative Psychiatric Research, iPSYCH, Aarhus and Copenhagen, Denmark.'}, {'name': 'Esben Agerbo Ditte Demontis Thomas Hansen Manuel Mattheisen Ole Mors Line Olsen Henrik B Rasmussen Anders D Børglum Preben B Mortensen  Thomas Werge', 'org': 'The Lundbeck Foundation Initiative for Integrative Psychiatric Research, iPSYCH, Aarhus and Copenhagen, Denmark.'}, {'name': 'Esben Agerbo Ditte Demontis Thomas Hansen Manuel Mattheisen Ole Mors Line Olsen Henrik B Rasmussen Anders D Børglum Preben B Mortensen  Thomas Werge', 'org': 'The Lundbeck Foundation Initiative for Integrative Psychiatric Research, iPSYCH, Aarhus and Copenhagen, Denmark.'}, {'name': 'Esben Agerbo Ditte Demontis Thomas Hansen Manuel Mattheisen Ole Mors Line Olsen Henrik B Rasmussen Anders D Børglum Preben B Mortensen  Thomas Werge', 'org': 'The Lundbeck Foundation Initiative for Integrative Psychiatric Research, iPSYCH, Aarhus and Copenhagen, Denmark.'}, {'name': 'Esben Agerbo Ditte Demontis Thomas Hansen Manuel Mattheisen Ole Mors Line Olsen Henrik B Rasmussen Anders D Børglum Preben B Mortensen  Thomas Werge', 'org': 'The Lundbeck Foundation Initiative for Integrative Psychiatric Research, iPSYCH, Aarhus and Copenhagen, Denmark.'}, {'name': 'Manuel Mattheisen', 'org': 'Center for integrated Sequencing, iSEQ, Aarhus University, Aarhus, Denmark.'}, {'name': 'Ole A Andreassen Andrew A Brown Lavinia Athanasiu Cecilie B Hartberg  Unn Haukvik', 'org': 'NORMENT - KG Jebsen Centre, Institute of Clinical Medicine, University of Oslo, Oslo, Norway.'}, {'name': 'Ole A Andreassen Andrew A Brown Lavinia Athanasiu Cecilie B Hartberg  Unn Haukvik', 'org': 'NORMENT - KG Jebsen Centre, Institute of Clinical Medicine, University of Oslo, Oslo, Norway.'}, {'name': 'Ole A Andreassen Andrew A Brown Lavinia Athanasiu Cecilie B Hartberg  Unn Haukvik', 'org': 'NORMENT - KG Jebsen Centre, Institute of Clinical Medicine, University of Oslo, Oslo, Norway.'}, {'name': 'Ole A Andreassen Andrew A Brown Lavinia Athanasiu Cecilie B Hartberg  Unn Haukvik', 'org': 'NORMENT - KG Jebsen Centre, Institute of Clinical Medicine, University of Oslo, Oslo, Norway.'}, {'name': 'Ole A Andreassen Andrew A Brown Lavinia Athanasiu Cecilie B Hartberg  Unn Haukvik', 'org': 'NORMENT - KG Jebsen Centre, Institute of Clinical Medicine, University of Oslo, Oslo, Norway.'}, {'name': 'Ole A Andreassen  Ingrid Melle', 'org': 'Division of Mental Health and Addiction, Oslo University Hospital, Oslo, Norway.'}, {'name': 'Ole A Andreassen  Ingrid Melle', 'org': 'Division of Mental Health and Addiction, Oslo University Hospital, Oslo, Norway.'}, {'name': 'Oliver Gruber Bernd Kraemer  Maria Keil', 'org': 'Center for Translational Research in Systems Neuroscience and Psychiatry, Department of Psychiatry and Psychotherapy, University Medical Center, Goettingen, Germany.'}, {'name': 'Oliver Gruber Bernd Kraemer  Maria Keil', 'org': 'Center for Translational Research in Systems Neuroscience and Psychiatry, Department of Psychiatry and Psychotherapy, University Medical Center, Goettingen, Germany.'}, {'name': 'Oliver Gruber Bernd Kraemer  Maria Keil', 'org': 'Center for Translational Research in Systems Neuroscience and Psychiatry, Department of Psychiatry and Psychotherapy, University Medical Center, Goettingen, Germany.'}, {'name': 'Perminder S Sachdev Karen A Mather Anbupalam Thalamuthu Nicola J Armstrong Amelia A Assareh Henry Brodaty Simone Reppermund  Wei Wen', 'org': 'Centre for Healthy Brain Ageing, School of Psychiatry, University of New South Wales (UNSW), Sydney, New South Wales, Australia.'}, {'name': 'Perminder S Sachdev Karen A Mather Anbupalam Thalamuthu Nicola J Armstrong Amelia A Assareh Henry Brodaty Simone Reppermund  Wei Wen', 'org': 'Centre for Healthy Brain Ageing, School of Psychiatry, University of New South Wales (UNSW), Sydney, New South Wales, Australia.'}, {'name': 'Perminder S Sachdev Karen A Mather Anbupalam Thalamuthu Nicola J Armstrong Amelia A Assareh Henry Brodaty Simone Reppermund  Wei Wen', 'org': 'Centre for Healthy Brain Ageing, School of Psychiatry, University of New South Wales (UNSW), Sydney, New South Wales, Australia.'}, {'name': 'Perminder S Sachdev Karen A Mather Anbupalam Thalamuthu Nicola J Armstrong Amelia A Assareh Henry Brodaty Simone Reppermund  Wei Wen', 'org': 'Centre for Healthy Brain Ageing, School of Psychiatry, University of New South Wales (UNSW), Sydney, New South Wales, Australia.'}, {'name': 'Perminder S Sachdev Karen A Mather Anbupalam Thalamuthu Nicola J Armstrong Amelia A Assareh Henry Brodaty Simone Reppermund  Wei Wen', 'org': 'Centre for Healthy Brain Ageing, School of Psychiatry, University of New South Wales (UNSW), Sydney, New South Wales, Australia.'}, {'name': 'Perminder S Sachdev Karen A Mather Anbupalam Thalamuthu Nicola J Armstrong Amelia A Assareh Henry Brodaty Simone Reppermund  Wei Wen', 'org': 'Centre for Healthy Brain Ageing, School of Psychiatry, University of New South Wales (UNSW), Sydney, New South Wales, Australia.'}, {'name': 'Perminder S Sachdev Karen A Mather Anbupalam Thalamuthu Nicola J Armstrong Amelia A Assareh Henry Brodaty Simone Reppermund  Wei Wen', 'org': 'Centre for Healthy Brain Ageing, School of Psychiatry, University of New South Wales (UNSW), Sydney, New South Wales, Australia.'}, {'name': 'Perminder S Sachdev Karen A Mather Anbupalam Thalamuthu Nicola J Armstrong Amelia A Assareh Henry Brodaty Simone Reppermund  Wei Wen', 'org': 'Centre for Healthy Brain Ageing, School of Psychiatry, University of New South Wales (UNSW), Sydney, New South Wales, Australia.'}, {'name': 'Perminder S Sachdev', 'org': 'Neuropsychiatric Institute, Prince of Wales Hospital, Sydney, New South Wales, Australia.'}, {'name': 'Roberto Roiz-Santiañez', 'org': 'Department of Psychiatry, University Hospital Marqués de Valdecilla, School of Medicine, University of Cantabria-IDIVAL, Santander, Spain.'}, {'name': 'Roberto Roiz-Santiañez  Rocio Perez-Iglesias', 'org': 'Cibersam (Centro Investigación Biomédica en Red Salud Mental), Madrid, Spain.'}, {'name': 'Roberto Roiz-Santiañez  Rocio Perez-Iglesias', 'org': 'Cibersam (Centro Investigación Biomédica en Red Salud Mental), Madrid, Spain.'}, {'name': 'Andrew J Saykin Sungeun Kim Kwangsik Nho Shannon L Risacher  Li Shen', 'org': 'Center for Neuroimaging, Radiology and Imaging Sciences, Indiana University School of Medicine, Indianapolis, Indiana, USA.'}, {'name': 'Andrew J Saykin Sungeun Kim Kwangsik Nho Shannon L Risacher  Li Shen', 'org': 'Center for Neuroimaging, Radiology and Imaging Sciences, Indiana University School of Medicine, Indianapolis, Indiana, USA.'}, {'name': 'Andrew J Saykin Sungeun Kim Kwangsik Nho Shannon L Risacher  Li Shen', 'org': 'Center for Neuroimaging, Radiology and Imaging Sciences, Indiana University School of Medicine, Indianapolis, Indiana, USA.'}, {'name': 'Andrew J Saykin Sungeun Kim Kwangsik Nho Shannon L Risacher  Li Shen', 'org': 'Center for Neuroimaging, Radiology and Imaging Sciences, Indiana University School of Medicine, Indianapolis, Indiana, USA.'}, {'name': 'Andrew J Saykin Sungeun Kim Kwangsik Nho Shannon L Risacher  Li Shen', 'org': 'Center for Neuroimaging, Radiology and Imaging Sciences, Indiana University School of Medicine, Indianapolis, Indiana, USA.'}, {'name': 'Andrew J Saykin Sungeun Kim Kwangsik Nho Shannon L Risacher Li Shen  Tatiana M Foroud', 'org': 'Indiana Alzheimer Disease Center, Indiana University School of Medicine, Indianapolis, Indiana, USA.'}, {'name': 'Andrew J Saykin Sungeun Kim Kwangsik Nho Shannon L Risacher Li Shen  Tatiana M Foroud', 'org': 'Indiana Alzheimer Disease Center, Indiana University School of Medicine, Indianapolis, Indiana, USA.'}, {'name': 'Andrew J Saykin Sungeun Kim Kwangsik Nho Shannon L Risacher Li Shen  Tatiana M Foroud', 'org': 'Indiana Alzheimer Disease Center, Indiana University School of Medicine, Indianapolis, Indiana, USA.'}, {'name': 'Andrew J Saykin Sungeun Kim Kwangsik Nho Shannon L Risacher Li Shen  Tatiana M Foroud', 'org': 'Indiana Alzheimer Disease Center, Indiana University School of Medicine, Indianapolis, Indiana, USA.'}, {'name': 'Andrew J Saykin Sungeun Kim Kwangsik Nho Shannon L Risacher Li Shen  Tatiana M Foroud', 'org': 'Indiana Alzheimer Disease Center, Indiana University School of Medicine, Indianapolis, Indiana, USA.'}, {'name': 'Andrew J Saykin Sungeun Kim Kwangsik Nho Shannon L Risacher Li Shen  Tatiana M Foroud', 'org': 'Indiana Alzheimer Disease Center, Indiana University School of Medicine, Indianapolis, Indiana, USA.'}, {'name': 'Andrew J Saykin  Tatiana M Foroud', 'org': 'Medical and Molecular Genetics, Indiana University School of Medicine, Indianapolis, Indiana, USA.'}, {'name': 'Andrew J Saykin  Tatiana M Foroud', 'org': 'Medical and Molecular Genetics, Indiana University School of Medicine, Indianapolis, Indiana, USA.'}, {'name': 'Stefan Ehrlich Johanna Hass  Esther Walton', 'org': 'Department of Child and Adolescent Psychiatry, Faculty of Medicine and University Hospital, Technische Universität Dresden, Dresden, Germany.'}, {'name': 'Stefan Ehrlich Johanna Hass  Esther Walton', 'org': 'Department of Child and Adolescent Psychiatry, Faculty of Medicine and University Hospital, Technische Universität Dresden, Dresden, Germany.'}, {'name': 'Stefan Ehrlich Johanna Hass  Esther Walton', 'org': 'Department of Child and Adolescent Psychiatry, Faculty of Medicine and University Hospital, Technische Universität Dresden, Dresden, Germany.'}, {'name': 'Jessica A Turner', 'org': 'Georgia State University, Atlanta, Georgia, USA.'}, {'name': 'Jessica A Turner', 'org': 'Mind Research Network, Albuquerque, New Mexico, USA.'}, {'name': 'Yvonne Y W Ho Nicholas G Martin Margaret J Wright Miguel E Renteria Gabriel Cuellar-Partida Lachlan T Strike Narelle K Hansell Grant W Montgomery  Sarah E Medland', 'org': 'QIMR Berghofer Medical Research Institute, Brisbane, Queensland, Australia.'}, {'name': 'Yvonne Y W Ho Nicholas G Martin Margaret J Wright Miguel E Renteria Gabriel Cuellar-Partida Lachlan T Strike Narelle K Hansell Grant W Montgomery  Sarah E Medland', 'org': 'QIMR Berghofer Medical Research Institute, Brisbane, Queensland, Australia.'}, {'name': 'Yvonne Y W Ho Nicholas G Martin Margaret J Wright Miguel E Renteria Gabriel Cuellar-Partida Lachlan T Strike Narelle K Hansell Grant W Montgomery  Sarah E Medland', 'org': 'QIMR Berghofer Medical Research Institute, Brisbane, Queensland, Australia.'}, {'name': 'Yvonne Y W Ho Nicholas G Martin Margaret J Wright Miguel E Renteria Gabriel Cuellar-Partida Lachlan T Strike Narelle K Hansell Grant W Montgomery  Sarah E Medland', 'org': 'QIMR Berghofer Medical Research Institute, Brisbane, Queensland, Australia.'}, {'name': 'Yvonne Y W Ho Nicholas G Martin Margaret J Wright Miguel E Renteria Gabriel Cuellar-Partida Lachlan T Strike Narelle K Hansell Grant W Montgomery  Sarah E Medland', 'org': 'QIMR Berghofer Medical Research Institute, Brisbane, Queensland, Australia.'}, {'name': 'Yvonne Y W Ho Nicholas G Martin Margaret J Wright Miguel E Renteria Gabriel Cuellar-Partida Lachlan T Strike Narelle K Hansell Grant W Montgomery  Sarah E Medland', 'org': 'QIMR Berghofer Medical Research Institute, Brisbane, Queensland, Australia.'}, {'name': 'Yvonne Y W Ho Nicholas G Martin Margaret J Wright Miguel E Renteria Gabriel Cuellar-Partida Lachlan T Strike Narelle K Hansell Grant W Montgomery  Sarah E Medland', 'org': 'QIMR Berghofer Medical Research Institute, Brisbane, Queensland, Australia.'}, {'name': 'Yvonne Y W Ho Nicholas G Martin Margaret J Wright Miguel E Renteria Gabriel Cuellar-Partida Lachlan T Strike Narelle K Hansell Grant W Montgomery  Sarah E Medland', 'org': 'QIMR Berghofer Medical Research Institute, Brisbane, Queensland, Australia.'}, {'name': 'Yvonne Y W Ho Nicholas G Martin Margaret J Wright Miguel E Renteria Gabriel Cuellar-Partida Lachlan T Strike Narelle K Hansell Grant W Montgomery  Sarah E Medland', 'org': 'QIMR Berghofer Medical Research Institute, Brisbane, Queensland, Australia.'}, {'name': 'Margaret J Wright Lachlan T Strike  Greig I de Zubicaray', 'org': 'School of Psychology, University of Queensland, Brisbane, Queensland, Australia.'}, {'name': 'Margaret J Wright Lachlan T Strike  Greig I de Zubicaray', 'org': 'School of Psychology, University of Queensland, Brisbane, Queensland, Australia.'}, {'name': 'Margaret J Wright Lachlan T Strike  Greig I de Zubicaray', 'org': 'School of Psychology, University of Queensland, Brisbane, Queensland, Australia.'}, {'name': "James T R Walters Peter A Holmans Noa Carrera Nick Craddock Valentina Escott-Price Lyudmila Georgieva Marian L Hamshere David Kavanagh Sophie E Legge Andrew J Pocklington Alexander L Richards Douglas M Ruderfer Nigel M Williams George Kirov Michael J Owen  Michael C O'Donovan", 'org': 'MRC Centre for Neuropsychiatric Genetics and Genomics, Institute of Psychological Medicine and Clinical Neurosciences, School of Medicine, Cardiff University, Cardiff, UK.'}, {'name': "James T R Walters Peter A Holmans Noa Carrera Nick Craddock Valentina Escott-Price Lyudmila Georgieva Marian L Hamshere David Kavanagh Sophie E Legge Andrew J Pocklington Alexander L Richards Douglas M Ruderfer Nigel M Williams George Kirov Michael J Owen  Michael C O'Donovan", 'org': 'MRC Centre for Neuropsychiatric Genetics and Genomics, Institute of Psychological Medicine and Clinical Neurosciences, School of Medicine, Cardiff University, Cardiff, UK.'}, {'name': "James T R Walters Peter A Holmans Noa Carrera Nick Craddock Valentina Escott-Price Lyudmila Georgieva Marian L Hamshere David Kavanagh Sophie E Legge Andrew J Pocklington Alexander L Richards Douglas M Ruderfer Nigel M Williams George Kirov Michael J Owen  Michael C O'Donovan", 'org': 'MRC Centre for Neuropsychiatric Genetics and Genomics, Institute of Psychological Medicine and Clinical Neurosciences, School of Medicine, Cardiff University, Cardiff, UK.'}, {'name': "James T R Walters Peter A Holmans Noa Carrera Nick Craddock Valentina Escott-Price Lyudmila Georgieva Marian L Hamshere David Kavanagh Sophie E Legge Andrew J Pocklington Alexander L Richards Douglas M Ruderfer Nigel M Williams George Kirov Michael J Owen  Michael C O'Donovan", 'org': 'MRC Centre for Neuropsychiatric Genetics and Genomics, Institute of Psychological Medicine and Clinical Neurosciences, School of Medicine, Cardiff University, Cardiff, UK.'}, {'name': "James T R Walters Peter A Holmans Noa Carrera Nick Craddock Valentina Escott-Price Lyudmila Georgieva Marian L Hamshere David Kavanagh Sophie E Legge Andrew J Pocklington Alexander L Richards Douglas M Ruderfer Nigel M Williams George Kirov Michael J Owen  Michael C O'Donovan", 'org': 'MRC Centre for Neuropsychiatric Genetics and Genomics, Institute of Psychological Medicine and Clinical Neurosciences, School of Medicine, Cardiff University, Cardiff, UK.'}, {'name': "James T R Walters Peter A Holmans Noa Carrera Nick Craddock Valentina Escott-Price Lyudmila Georgieva Marian L Hamshere David Kavanagh Sophie E Legge Andrew J Pocklington Alexander L Richards Douglas M Ruderfer Nigel M Williams George Kirov Michael J Owen  Michael C O'Donovan", 'org': 'MRC Centre for Neuropsychiatric Genetics and Genomics, Institute of Psychological Medicine and Clinical Neurosciences, School of Medicine, Cardiff University, Cardiff, UK.'}, {'name': "James T R Walters Peter A Holmans Noa Carrera Nick Craddock Valentina Escott-Price Lyudmila Georgieva Marian L Hamshere David Kavanagh Sophie E Legge Andrew J Pocklington Alexander L Richards Douglas M Ruderfer Nigel M Williams George Kirov Michael J Owen  Michael C O'Donovan", 'org': 'MRC Centre for Neuropsychiatric Genetics and Genomics, Institute of Psychological Medicine and Clinical Neurosciences, School of Medicine, Cardiff University, Cardiff, UK.'}, {'name': "James T R Walters Peter A Holmans Noa Carrera Nick Craddock Valentina Escott-Price Lyudmila Georgieva Marian L Hamshere David Kavanagh Sophie E Legge Andrew J Pocklington Alexander L Richards Douglas M Ruderfer Nigel M Williams George Kirov Michael J Owen  Michael C O'Donovan", 'org': 'MRC Centre for Neuropsychiatric Genetics and Genomics, Institute of Psychological Medicine and Clinical Neurosciences, School of Medicine, Cardiff University, Cardiff, UK.'}, {'name': "James T R Walters Peter A Holmans Noa Carrera Nick Craddock Valentina Escott-Price Lyudmila Georgieva Marian L Hamshere David Kavanagh Sophie E Legge Andrew J Pocklington Alexander L Richards Douglas M Ruderfer Nigel M Williams George Kirov Michael J Owen  Michael C O'Donovan", 'org': 'MRC Centre for Neuropsychiatric Genetics and Genomics, Institute of Psychological Medicine and Clinical Neurosciences, School of Medicine, Cardiff University, Cardiff, UK.'}, {'name': "James T R Walters Peter A Holmans Noa Carrera Nick Craddock Valentina Escott-Price Lyudmila Georgieva Marian L Hamshere David Kavanagh Sophie E Legge Andrew J Pocklington Alexander L Richards Douglas M Ruderfer Nigel M Williams George Kirov Michael J Owen  Michael C O'Donovan", 'org': 'MRC Centre for Neuropsychiatric Genetics and Genomics, Institute of Psychological Medicine and Clinical Neurosciences, School of Medicine, Cardiff University, Cardiff, UK.'}, {'name': "James T R Walters Peter A Holmans Noa Carrera Nick Craddock Valentina Escott-Price Lyudmila Georgieva Marian L Hamshere David Kavanagh Sophie E Legge Andrew J Pocklington Alexander L Richards Douglas M Ruderfer Nigel M Williams George Kirov Michael J Owen  Michael C O'Donovan", 'org': 'MRC Centre for Neuropsychiatric Genetics and Genomics, Institute of Psychological Medicine and Clinical Neurosciences, School of Medicine, Cardiff University, Cardiff, UK.'}, {'name': "James T R Walters Peter A Holmans Noa Carrera Nick Craddock Valentina Escott-Price Lyudmila Georgieva Marian L Hamshere David Kavanagh Sophie E Legge Andrew J Pocklington Alexander L Richards Douglas M Ruderfer Nigel M Williams George Kirov Michael J Owen  Michael C O'Donovan", 'org': 'MRC Centre for Neuropsychiatric Genetics and Genomics, Institute of Psychological Medicine and Clinical Neurosciences, School of Medicine, Cardiff University, Cardiff, UK.'}, {'name': "James T R Walters Peter A Holmans Noa Carrera Nick Craddock Valentina Escott-Price Lyudmila Georgieva Marian L Hamshere David Kavanagh Sophie E Legge Andrew J Pocklington Alexander L Richards Douglas M Ruderfer Nigel M Williams George Kirov Michael J Owen  Michael C O'Donovan", 'org': 'MRC Centre for Neuropsychiatric Genetics and Genomics, Institute of Psychological Medicine and Clinical Neurosciences, School of Medicine, Cardiff University, Cardiff, UK.'}, {'name': "James T R Walters Peter A Holmans Noa Carrera Nick Craddock Valentina Escott-Price Lyudmila Georgieva Marian L Hamshere David Kavanagh Sophie E Legge Andrew J Pocklington Alexander L Richards Douglas M Ruderfer Nigel M Williams George Kirov Michael J Owen  Michael C O'Donovan", 'org': 'MRC Centre for Neuropsychiatric Genetics and Genomics, Institute of Psychological Medicine and Clinical Neurosciences, School of Medicine, Cardiff University, Cardiff, UK.'}, {'name': "James T R Walters Peter A Holmans Noa Carrera Nick Craddock Valentina Escott-Price Lyudmila Georgieva Marian L Hamshere David Kavanagh Sophie E Legge Andrew J Pocklington Alexander L Richards Douglas M Ruderfer Nigel M Williams George Kirov Michael J Owen  Michael C O'Donovan", 'org': 'MRC Centre for Neuropsychiatric Genetics and Genomics, Institute of Psychological Medicine and Clinical Neurosciences, School of Medicine, Cardiff University, Cardiff, UK.'}, {'name': "James T R Walters Peter A Holmans Noa Carrera Nick Craddock Valentina Escott-Price Lyudmila Georgieva Marian L Hamshere David Kavanagh Sophie E Legge Andrew J Pocklington Alexander L Richards Douglas M Ruderfer Nigel M Williams George Kirov Michael J Owen  Michael C O'Donovan", 'org': 'MRC Centre for Neuropsychiatric Genetics and Genomics, Institute of Psychological Medicine and Clinical Neurosciences, School of Medicine, Cardiff University, Cardiff, UK.'}, {'name': "Peter A Holmans Nick Craddock Alexander L Richards Michael J Owen  Michael C O'Donovan", 'org': 'National Centre for Mental Health, Cardiff University, Cardiff, UK.'}, {'name': "Peter A Holmans Nick Craddock Alexander L Richards Michael J Owen  Michael C O'Donovan", 'org': 'National Centre for Mental Health, Cardiff University, Cardiff, UK.'}, {'name': "Peter A Holmans Nick Craddock Alexander L Richards Michael J Owen  Michael C O'Donovan", 'org': 'National Centre for Mental Health, Cardiff University, Cardiff, UK.'}, {'name': "Peter A Holmans Nick Craddock Alexander L Richards Michael J Owen  Michael C O'Donovan", 'org': 'National Centre for Mental Health, Cardiff University, Cardiff, UK.'}, {'name': "Peter A Holmans Nick Craddock Alexander L Richards Michael J Owen  Michael C O'Donovan", 'org': 'National Centre for Mental Health, Cardiff University, Cardiff, UK.'}, {'name': 'Benjamin M Neale Hailiang Huang Tune H Pers Jacqueline I Goldstein Joel N Hirschhorn Alkes Price Eli A Stahl Tõnu Esko Mark J Daly  Raymond K Walters', 'org': 'Medical and Population Genetics Program, Broad Institute of MIT and Harvard, Cambridge, Massachusetts, USA.'}, {'name': 'Benjamin M Neale Hailiang Huang Tune H Pers Jacqueline I Goldstein Joel N Hirschhorn Alkes Price Eli A Stahl Tõnu Esko Mark J Daly  Raymond K Walters', 'org': 'Medical and Population Genetics Program, Broad Institute of MIT and Harvard, Cambridge, Massachusetts, USA.'}, {'name': 'Benjamin M Neale Hailiang Huang Tune H Pers Jacqueline I Goldstein Joel N Hirschhorn Alkes Price Eli A Stahl Tõnu Esko Mark J Daly  Raymond K Walters', 'org': 'Medical and Population Genetics Program, Broad Institute of MIT and Harvard, Cambridge, Massachusetts, USA.'}, {'name': 'Benjamin M Neale Hailiang Huang Tune H Pers Jacqueline I Goldstein Joel N Hirschhorn Alkes Price Eli A Stahl Tõnu Esko Mark J Daly  Raymond K Walters', 'org': 'Medical and Population Genetics Program, Broad Institute of MIT and Harvard, Cambridge, Massachusetts, USA.'}, {'name': 'Benjamin M Neale Hailiang Huang Tune H Pers Jacqueline I Goldstein Joel N Hirschhorn Alkes Price Eli A Stahl Tõnu Esko Mark J Daly  Raymond K Walters', 'org': 'Medical and Population Genetics Program, Broad Institute of MIT and Harvard, Cambridge, Massachusetts, USA.'}, {'name': 'Benjamin M Neale Hailiang Huang Tune H Pers Jacqueline I Goldstein Joel N Hirschhorn Alkes Price Eli A Stahl Tõnu Esko Mark J Daly  Raymond K Walters', 'org': 'Medical and Population Genetics Program, Broad Institute of MIT and Harvard, Cambridge, Massachusetts, USA.'}, {'name': 'Benjamin M Neale Hailiang Huang Tune H Pers Jacqueline I Goldstein Joel N Hirschhorn Alkes Price Eli A Stahl Tõnu Esko Mark J Daly  Raymond K Walters', 'org': 'Medical and Population Genetics Program, Broad Institute of MIT and Harvard, Cambridge, Massachusetts, USA.'}, {'name': 'Benjamin M Neale Hailiang Huang Tune H Pers Jacqueline I Goldstein Joel N Hirschhorn Alkes Price Eli A Stahl Tõnu Esko Mark J Daly  Raymond K Walters', 'org': 'Medical and Population Genetics Program, Broad Institute of MIT and Harvard, Cambridge, Massachusetts, USA.'}, {'name': 'Benjamin M Neale Hailiang Huang Tune H Pers Jacqueline I Goldstein Joel N Hirschhorn Alkes Price Eli A Stahl Tõnu Esko Mark J Daly  Raymond K Walters', 'org': 'Medical and Population Genetics Program, Broad Institute of MIT and Harvard, Cambridge, Massachusetts, USA.'}, {'name': 'Benjamin M Neale Hailiang Huang Tune H Pers Jacqueline I Goldstein Joel N Hirschhorn Alkes Price Eli A Stahl Tõnu Esko Mark J Daly  Raymond K Walters', 'org': 'Medical and Population Genetics Program, Broad Institute of MIT and Harvard, Cambridge, Massachusetts, USA.'}, {'name': 'Sarah E Bergen Anna K Kähler Patrik K E Magnusson Christina M Hultman  Patrick F Sullivan', 'org': 'Department of Medical Epidemiology and Biostatistics, Karolinska Institutet, Stockholm, Sweden.'}, {'name': 'Sarah E Bergen Anna K Kähler Patrik K E Magnusson Christina M Hultman  Patrick F Sullivan', 'org': 'Department of Medical Epidemiology and Biostatistics, Karolinska Institutet, Stockholm, Sweden.'}, {'name': 'Sarah E Bergen Anna K Kähler Patrik K E Magnusson Christina M Hultman  Patrick F Sullivan', 'org': 'Department of Medical Epidemiology and Biostatistics, Karolinska Institutet, Stockholm, Sweden.'}, {'name': 'Sarah E Bergen Anna K Kähler Patrik K E Magnusson Christina M Hultman  Patrick F Sullivan', 'org': 'Department of Medical Epidemiology and Biostatistics, Karolinska Institutet, Stockholm, Sweden.'}, {'name': 'Sarah E Bergen Anna K Kähler Patrik K E Magnusson Christina M Hultman  Patrick F Sullivan', 'org': 'Department of Medical Epidemiology and Biostatistics, Karolinska Institutet, Stockholm, Sweden.'}, {'name': 'James J Crowley Martilias S Farrell Paola Giusti-Rodríguez Yunjung Kim Jin P Szatkiewicz Stephanie Williams  Patrick F Sullivan', 'org': 'Department of Genetics, University of North Carolina, Chapel Hill, North Carolina, USA.'}, {'name': 'James J Crowley Martilias S Farrell Paola Giusti-Rodríguez Yunjung Kim Jin P Szatkiewicz Stephanie Williams  Patrick F Sullivan', 'org': 'Department of Genetics, University of North Carolina, Chapel Hill, North Carolina, USA.'}, {'name': 'James J Crowley Martilias S Farrell Paola Giusti-Rodríguez Yunjung Kim Jin P Szatkiewicz Stephanie Williams  Patrick F Sullivan', 'org': 'Department of Genetics, University of North Carolina, Chapel Hill, North Carolina, USA.'}, {'name': 'James J Crowley Martilias S Farrell Paola Giusti-Rodríguez Yunjung Kim Jin P Szatkiewicz Stephanie Williams  Patrick F Sullivan', 'org': 'Department of Genetics, University of North Carolina, Chapel Hill, North Carolina, USA.'}, {'name': 'James J Crowley Martilias S Farrell Paola Giusti-Rodríguez Yunjung Kim Jin P Szatkiewicz Stephanie Williams  Patrick F Sullivan', 'org': 'Department of Genetics, University of North Carolina, Chapel Hill, North Carolina, USA.'}, {'name': 'James J Crowley Martilias S Farrell Paola Giusti-Rodríguez Yunjung Kim Jin P Szatkiewicz Stephanie Williams  Patrick F Sullivan', 'org': 'Department of Genetics, University of North Carolina, Chapel Hill, North Carolina, USA.'}, {'name': 'James J Crowley Martilias S Farrell Paola Giusti-Rodríguez Yunjung Kim Jin P Szatkiewicz Stephanie Williams  Patrick F Sullivan', 'org': 'Department of Genetics, University of North Carolina, Chapel Hill, North Carolina, USA.'}, {'name': 'Diana O Perkins  Patrick F Sullivan', 'org': 'Department of Psychiatry, University of North Carolina, Chapel Hill, North Carolina, USA.'}, {'name': 'Diana O Perkins  Patrick F Sullivan', 'org': 'Department of Psychiatry, University of North Carolina, Chapel Hill, North Carolina, USA.'}, {'name': 'Aiden Corvin Paul Cormican Gary Donohoe Derek W Morris Michael Gill Emma J Rose  Margaret Needham', 'org': 'Neuropsychiatric Genetics Research Group, Department of Psychiatry, Trinity College Dublin, Dublin, Ireland.'}, {'name': 'Aiden Corvin Paul Cormican Gary Donohoe Derek W Morris Michael Gill Emma J Rose  Margaret Needham', 'org': 'Neuropsychiatric Genetics Research Group, Department of Psychiatry, Trinity College Dublin, Dublin, Ireland.'}, {'name': 'Aiden Corvin Paul Cormican Gary Donohoe Derek W Morris Michael Gill Emma J Rose  Margaret Needham', 'org': 'Neuropsychiatric Genetics Research Group, Department of Psychiatry, Trinity College Dublin, Dublin, Ireland.'}, {'name': 'Aiden Corvin Paul Cormican Gary Donohoe Derek W Morris Michael Gill Emma J Rose  Margaret Needham', 'org': 'Neuropsychiatric Genetics Research Group, Department of Psychiatry, Trinity College Dublin, Dublin, Ireland.'}, {'name': 'Aiden Corvin Paul Cormican Gary Donohoe Derek W Morris Michael Gill Emma J Rose  Margaret Needham', 'org': 'Neuropsychiatric Genetics Research Group, Department of Psychiatry, Trinity College Dublin, Dublin, Ireland.'}, {'name': 'Aiden Corvin Paul Cormican Gary Donohoe Derek W Morris Michael Gill Emma J Rose  Margaret Needham', 'org': 'Neuropsychiatric Genetics Research Group, Department of Psychiatry, Trinity College Dublin, Dublin, Ireland.'}, {'name': 'Aiden Corvin Paul Cormican Gary Donohoe Derek W Morris Michael Gill Emma J Rose  Margaret Needham', 'org': 'Neuropsychiatric Genetics Research Group, Department of Psychiatry, Trinity College Dublin, Dublin, Ireland.'}, {'name': 'David A Collier', 'org': 'Eli Lilly and Company Limited, Erl Wood Manor, Sunninghill Road, Windlesham, Surrey, UK.'}, {'name': 'David A Collier', 'org': "Social, Genetic and Developmental Psychiatry Centre, Institute of Psychiatry, King's College London, London, UK."}, {'name': 'Ingrid Agartz Srdjan Djurovic Morten Mattingsdal  Erik G Jönsson', 'org': 'NORMENT, KG Jebsen Centre for Psychosis Research, Institute of Clinical Medicine, University of Oslo, Oslo, Norway.'}, {'name': 'Ingrid Agartz Srdjan Djurovic Morten Mattingsdal  Erik G Jönsson', 'org': 'NORMENT, KG Jebsen Centre for Psychosis Research, Institute of Clinical Medicine, University of Oslo, Oslo, Norway.'}, {'name': 'Ingrid Agartz Srdjan Djurovic Morten Mattingsdal  Erik G Jönsson', 'org': 'NORMENT, KG Jebsen Centre for Psychosis Research, Institute of Clinical Medicine, University of Oslo, Oslo, Norway.'}, {'name': 'Ingrid Agartz Srdjan Djurovic Morten Mattingsdal  Erik G Jönsson', 'org': 'NORMENT, KG Jebsen Centre for Psychosis Research, Institute of Clinical Medicine, University of Oslo, Oslo, Norway.'}, {'name': 'Ingrid Agartz', 'org': 'Department of Psychiatry, Diakonhjemmet Hospital, Oslo, Norway.'}, {'name': 'Ingrid Agartz Erik Söderman  Erik G Jönsson', 'org': 'Department of Clinical Neuroscience, Psychiatry Section, Karolinska Institutet, Stockholm, Sweden.'}, {'name': 'Ingrid Agartz Erik Söderman  Erik G Jönsson', 'org': 'Department of Clinical Neuroscience, Psychiatry Section, Karolinska Institutet, Stockholm, Sweden.'}, {'name': 'Ingrid Agartz Erik Söderman  Erik G Jönsson', 'org': 'Department of Clinical Neuroscience, Psychiatry Section, Karolinska Institutet, Stockholm, Sweden.'}, {'name': 'Margot Albus', 'org': 'State Mental Hospital, Haar, Germany.'}, {'name': 'Madeline Alexander Claudine Laurent  Douglas F Levinson', 'org': 'Department of Psychiatry and Behavioral Sciences, Stanford University, Stanford, California, USA.'}, {'name': 'Madeline Alexander Claudine Laurent  Douglas F Levinson', 'org': 'Department of Psychiatry and Behavioral Sciences, Stanford University, Stanford, California, USA.'}, {'name': 'Madeline Alexander Claudine Laurent  Douglas F Levinson', 'org': 'Department of Psychiatry and Behavioral Sciences, Stanford University, Stanford, California, USA.'}, {'name': 'Farooq Amin', 'org': 'Department of Psychiatry and Behavioral Sciences, Emory University, Atlanta, Georgia, USA.'}, {'name': 'Farooq Amin', 'org': 'Department of Psychiatry and Behavioral Sciences, Atlanta Veterans Affairs Medical Center, Atlanta, Georgia, USA.'}, {'name': 'Silviu A Bacanu Tim B Bigdeli Mark A Reimers Bradley T Webb Aaron R Wolen Brandon K Wormley Kenneth S Kendler  Brien P Riley', 'org': 'Virginia Institute for Psychiatric and Behavioral Genetics, Virginia Commonwealth University, Richmond, Virginia, USA.'}, {'name': 'Silviu A Bacanu Tim B Bigdeli Mark A Reimers Bradley T Webb Aaron R Wolen Brandon K Wormley Kenneth S Kendler  Brien P Riley', 'org': 'Virginia Institute for Psychiatric and Behavioral Genetics, Virginia Commonwealth University, Richmond, Virginia, USA.'}, {'name': 'Silviu A Bacanu Tim B Bigdeli Mark A Reimers Bradley T Webb Aaron R Wolen Brandon K Wormley Kenneth S Kendler  Brien P Riley', 'org': 'Virginia Institute for Psychiatric and Behavioral Genetics, Virginia Commonwealth University, Richmond, Virginia, USA.'}, {'name': 'Silviu A Bacanu Tim B Bigdeli Mark A Reimers Bradley T Webb Aaron R Wolen Brandon K Wormley Kenneth S Kendler  Brien P Riley', 'org': 'Virginia Institute for Psychiatric and Behavioral Genetics, Virginia Commonwealth University, Richmond, Virginia, USA.'}, {'name': 'Silviu A Bacanu Tim B Bigdeli Mark A Reimers Bradley T Webb Aaron R Wolen Brandon K Wormley Kenneth S Kendler  Brien P Riley', 'org': 'Virginia Institute for Psychiatric and Behavioral Genetics, Virginia Commonwealth University, Richmond, Virginia, USA.'}, {'name': 'Silviu A Bacanu Tim B Bigdeli Mark A Reimers Bradley T Webb Aaron R Wolen Brandon K Wormley Kenneth S Kendler  Brien P Riley', 'org': 'Virginia Institute for Psychiatric and Behavioral Genetics, Virginia Commonwealth University, Richmond, Virginia, USA.'}, {'name': 'Silviu A Bacanu Tim B Bigdeli Mark A Reimers Bradley T Webb Aaron R Wolen Brandon K Wormley Kenneth S Kendler  Brien P Riley', 'org': 'Virginia Institute for Psychiatric and Behavioral Genetics, Virginia Commonwealth University, Richmond, Virginia, USA.'}, {'name': 'Silviu A Bacanu Tim B Bigdeli Mark A Reimers Bradley T Webb Aaron R Wolen Brandon K Wormley Kenneth S Kendler  Brien P Riley', 'org': 'Virginia Institute for Psychiatric and Behavioral Genetics, Virginia Commonwealth University, Richmond, Virginia, USA.'}, {'name': 'Martin Begemann Christian Hammer Sergi Papiol  Hannelore Ehrenreich', 'org': 'Clinical Neuroscience, Max Planck Institute of Experimental Medicine, Göttingen, Germany.'}, {'name': 'Martin Begemann Christian Hammer Sergi Papiol  Hannelore Ehrenreich', 'org': 'Clinical Neuroscience, Max Planck Institute of Experimental Medicine, Göttingen, Germany.'}, {'name': 'Martin Begemann Christian Hammer Sergi Papiol  Hannelore Ehrenreich', 'org': 'Clinical Neuroscience, Max Planck Institute of Experimental Medicine, Göttingen, Germany.'}, {'name': 'Martin Begemann Christian Hammer Sergi Papiol  Hannelore Ehrenreich', 'org': 'Clinical Neuroscience, Max Planck Institute of Experimental Medicine, Göttingen, Germany.'}, {'name': 'Judit Bene  Bela Melegh', 'org': 'Department of Medical Genetics, University of Pécs, Pécs, Hungary.'}, {'name': 'Judit Bene  Bela Melegh', 'org': 'Department of Medical Genetics, University of Pécs, Pécs, Hungary.'}, {'name': 'Judit Bene  Bela Melegh', 'org': 'Szentagothai Research Center, University of Pécs, Pécs, Hungary.'}, {'name': 'Judit Bene  Bela Melegh', 'org': 'Szentagothai Research Center, University of Pécs, Pécs, Hungary.'}, {'name': 'Donald W Black', 'org': 'Department of Psychiatry, University of Iowa Carver College of Medicine, Iowa City, Iowa, USA.'}, {'name': 'Richard Bruggeman', 'org': 'University Medical Center Groningen, Department of Psychiatry, University of Groningen, the Netherlands.'}, {'name': 'Nancy G Buccola', 'org': 'School of Nursing, Louisiana State University Health Sciences Center, New Orleans, Louisiana, USA.'}, {'name': 'Randy L Buckner', 'org': 'Center for Brain Science, Harvard University, Cambridge, Massachusetts, USA.'}, {'name': 'Randy L Buckner Joshua L Roffman Avram J Holmes  Randy L Gollub', 'org': 'Department of Psychiatry, Massachusetts General Hospital, Boston, Massachusetts, USA.'}, {'name': 'Randy L Buckner Joshua L Roffman Avram J Holmes  Randy L Gollub', 'org': 'Department of Psychiatry, Massachusetts General Hospital, Boston, Massachusetts, USA.'}, {'name': 'Randy L Buckner Joshua L Roffman Avram J Holmes  Randy L Gollub', 'org': 'Department of Psychiatry, Massachusetts General Hospital, Boston, Massachusetts, USA.'}, {'name': 'Randy L Buckner Joshua L Roffman Avram J Holmes  Randy L Gollub', 'org': 'Department of Psychiatry, Massachusetts General Hospital, Boston, Massachusetts, USA.'}, {'name': 'Randy L Buckner  Joshua L Roffman', 'org': 'Athinoula A Martinos Center, Massachusetts General Hospital, Boston, Massachusetts, USA.'}, {'name': 'Randy L Buckner  Joshua L Roffman', 'org': 'Athinoula A Martinos Center, Massachusetts General Hospital, Boston, Massachusetts, USA.'}, {'name': 'William F Byerley', 'org': 'Department of Psychiatry, University of California at San Francisco, San Francisco, California, USA.'}, {'name': 'Wiepke Cahn René S Kahn Eric Strengman  Roel A Ophoff', 'org': 'University Medical Center Utrecht, Department of Psychiatry, Rudolf Magnus Institute of Neuroscience, Utrecht, the Netherlands.'}, {'name': 'Wiepke Cahn René S Kahn Eric Strengman  Roel A Ophoff', 'org': 'University Medical Center Utrecht, Department of Psychiatry, Rudolf Magnus Institute of Neuroscience, Utrecht, the Netherlands.'}, {'name': 'Wiepke Cahn René S Kahn Eric Strengman  Roel A Ophoff', 'org': 'University Medical Center Utrecht, Department of Psychiatry, Rudolf Magnus Institute of Neuroscience, Utrecht, the Netherlands.'}, {'name': 'Wiepke Cahn René S Kahn Eric Strengman  Roel A Ophoff', 'org': 'University Medical Center Utrecht, Department of Psychiatry, Rudolf Magnus Institute of Neuroscience, Utrecht, the Netherlands.'}, {'name': 'Guiqing Cai Kenneth L Davis Elodie Drapeau Joseph I Friedman Vahram Haroutunian Elena Parkhomenko Abraham Reichenberg Jeremy M Silverman  Joseph D Buxbaum', 'org': 'Department of Psychiatry, Icahn School of Medicine at Mount Sinai, New York, New York, USA.'}, {'name': 'Guiqing Cai Kenneth L Davis Elodie Drapeau Joseph I Friedman Vahram Haroutunian Elena Parkhomenko Abraham Reichenberg Jeremy M Silverman  Joseph D Buxbaum', 'org': 'Department of Psychiatry, Icahn School of Medicine at Mount Sinai, New York, New York, USA.'}, {'name': 'Guiqing Cai Kenneth L Davis Elodie Drapeau Joseph I Friedman Vahram Haroutunian Elena Parkhomenko Abraham Reichenberg Jeremy M Silverman  Joseph D Buxbaum', 'org': 'Department of Psychiatry, Icahn School of Medicine at Mount Sinai, New York, New York, USA.'}, {'name': 'Guiqing Cai Kenneth L Davis Elodie Drapeau Joseph I Friedman Vahram Haroutunian Elena Parkhomenko Abraham Reichenberg Jeremy M Silverman  Joseph D Buxbaum', 'org': 'Department of Psychiatry, Icahn School of Medicine at Mount Sinai, New York, New York, USA.'}, {'name': 'Guiqing Cai Kenneth L Davis Elodie Drapeau Joseph I Friedman Vahram Haroutunian Elena Parkhomenko Abraham Reichenberg Jeremy M Silverman  Joseph D Buxbaum', 'org': 'Department of Psychiatry, Icahn School of Medicine at Mount Sinai, New York, New York, USA.'}, {'name': 'Guiqing Cai Kenneth L Davis Elodie Drapeau Joseph I Friedman Vahram Haroutunian Elena Parkhomenko Abraham Reichenberg Jeremy M Silverman  Joseph D Buxbaum', 'org': 'Department of Psychiatry, Icahn School of Medicine at Mount Sinai, New York, New York, USA.'}, {'name': 'Guiqing Cai Kenneth L Davis Elodie Drapeau Joseph I Friedman Vahram Haroutunian Elena Parkhomenko Abraham Reichenberg Jeremy M Silverman  Joseph D Buxbaum', 'org': 'Department of Psychiatry, Icahn School of Medicine at Mount Sinai, New York, New York, USA.'}, {'name': 'Guiqing Cai Kenneth L Davis Elodie Drapeau Joseph I Friedman Vahram Haroutunian Elena Parkhomenko Abraham Reichenberg Jeremy M Silverman  Joseph D Buxbaum', 'org': 'Department of Psychiatry, Icahn School of Medicine at Mount Sinai, New York, New York, USA.'}, {'name': 'Guiqing Cai Kenneth L Davis Elodie Drapeau Joseph I Friedman Vahram Haroutunian Elena Parkhomenko Abraham Reichenberg Jeremy M Silverman  Joseph D Buxbaum', 'org': 'Department of Psychiatry, Icahn School of Medicine at Mount Sinai, New York, New York, USA.'}, {'name': 'Guiqing Cai  Joseph D Buxbaum', 'org': 'Department of Human Genetics, Icahn School of Medicine at Mount Sinai, New York, New York, USA.'}, {'name': 'Guiqing Cai  Joseph D Buxbaum', 'org': 'Department of Human Genetics, Icahn School of Medicine at Mount Sinai, New York, New York, USA.'}, {'name': 'Murray J Cairns Vaughan J Carr Stanley V Catts Frans A Henskens Carmel M Loughland Patricia T Michie Christos Pantelis Ulrich Schall Rodney J Scott Paul A Tooney Jing Qin Wu  Assen V Jablensky', 'org': 'Schizophrenia Research Institute, Sydney, New South Wales, Australia.'}, {'name': 'Murray J Cairns Vaughan J Carr Stanley V Catts Frans A Henskens Carmel M Loughland Patricia T Michie Christos Pantelis Ulrich Schall Rodney J Scott Paul A Tooney Jing Qin Wu  Assen V Jablensky', 'org': 'Schizophrenia Research Institute, Sydney, New South Wales, Australia.'}, {'name': 'Murray J Cairns Vaughan J Carr Stanley V Catts Frans A Henskens Carmel M Loughland Patricia T Michie Christos Pantelis Ulrich Schall Rodney J Scott Paul A Tooney Jing Qin Wu  Assen V Jablensky', 'org': 'Schizophrenia Research Institute, Sydney, New South Wales, Australia.'}, {'name': 'Murray J Cairns Vaughan J Carr Stanley V Catts Frans A Henskens Carmel M Loughland Patricia T Michie Christos Pantelis Ulrich Schall Rodney J Scott Paul A Tooney Jing Qin Wu  Assen V Jablensky', 'org': 'Schizophrenia Research Institute, Sydney, New South Wales, Australia.'}, {'name': 'Murray J Cairns Vaughan J Carr Stanley V Catts Frans A Henskens Carmel M Loughland Patricia T Michie Christos Pantelis Ulrich Schall Rodney J Scott Paul A Tooney Jing Qin Wu  Assen V Jablensky', 'org': 'Schizophrenia Research Institute, Sydney, New South Wales, Australia.'}, {'name': 'Murray J Cairns Vaughan J Carr Stanley V Catts Frans A Henskens Carmel M Loughland Patricia T Michie Christos Pantelis Ulrich Schall Rodney J Scott Paul A Tooney Jing Qin Wu  Assen V Jablensky', 'org': 'Schizophrenia Research Institute, Sydney, New South Wales, Australia.'}, {'name': 'Murray J Cairns Vaughan J Carr Stanley V Catts Frans A Henskens Carmel M Loughland Patricia T Michie Christos Pantelis Ulrich Schall Rodney J Scott Paul A Tooney Jing Qin Wu  Assen V Jablensky', 'org': 'Schizophrenia Research Institute, Sydney, New South Wales, Australia.'}, {'name': 'Murray J Cairns Vaughan J Carr Stanley V Catts Frans A Henskens Carmel M Loughland Patricia T Michie Christos Pantelis Ulrich Schall Rodney J Scott Paul A Tooney Jing Qin Wu  Assen V Jablensky', 'org': 'Schizophrenia Research Institute, Sydney, New South Wales, Australia.'}, {'name': 'Murray J Cairns Vaughan J Carr Stanley V Catts Frans A Henskens Carmel M Loughland Patricia T Michie Christos Pantelis Ulrich Schall Rodney J Scott Paul A Tooney Jing Qin Wu  Assen V Jablensky', 'org': 'Schizophrenia Research Institute, Sydney, New South Wales, Australia.'}, {'name': 'Murray J Cairns Vaughan J Carr Stanley V Catts Frans A Henskens Carmel M Loughland Patricia T Michie Christos Pantelis Ulrich Schall Rodney J Scott Paul A Tooney Jing Qin Wu  Assen V Jablensky', 'org': 'Schizophrenia Research Institute, Sydney, New South Wales, Australia.'}, {'name': 'Murray J Cairns Vaughan J Carr Stanley V Catts Frans A Henskens Carmel M Loughland Patricia T Michie Christos Pantelis Ulrich Schall Rodney J Scott Paul A Tooney Jing Qin Wu  Assen V Jablensky', 'org': 'Schizophrenia Research Institute, Sydney, New South Wales, Australia.'}, {'name': 'Murray J Cairns Vaughan J Carr Stanley V Catts Frans A Henskens Carmel M Loughland Patricia T Michie Christos Pantelis Ulrich Schall Rodney J Scott Paul A Tooney Jing Qin Wu  Assen V Jablensky', 'org': 'Schizophrenia Research Institute, Sydney, New South Wales, Australia.'}, {'name': 'Murray J Cairns Brian J Kelly Carmel M Loughland  Ulrich Schall', 'org': 'Priority Centre for Translational Neuroscience and Mental Health, University of Newcastle, Newcastle, New South Wales, Australia.'}, {'name': 'Murray J Cairns Brian J Kelly Carmel M Loughland  Ulrich Schall', 'org': 'Priority Centre for Translational Neuroscience and Mental Health, University of Newcastle, Newcastle, New South Wales, Australia.'}, {'name': 'Murray J Cairns Brian J Kelly Carmel M Loughland  Ulrich Schall', 'org': 'Priority Centre for Translational Neuroscience and Mental Health, University of Newcastle, Newcastle, New South Wales, Australia.'}, {'name': 'Murray J Cairns Brian J Kelly Carmel M Loughland  Ulrich Schall', 'org': 'Priority Centre for Translational Neuroscience and Mental Health, University of Newcastle, Newcastle, New South Wales, Australia.'}, {'name': 'Murray J Cairns Rodney J Scott Paul A Tooney  Jing Qin Wu', 'org': 'School of Biomedical Sciences and Pharmacy, University of Newcastle, Callaghan, New South Wales, Australia.'}, {'name': 'Murray J Cairns Rodney J Scott Paul A Tooney  Jing Qin Wu', 'org': 'School of Biomedical Sciences and Pharmacy, University of Newcastle, Callaghan, New South Wales, Australia.'}, {'name': 'Murray J Cairns Rodney J Scott Paul A Tooney  Jing Qin Wu', 'org': 'School of Biomedical Sciences and Pharmacy, University of Newcastle, Callaghan, New South Wales, Australia.'}, {'name': 'Murray J Cairns Rodney J Scott Paul A Tooney  Jing Qin Wu', 'org': 'School of Biomedical Sciences and Pharmacy, University of Newcastle, Callaghan, New South Wales, Australia.'}, {'name': 'Dominique Campion', 'org': 'Centre Hospitalier du Rouvray and INSERM U1079 Faculty of Medicine, Rouen, France.'}, {'name': 'Rita M Cantor  Roel A Ophoff', 'org': 'Department of Human Genetics, David Geffen School of Medicine, University of California, Los Angeles, California, USA.'}, {'name': 'Rita M Cantor  Roel A Ophoff', 'org': 'Department of Human Genetics, David Geffen School of Medicine, University of California, Los Angeles, California, USA.'}, {'name': 'Vaughan J Carr', 'org': 'School of Psychiatry, University of New South Wales, Sydney, New South Wales, Australia.'}, {'name': 'Stanley V Catts', 'org': "Royal Brisbane and Women's Hospital, University of Queensland, Brisbane, Queensland, Australia."}, {'name': 'Raymond C K Chan', 'org': 'Institute of Psychology, Chinese Academy of Science, Beijing, China.'}, {'name': 'Eric Y H Chen Miaoxin Li  Pak C Sham', 'org': 'State Key Laboratory for Brain and Cognitive Sciences, Li Ka Shing Faculty of Medicine, The University of Hong Kong, Hong Kong, China.'}, {'name': 'Eric Y H Chen Miaoxin Li  Pak C Sham', 'org': 'State Key Laboratory for Brain and Cognitive Sciences, Li Ka Shing Faculty of Medicine, The University of Hong Kong, Hong Kong, China.'}, {'name': 'Eric Y H Chen Miaoxin Li  Pak C Sham', 'org': 'State Key Laboratory for Brain and Cognitive Sciences, Li Ka Shing Faculty of Medicine, The University of Hong Kong, Hong Kong, China.'}, {'name': 'Eric Y H Chen Ronald Y L Chen Miaoxin Li Hon-Cheong So Emily H M Wong  Pak C Sham', 'org': 'Department of Psychiatry, Li Ka Shing Faculty of Medicine, The University of Hong Kong, Hong Kong, China.'}, {'name': 'Eric Y H Chen Ronald Y L Chen Miaoxin Li Hon-Cheong So Emily H M Wong  Pak C Sham', 'org': 'Department of Psychiatry, Li Ka Shing Faculty of Medicine, The University of Hong Kong, Hong Kong, China.'}, {'name': 'Eric Y H Chen Ronald Y L Chen Miaoxin Li Hon-Cheong So Emily H M Wong  Pak C Sham', 'org': 'Department of Psychiatry, Li Ka Shing Faculty of Medicine, The University of Hong Kong, Hong Kong, China.'}, {'name': 'Eric Y H Chen Ronald Y L Chen Miaoxin Li Hon-Cheong So Emily H M Wong  Pak C Sham', 'org': 'Department of Psychiatry, Li Ka Shing Faculty of Medicine, The University of Hong Kong, Hong Kong, China.'}, {'name': 'Eric Y H Chen Ronald Y L Chen Miaoxin Li Hon-Cheong So Emily H M Wong  Pak C Sham', 'org': 'Department of Psychiatry, Li Ka Shing Faculty of Medicine, The University of Hong Kong, Hong Kong, China.'}, {'name': 'Eric Y H Chen Ronald Y L Chen Miaoxin Li Hon-Cheong So Emily H M Wong  Pak C Sham', 'org': 'Department of Psychiatry, Li Ka Shing Faculty of Medicine, The University of Hong Kong, Hong Kong, China.'}, {'name': 'Wei Cheng', 'org': 'Department of Computer Science, University of North Carolina, Chapel Hill, North Carolina, USA.'}, {'name': 'Eric F C Cheung', 'org': 'Castle Peak Hospital, Hong Kong, China.'}, {'name': 'Siow Ann Chong Jimmy Lee Chee Keong Kang Sim  Mythily Subramaniam', 'org': 'Institute of Mental Health, Singapore, Singapore.'}, {'name': 'Siow Ann Chong Jimmy Lee Chee Keong Kang Sim  Mythily Subramaniam', 'org': 'Institute of Mental Health, Singapore, Singapore.'}, {'name': 'Siow Ann Chong Jimmy Lee Chee Keong Kang Sim  Mythily Subramaniam', 'org': 'Institute of Mental Health, Singapore, Singapore.'}, {'name': 'Siow Ann Chong Jimmy Lee Chee Keong Kang Sim  Mythily Subramaniam', 'org': 'Institute of Mental Health, Singapore, Singapore.'}, {'name': 'C Robert Cloninger  Dragan M Svrakic', 'org': 'Department of Psychiatry, Washington University, St. Louis, Missouri, USA.'}, {'name': 'C Robert Cloninger  Dragan M Svrakic', 'org': 'Department of Psychiatry, Washington University, St. Louis, Missouri, USA.'}, {'name': 'David Cohen', 'org': 'Department of Child and Adolescent Psychiatry, Assistance Publique Hospitaux de Paris, Pierre and Marie Curie Faculty of Medicine and Institute for Intelligent Systems and Robotics, Paris, France.'}, {'name': 'Nadine Cohen', 'org': 'Blue Note Biosciences, Princeton, New Jersey, USA.'}, {'name': 'Benedicto Crespo-Facorro', 'org': 'Department of Psychiatry, University Hospital Marqués de Valdecilla, School of Medicine, University of Cantabria-IDIVAL, Santander, Spain.'}, {'name': 'Benedicto Crespo-Facorro', 'org': 'Centro Investigacion Biomedica en Red Salud Mental, Madrid, Spain.'}, {'name': 'David Curtis', 'org': 'Department of Psychological Medicine, Queen Mary University of London, London, UK.'}, {'name': 'David Curtis Jonathan Pimm Hugh Gurling  Andrew McQuillin', 'org': 'Molecular Psychiatry Laboratory, Division of Psychiatry, University College London, London, UK.'}, {'name': 'David Curtis Jonathan Pimm Hugh Gurling  Andrew McQuillin', 'org': 'Molecular Psychiatry Laboratory, Division of Psychiatry, University College London, London, UK.'}, {'name': 'David Curtis Jonathan Pimm Hugh Gurling  Andrew McQuillin', 'org': 'Molecular Psychiatry Laboratory, Division of Psychiatry, University College London, London, UK.'}, {'name': 'David Curtis Jonathan Pimm Hugh Gurling  Andrew McQuillin', 'org': 'Molecular Psychiatry Laboratory, Division of Psychiatry, University College London, London, UK.'}, {'name': 'Michael Davidson  Mark Weiser', 'org': 'Sheba Medical Center, Tel Hashomer, Israel.'}, {'name': 'Michael Davidson  Mark Weiser', 'org': 'Sheba Medical Center, Tel Hashomer, Israel.'}, {'name': 'Franziska Degenhardt Stefan L Herms Per Hoffmann Andrea Hofman Sven Cichon  Markus M Nöthen', 'org': 'Institute of Human Genetics, University of Bonn, Bonn, Germany.'}, {'name': 'Franziska Degenhardt Stefan L Herms Per Hoffmann Andrea Hofman Sven Cichon  Markus M Nöthen', 'org': 'Institute of Human Genetics, University of Bonn, Bonn, Germany.'}, {'name': 'Franziska Degenhardt Stefan L Herms Per Hoffmann Andrea Hofman Sven Cichon  Markus M Nöthen', 'org': 'Institute of Human Genetics, University of Bonn, Bonn, Germany.'}, {'name': 'Franziska Degenhardt Stefan L Herms Per Hoffmann Andrea Hofman Sven Cichon  Markus M Nöthen', 'org': 'Institute of Human Genetics, University of Bonn, Bonn, Germany.'}, {'name': 'Franziska Degenhardt Stefan L Herms Per Hoffmann Andrea Hofman Sven Cichon  Markus M Nöthen', 'org': 'Institute of Human Genetics, University of Bonn, Bonn, Germany.'}, {'name': 'Franziska Degenhardt Stefan L Herms Per Hoffmann Andrea Hofman Sven Cichon  Markus M Nöthen', 'org': 'Institute of Human Genetics, University of Bonn, Bonn, Germany.'}, {'name': 'Franziska Degenhardt Stefan L Herms Per Hoffmann Andrea Hofman Sven Cichon  Markus M Nöthen', 'org': 'Department of Genomics, Life and Brain Center, Bonn, Germany.'}, {'name': 'Franziska Degenhardt Stefan L Herms Per Hoffmann Andrea Hofman Sven Cichon  Markus M Nöthen', 'org': 'Department of Genomics, Life and Brain Center, Bonn, Germany.'}, {'name': 'Franziska Degenhardt Stefan L Herms Per Hoffmann Andrea Hofman Sven Cichon  Markus M Nöthen', 'org': 'Department of Genomics, Life and Brain Center, Bonn, Germany.'}, {'name': 'Franziska Degenhardt Stefan L Herms Per Hoffmann Andrea Hofman Sven Cichon  Markus M Nöthen', 'org': 'Department of Genomics, Life and Brain Center, Bonn, Germany.'}, {'name': 'Franziska Degenhardt Stefan L Herms Per Hoffmann Andrea Hofman Sven Cichon  Markus M Nöthen', 'org': 'Department of Genomics, Life and Brain Center, Bonn, Germany.'}, {'name': 'Franziska Degenhardt Stefan L Herms Per Hoffmann Andrea Hofman Sven Cichon  Markus M Nöthen', 'org': 'Department of Genomics, Life and Brain Center, Bonn, Germany.'}, {'name': 'Jurgen Del Favero', 'org': 'Applied Molecular Genomics Unit, VIB Department of Molecular Genetics, University of Antwerp, Antwerp, Belgium.'}, {'name': 'Dimitris Dikeos  George N Papadimitriou', 'org': 'First Department of Psychiatry, University of Athens Medical School, Athens, Greece.'}, {'name': 'Dimitris Dikeos  George N Papadimitriou', 'org': 'First Department of Psychiatry, University of Athens Medical School, Athens, Greece.'}, {'name': 'Timothy Dinan', 'org': 'Department of Psychiatry, University College Cork, County Cork, Ireland.'}, {'name': 'Srdjan Djurovic', 'org': 'Department of Medical Genetics, Oslo University Hospital, Oslo, Norway.'}, {'name': 'Gary Donohoe  Derek W Morris', 'org': 'Cognitive Genetics and Therapy Group, School of Psychology and Discipline of Biochemistry, National University of Ireland Galway, County Galway, Ireland.'}, {'name': 'Gary Donohoe  Derek W Morris', 'org': 'Cognitive Genetics and Therapy Group, School of Psychology and Discipline of Biochemistry, National University of Ireland Galway, County Galway, Ireland.'}, {'name': 'Jubao Duan Alan R Sanders  Pablo V Gejman', 'org': 'Department of Psychiatry and Behavioral Sciences, NorthShore University HealthSystem, Evanston, Illinois, USA.'}, {'name': 'Jubao Duan Alan R Sanders  Pablo V Gejman', 'org': 'Department of Psychiatry and Behavioral Sciences, NorthShore University HealthSystem, Evanston, Illinois, USA.'}, {'name': 'Jubao Duan Alan R Sanders  Pablo V Gejman', 'org': 'Department of Psychiatry and Behavioral Sciences, NorthShore University HealthSystem, Evanston, Illinois, USA.'}, {'name': 'Jubao Duan', 'org': 'Department of Psychiatry and Behavioral Neuroscience, University of Chicago, Chicago, Illinois, USA.'}, {'name': 'Frank Dudbridge', 'org': 'Department of Non-Communicable Disease Epidemiology, London School of Hygiene and Tropical Medicine, London, UK.'}, {'name': 'Peter Eichhammer', 'org': 'Department of Psychiatry, University of Regensburg, Regensburg, Germany.'}, {'name': 'Johan Eriksson', 'org': 'Folkhälsan Research Center, Helsinki, Finland, Biomedicum Helsinki, Helsinki, Finland.'}, {'name': 'Johan Eriksson  Veikko Salomaa', 'org': 'National Institute for Health and Welfare, Helsinki, Finland.'}, {'name': 'Johan Eriksson  Veikko Salomaa', 'org': 'National Institute for Health and Welfare, Helsinki, Finland.'}, {'name': 'Johan Eriksson', 'org': 'Department of General Practice, Helsinki University Central Hospital, University of Helsinki, Helsinki, Finland.'}, {'name': 'Laurent Essioux', 'org': 'Translational Technologies and Bioinformatics, Pharma Research and Early Development, F. Hoffman-La Roche, Basel, Switzerland.'}, {'name': 'Ayman H Fanous', 'org': 'Department of Medicine, McGill University, Montreal, Quebec, Canada.'}, {'name': 'Ayman H Fanous', 'org': 'Department of Psychiatry, Georgetown University School of Medicine, Washington, DC, USA.'}, {'name': 'Ayman H Fanous', 'org': 'Department of Psychiatry, Virginia Commonwealth University School of Medicine, Richmond, Virginia, USA.'}, {'name': 'Ayman H Fanous', 'org': 'Department of Psychiatry, Keck School of Medicine of the University of Southern California, Los Angeles, California, USA.'}, {'name': 'Josef Frank Thomas G Schulze Stephanie H Witt  Marcella Rietschel', 'org': 'Department of Genetic Epidemiology in Psychiatry, Central Institute of Mental Health, Medical Faculty Mannheim, University of Heidelberg, Heidelberg, Mannheim, Germany.'}, {'name': 'Josef Frank Thomas G Schulze Stephanie H Witt  Marcella Rietschel', 'org': 'Department of Genetic Epidemiology in Psychiatry, Central Institute of Mental Health, Medical Faculty Mannheim, University of Heidelberg, Heidelberg, Mannheim, Germany.'}, {'name': 'Josef Frank Thomas G Schulze Stephanie H Witt  Marcella Rietschel', 'org': 'Department of Genetic Epidemiology in Psychiatry, Central Institute of Mental Health, Medical Faculty Mannheim, University of Heidelberg, Heidelberg, Mannheim, Germany.'}, {'name': 'Josef Frank Thomas G Schulze Stephanie H Witt  Marcella Rietschel', 'org': 'Department of Genetic Epidemiology in Psychiatry, Central Institute of Mental Health, Medical Faculty Mannheim, University of Heidelberg, Heidelberg, Mannheim, Germany.'}, {'name': 'Lude Franke  Juha Karjalainen', 'org': 'Department of Genetics, University of Groningen, University Medical Centre Groningen, Groningen, the Netherlands.'}, {'name': 'Lude Franke  Juha Karjalainen', 'org': 'Department of Genetics, University of Groningen, University Medical Centre Groningen, Groningen, the Netherlands.'}, {'name': 'Robert Freedman  Ann Olincy', 'org': 'Department of Psychiatry, University of Colorado Denver, Aurora, Colorado, USA.'}, {'name': 'Robert Freedman  Ann Olincy', 'org': 'Department of Psychiatry, University of Colorado Denver, Aurora, Colorado, USA.'}, {'name': 'Nelson B Freimer  Roel A Ophoff', 'org': 'Center for Neurobehavioral Genetics, Semel Institute for Neuroscience and Human Behavior, University of California, Los Angeles, California, USA.'}, {'name': 'Nelson B Freimer  Roel A Ophoff', 'org': 'Center for Neurobehavioral Genetics, Semel Institute for Neuroscience and Human Behavior, University of California, Los Angeles, California, USA.'}, {'name': 'Elliot S Gershon', 'org': 'Departments of Psychiatry and Human Genetics, University of Chicago, Chicago, Illinois, USA.'}, {'name': 'Ina Giegling Annette M Hartmann Bettina Konte  Dan Rujescu', 'org': 'Department of Psychiatry, University of Halle, Halle, Germany.'}, {'name': 'Ina Giegling Annette M Hartmann Bettina Konte  Dan Rujescu', 'org': 'Department of Psychiatry, University of Halle, Halle, Germany.'}, {'name': 'Ina Giegling Annette M Hartmann Bettina Konte  Dan Rujescu', 'org': 'Department of Psychiatry, University of Halle, Halle, Germany.'}, {'name': 'Ina Giegling Annette M Hartmann Bettina Konte  Dan Rujescu', 'org': 'Department of Psychiatry, University of Halle, Halle, Germany.'}, {'name': 'Ina Giegling  Dan Rujescu', 'org': 'Department of Psychiatry, University of Munich, Munich, Germany.'}, {'name': 'Ina Giegling  Dan Rujescu', 'org': 'Department of Psychiatry, University of Munich, Munich, Germany.'}, {'name': 'Stephanie Godard', 'org': 'Departments of Psychiatry and Human and Molecular Genetics, INSERM, Institut de Myologie, Hôpital de la Pitiè-Salpêtrière, Paris, France.'}, {'name': 'Srihari Gopal Dai Wang  Qingqin S Li', 'org': 'Neuroscience Therapeutic Area, Janssen Research and Development, Raritan, New Jersey, USA.'}, {'name': 'Srihari Gopal Dai Wang  Qingqin S Li', 'org': 'Neuroscience Therapeutic Area, Janssen Research and Development, Raritan, New Jersey, USA.'}, {'name': 'Srihari Gopal Dai Wang  Qingqin S Li', 'org': 'Neuroscience Therapeutic Area, Janssen Research and Development, Raritan, New Jersey, USA.'}, {'name': 'Jacob Gratten S Hong Lee Naomi R Wray  Peter M Visscher', 'org': 'Queensland Brain Institute, The University of Queensland, Brisbane, Queensland, Australia.'}, {'name': 'Jacob Gratten S Hong Lee Naomi R Wray  Peter M Visscher', 'org': 'Queensland Brain Institute, The University of Queensland, Brisbane, Queensland, Australia.'}, {'name': 'Jacob Gratten S Hong Lee Naomi R Wray  Peter M Visscher', 'org': 'Queensland Brain Institute, The University of Queensland, Brisbane, Queensland, Australia.'}, {'name': 'Jacob Gratten S Hong Lee Naomi R Wray  Peter M Visscher', 'org': 'Queensland Brain Institute, The University of Queensland, Brisbane, Queensland, Australia.'}, {'name': 'Lieuwe de Haan  Carin J Meijer', 'org': 'Academic Medical Centre University of Amsterdam, Department of Psychiatry, Amsterdam, the Netherlands.'}, {'name': 'Lieuwe de Haan  Carin J Meijer', 'org': 'Academic Medical Centre University of Amsterdam, Department of Psychiatry, Amsterdam, the Netherlands.'}, {'name': 'Mark Hansen', 'org': 'Illumina, La Jolla, California, California, USA.'}, {'name': 'Vahram Haroutunian', 'org': 'JJ Peters Veterans Affairs Medical Center, Bronx, New York, New York, USA.'}, {'name': 'Vahram Haroutunian Joseph D Buxbaum  Pamela Sklar', 'org': 'Friedman Brain Institute, Icahn School of Medicine at Mount Sinai, New York, New York, USA.'}, {'name': 'Vahram Haroutunian Joseph D Buxbaum  Pamela Sklar', 'org': 'Friedman Brain Institute, Icahn School of Medicine at Mount Sinai, New York, New York, USA.'}, {'name': 'Vahram Haroutunian Joseph D Buxbaum  Pamela Sklar', 'org': 'Friedman Brain Institute, Icahn School of Medicine at Mount Sinai, New York, New York, USA.'}, {'name': 'Frans A Henskens', 'org': 'School of Electrical Engineering and Computer Science, University of Newcastle, Newcastle, New South Wales, Australia.'}, {'name': 'Frans A Henskens', 'org': 'Priority Research Centre for Health Behaviour, University of Newcastle, Newcastle, New South Wales, Australia.'}, {'name': 'Stefan L Herms Per Hoffmann  Sven Cichon', 'org': 'Division of Medical Genetics, Department of Biomedicine, University of Basel, Basel, Switzerland.'}, {'name': 'Stefan L Herms Per Hoffmann  Sven Cichon', 'org': 'Division of Medical Genetics, Department of Biomedicine, University of Basel, Basel, Switzerland.'}, {'name': 'Stefan L Herms Per Hoffmann  Sven Cichon', 'org': 'Division of Medical Genetics, Department of Biomedicine, University of Basel, Basel, Switzerland.'}, {'name': 'Mads V Hollegaard  David M Hougaard', 'org': 'Section of Neonatal Screening and Hormones, Department of Clinical Biochemistry, Immunology and Genetics, Statens Serum Institut, Copenhagen, Denmark.'}, {'name': 'Mads V Hollegaard  David M Hougaard', 'org': 'Section of Neonatal Screening and Hormones, Department of Clinical Biochemistry, Immunology and Genetics, Statens Serum Institut, Copenhagen, Denmark.'}, {'name': 'Masashi Ikeda  Nakao Iwata', 'org': 'Department of Psychiatry, Fujita Health University School of Medicine, Toyoake, Aichi, Japan.'}, {'name': 'Masashi Ikeda  Nakao Iwata', 'org': 'Department of Psychiatry, Fujita Health University School of Medicine, Toyoake, Aichi, Japan.'}, {'name': 'Inge Joa', 'org': 'Regional Centre for Clinical Research in Psychosis, Department of Psychiatry, Stavanger University Hospital, Stavanger, Norway.'}, {'name': 'Antonio Julià  Sara Marsal', 'org': "Rheumatology Research Group, Vall d'Hebron Research Institute, Barcelona, Spain."}, {'name': 'Antonio Julià  Sara Marsal', 'org': "Rheumatology Research Group, Vall d'Hebron Research Institute, Barcelona, Spain."}, {'name': 'Luba Kalaydjieva', 'org': 'Centre for Medical Research, The University of Western Australia, Perth, Western Australia, Australia.'}, {'name': 'Luba Kalaydjieva', 'org': 'The Perkins Institute for Medical Research, The University of Western Australia, Perth, Western Australia, Australia.'}, {'name': 'Sena Karachanak-Yankova', 'org': 'Department of Medical Genetics, Medical University, Sofia, Bulgaria.'}, {'name': 'Matthew C Keller', 'org': 'Department of Psychology, University of Colorado Boulder, Boulder, Colorado, USA.'}, {'name': 'James L Kennedy Clement C Zai  Jo Knight', 'org': 'Campbell Family Mental Health Research Institute, Centre for Addiction and Mental Health, Toronto, Ontario, Canada.'}, {'name': 'James L Kennedy Clement C Zai  Jo Knight', 'org': 'Campbell Family Mental Health Research Institute, Centre for Addiction and Mental Health, Toronto, Ontario, Canada.'}, {'name': 'James L Kennedy Clement C Zai  Jo Knight', 'org': 'Campbell Family Mental Health Research Institute, Centre for Addiction and Mental Health, Toronto, Ontario, Canada.'}, {'name': 'James L Kennedy Clement C Zai  Jo Knight', 'org': 'Department of Psychiatry, University of Toronto, Toronto, Ontario, Canada.'}, {'name': 'James L Kennedy Clement C Zai  Jo Knight', 'org': 'Department of Psychiatry, University of Toronto, Toronto, Ontario, Canada.'}, {'name': 'James L Kennedy Clement C Zai  Jo Knight', 'org': 'Department of Psychiatry, University of Toronto, Toronto, Ontario, Canada.'}, {'name': 'James L Kennedy  Jo Knight', 'org': 'Institute of Medical Science, University of Toronto, Toronto, Ontario, Canada.'}, {'name': 'James L Kennedy  Jo Knight', 'org': 'Institute of Medical Science, University of Toronto, Toronto, Ontario, Canada.'}, {'name': 'Andrey Khrunin Svetlana Limborska  Petr Slominsky', 'org': 'Institute of Molecular Genetics, Russian Academy of Sciences, Moscow, Russia.'}, {'name': 'Andrey Khrunin Svetlana Limborska  Petr Slominsky', 'org': 'Institute of Molecular Genetics, Russian Academy of Sciences, Moscow, Russia.'}, {'name': 'Andrey Khrunin Svetlana Limborska  Petr Slominsky', 'org': 'Institute of Molecular Genetics, Russian Academy of Sciences, Moscow, Russia.'}, {'name': 'Janis Klovins  Liene Nikitina-Zake', 'org': 'Latvian Biomedical Research and Study Centre, Riga, Latvia.'}, {'name': 'Janis Klovins  Liene Nikitina-Zake', 'org': 'Latvian Biomedical Research and Study Centre, Riga, Latvia.'}, {'name': 'James A Knowles Michele T Pato  Carlos N Pato', 'org': 'Department of Psychiatry and Zilkha Neurogenetics Institute, Keck School of Medicine at University of Southern California, Los Angeles, California, USA.'}, {'name': 'James A Knowles Michele T Pato  Carlos N Pato', 'org': 'Department of Psychiatry and Zilkha Neurogenetics Institute, Keck School of Medicine at University of Southern California, Los Angeles, California, USA.'}, {'name': 'James A Knowles Michele T Pato  Carlos N Pato', 'org': 'Department of Psychiatry and Zilkha Neurogenetics Institute, Keck School of Medicine at University of Southern California, Los Angeles, California, USA.'}, {'name': 'Vaidutis Kucinskas  Zita Ausrele Kucinskiene', 'org': 'Faculty of Medicine, Vilnius University, Vilnius, Lithuania.'}, {'name': 'Vaidutis Kucinskas  Zita Ausrele Kucinskiene', 'org': 'Faculty of Medicine, Vilnius University, Vilnius, Lithuania.'}, {'name': 'Hana Kuzelova-Ptackova', 'org': 'Department of Biology and Medical Genetics, 2nd Faculty of Medicine and University Hospital Motol, Prague, Czech Republic.'}, {'name': 'Claudine Laurent', 'org': 'Department of Child and Adolescent Psychiatry, Pierre and Marie Curie Faculty of Medicine, Paris, France.'}, {'name': 'Jimmy Lee Chee Keong', 'org': 'Duke–National University Singapore Graduate Medical School, Singapore, Singapore.'}, {'name': 'Bernard Lerer', 'org': 'Department of Psychiatry, Hadassah-Hebrew University Medical Center, Jerusalem, Israel.'}, {'name': 'Miaoxin Li  Pak C Sham', 'org': 'Centre for Genomic Sciences, The University of Hong Kong, Hong Kong, China.'}, {'name': 'Miaoxin Li  Pak C Sham', 'org': 'Centre for Genomic Sciences, The University of Hong Kong, Hong Kong, China.'}, {'name': 'Tao Li', 'org': 'Mental Health Centre and Psychiatric Laboratory, West China Hospital, Sichuan University, Chengdu, Sichuan, China.'}, {'name': 'Kung-Yee Liang', 'org': 'Department of Biostatistics, Johns Hopkins University Bloomberg School of Public Health, Baltimore, Maryland, USA.'}, {'name': 'Jeffrey Lieberman  T Scott Stroup', 'org': 'Department of Psychiatry, Columbia University, New York, New York, USA.'}, {'name': 'Jeffrey Lieberman  T Scott Stroup', 'org': 'Department of Psychiatry, Columbia University, New York, New York, USA.'}, {'name': 'Jouko Lönnqvist  Jaana Suvisaari', 'org': 'Department of Mental Health and Substance Abuse Services, National Institute for Health and Welfare, Helsinki, Finland.'}, {'name': 'Jouko Lönnqvist  Jaana Suvisaari', 'org': 'Department of Mental Health and Substance Abuse Services, National Institute for Health and Welfare, Helsinki, Finland.'}, {'name': 'Jan Lubinski', 'org': 'Department of Genetics and Pathology, International Hereditary Cancer Center, Pomeranian Medical University in Szczecin, Szczecin, Poland.'}, {'name': 'Milan Macek Jr', 'org': 'Department of Biology and Medical Genetics, 2nd Faculty of Medicine and University Hospital Motol, Prague, Czech Republic.'}, {'name': 'Brion S Maher', 'org': 'Department of Mental Health, Bloomberg School of Public Health, Johns Hopkins University, Baltimore, Maryland, USA.'}, {'name': 'Wolfgang Maier', 'org': 'Department of Psychiatry, University of Bonn, Bonn, Germany.'}, {'name': 'Jacques Mallet', 'org': 'Centre National de la Recherche Scientifique, Laboratoire de Génétique Moléculaire de la Neurotransmission et des Processus Neurodégénératifs, Hôpital de la Pitié Salpêtrière, Paris, France.'}, {'name': 'Morten Mattingsdal', 'org': 'Research Unit, Sørlandet Hospital, Kristiansand, Norway.'}, {'name': 'Colm McDonald', 'org': 'Department of Psychiatry, National University of Ireland Galway, County Galway, Ireland.'}, {'name': 'Sandra Meier  Jana Strohmaier', 'org': 'Department of Genetic Epidemiology in Psychiatry, Central Institute of Mental Health, Medical Faculty Mannheim, University of Heidelberg, Heidelberg, Mannheim, Germany.'}, {'name': 'Sandra Meier  Jana Strohmaier', 'org': 'Department of Genetic Epidemiology in Psychiatry, Central Institute of Mental Health, Medical Faculty Mannheim, University of Heidelberg, Heidelberg, Mannheim, Germany.'}, {'name': 'Andres Metspalu Lili Milani  Mari Nelis', 'org': 'Estonian Genome Center, University of Tartu, Tartu, Estonia.'}, {'name': 'Andres Metspalu Lili Milani  Mari Nelis', 'org': 'Estonian Genome Center, University of Tartu, Tartu, Estonia.'}, {'name': 'Andres Metspalu Lili Milani  Mari Nelis', 'org': 'Estonian Genome Center, University of Tartu, Tartu, Estonia.'}, {'name': 'Patricia T Michie', 'org': 'School of Psychology, University of Newcastle, Newcastle, New South Wales, Australia.'}, {'name': 'Vihra Milanova', 'org': 'First Psychiatric Clinic, Medical University, Sofia, Bulgaria.'}, {'name': 'Younes Mokrab', 'org': 'Eli Lilly and Company Limited, Windlesham, Surrey, UK.'}, {'name': 'Bertram Müller-Myhsok', 'org': 'Max Planck Institute of Psychiatry, Munich, Germany.'}, {'name': 'Bertram Müller-Myhsok', 'org': 'Institute of Translational Medicine, University of Liverpool, Liverpool, UK.'}, {'name': 'Bertram Müller-Myhsok', 'org': 'Munich Cluster for Systems Neurology (SyNergy), Munich, Germany.'}, {'name': 'Kieran C Murphy', 'org': 'Department of Psychiatry, Royal College of Surgeons in Ireland, Dublin, Ireland.'}, {'name': 'Robin M Murray  John Powell', 'org': "King's College London, London, UK."}, {'name': 'Robin M Murray  John Powell', 'org': "King's College London, London, UK."}, {'name': 'Inez Myin-Germeys  Jim Van Os', 'org': 'Maastricht University Medical Centre, South Limburg Mental Health Research and Teaching Network, EURON, Maastricht, the Netherlands.'}, {'name': 'Inez Myin-Germeys  Jim Van Os', 'org': 'Maastricht University Medical Centre, South Limburg Mental Health Research and Teaching Network, EURON, Maastricht, the Netherlands.'}, {'name': 'Igor Nenadic', 'org': 'Department of Psychiatry and Psychotherapy, Jena University Hospital, Jena, Germany.'}, {'name': 'Deborah A Nertney  Bryan J Mowry', 'org': 'Queensland Centre for Mental Health Research, University of Queensland, Brisbane, Queensland, Australia.'}, {'name': 'Deborah A Nertney  Bryan J Mowry', 'org': 'Queensland Centre for Mental Health Research, University of Queensland, Brisbane, Queensland, Australia.'}, {'name': 'Gerald Nestadt  Ann E Pulver', 'org': 'Department of Psychiatry and Behavioral Sciences, Johns Hopkins University School of Medicine, Baltimore, Maryland, USA.'}, {'name': 'Gerald Nestadt  Ann E Pulver', 'org': 'Department of Psychiatry and Behavioral Sciences, Johns Hopkins University School of Medicine, Baltimore, Maryland, USA.'}, {'name': 'Kristin K Nicodemus', 'org': 'Department of Psychiatry, Trinity College Dublin, Dublin, Ireland.'}, {'name': 'Laura Nisenbaum', 'org': 'Eli Lilly and Company, Lilly Corporate Center, Indianapolis, Indiana, USA.'}, {'name': 'Annelie Nordin  Rolf Adolfsson', 'org': 'Department of Clinical Sciences, Psychiatry, Umeå University, Umeå, Sweden.'}, {'name': 'Annelie Nordin  Rolf Adolfsson', 'org': 'Department of Clinical Sciences, Psychiatry, Umeå University, Umeå, Sweden.'}, {'name': "Eadbhard O'Callaghan", 'org': 'DETECT Early Intervention Service for Psychosis, Blackrock, County Dublin, Ireland.'}, {'name': "F Anthony O'Neill", 'org': "Centre for Public Health, Institute of Clinical Sciences, Queen's University Belfast, Belfast, UK."}, {'name': 'Sang-Yun Oh', 'org': 'Lawrence Berkeley National Laboratory, University of California at Berkeley, Berkeley, California, USA.'}, {'name': 'Jim Van Os  Rocio Perez-Iglesias', 'org': "Institute of Psychiatry, King's College London, London, UK."}, {'name': 'Jim Van Os  Rocio Perez-Iglesias', 'org': "Institute of Psychiatry, King's College London, London, UK."}, {'name': 'Christos Pantelis', 'org': 'Melbourne Neuropsychiatry Centre, University of Melbourne & Melbourne Health, Melbourne, Victoria, Australia.'}, {'name': 'Tiina Paunio  Olli Pietiläinen', 'org': 'Public Health Genomics Unit, National Institute for Health and Welfare, Helsinki, Finland.'}, {'name': 'Tiina Paunio  Olli Pietiläinen', 'org': 'Public Health Genomics Unit, National Institute for Health and Welfare, Helsinki, Finland.'}, {'name': 'Tiina Paunio', 'org': 'Department of Psychiatry, University of Helsinki, Helsinki, Finland.'}, {'name': 'Olli Pietiläinen', 'org': 'Institute for Molecular Medicine Finland, FIMM, University of Helsinki, Helsinki, Finland.'}, {'name': 'Shaun M Purcell Panos Roussos  Pamela Sklar', 'org': 'Division of Psychiatric Genomics, Department of Psychiatry, Icahn School of Medicine at Mount Sinai, New York, New York, USA.'}, {'name': 'Shaun M Purcell Panos Roussos  Pamela Sklar', 'org': 'Division of Psychiatric Genomics, Department of Psychiatry, Icahn School of Medicine at Mount Sinai, New York, New York, USA.'}, {'name': 'Shaun M Purcell Panos Roussos  Pamela Sklar', 'org': 'Division of Psychiatric Genomics, Department of Psychiatry, Icahn School of Medicine at Mount Sinai, New York, New York, USA.'}, {'name': 'Digby Quested  Simon Lovestone', 'org': 'Department of Psychiatry, University of Oxford, Oxford, UK.'}, {'name': 'Digby Quested  Simon Lovestone', 'org': 'Department of Psychiatry, University of Oxford, Oxford, UK.'}, {'name': 'Panos Roussos  Pamela Sklar', 'org': 'Institute for Multiscale Biology, Icahn School of Medicine at Mount Sinai, New York, New York, USA.'}, {'name': 'Panos Roussos  Pamela Sklar', 'org': 'Institute for Multiscale Biology, Icahn School of Medicine at Mount Sinai, New York, New York, USA.'}, {'name': 'Alan R Sanders  Pablo V Gejman', 'org': 'Department of Psychiatry and Behavioral Neuroscience, University of Chicago, Chicago, Illinois, USA.'}, {'name': 'Alan R Sanders  Pablo V Gejman', 'org': 'Department of Psychiatry and Behavioral Neuroscience, University of Chicago, Chicago, Illinois, USA.'}, {'name': 'Christian R Schubert  Jens R Wendland', 'org': 'PharmaTherapeutics Clinical Research, Pfizer Worldwide Research and Development, Cambridge, Massachusetts, USA.'}, {'name': 'Christian R Schubert  Jens R Wendland', 'org': 'PharmaTherapeutics Clinical Research, Pfizer Worldwide Research and Development, Cambridge, Massachusetts, USA.'}, {'name': 'Thomas G Schulze  Knut Schnell', 'org': 'Department of Psychiatry and Psychotherapy, University of Gottingen, Gottingen, Germany.'}, {'name': 'Thomas G Schulze  Knut Schnell', 'org': 'Department of Psychiatry and Psychotherapy, University of Gottingen, Gottingen, Germany.'}, {'name': 'Sibylle G Schwab', 'org': 'Psychiatry and Psychotherapy Clinic, University of Erlangen, Erlangen, Germany.'}, {'name': 'Rodney J Scott', 'org': 'Hunter New England Health Service, Newcastle, New South Wales, Australia.'}, {'name': 'Jianxin Shi', 'org': 'Division of Cancer Epidemiology and Genetics, National Cancer Institute, Bethesda, Maryland, USA.'}, {'name': 'Jeremy M Silverman', 'org': 'Research and Development, Bronx Veterans Affairs Medical Center, New York, New York, USA.'}, {'name': 'Chris C A Spencer', 'org': 'Wellcome Trust Centre for Human Genetics, Oxford, UK.'}, {'name': 'Elisabeth Stogmann  Fritz Zimprich', 'org': 'Department of Clinical Neurology, Medical University of Vienna, Vienna, Austria.'}, {'name': 'Elisabeth Stogmann  Fritz Zimprich', 'org': 'Department of Clinical Neurology, Medical University of Vienna, Vienna, Austria.'}, {'name': 'Richard E Straub Daniel R Weinberger Qiang Chen Aaron L Goldman  Venkata S Mattay', 'org': 'Lieber Institute for Brain Development, Baltimore, Maryland, USA.'}, {'name': 'Richard E Straub Daniel R Weinberger Qiang Chen Aaron L Goldman  Venkata S Mattay', 'org': 'Lieber Institute for Brain Development, Baltimore, Maryland, USA.'}, {'name': 'Richard E Straub Daniel R Weinberger Qiang Chen Aaron L Goldman  Venkata S Mattay', 'org': 'Lieber Institute for Brain Development, Baltimore, Maryland, USA.'}, {'name': 'Richard E Straub Daniel R Weinberger Qiang Chen Aaron L Goldman  Venkata S Mattay', 'org': 'Lieber Institute for Brain Development, Baltimore, Maryland, USA.'}, {'name': 'Richard E Straub Daniel R Weinberger Qiang Chen Aaron L Goldman  Venkata S Mattay', 'org': 'Lieber Institute for Brain Development, Baltimore, Maryland, USA.'}, {'name': 'Eric Strengman', 'org': 'Department of Medical Genetics, University Medical Centre Utrecht, Utrecht, the Netherlands.'}, {'name': 'Srinivas Thirumalai', 'org': 'Berkshire Healthcare NHS Foundation Trust, Bracknell, UK.'}, {'name': 'Draga Toncheva', 'org': 'Department of Medical Genetics, Medical University, Sofia, Bulgaria.'}, {'name': 'Paul A Tooney', 'org': 'Priority Research Centre for Translational Neuroscience and Mental Health, University of Newcastle, Newcastle, New South Wales, Australia.'}, {'name': 'Juha Veijola', 'org': 'Department of Psychiatry, University of Oulu, Oulu, Finland.'}, {'name': 'Juha Veijola', 'org': 'University Hospital of Oulu, Oulu, Finland.'}, {'name': 'John Waddington', 'org': 'Molecular and Cellular Therapeutics, Royal College of Surgeons in Ireland, Dublin, Ireland.'}, {'name': 'Dermot Walsh', 'org': 'Health Research Board, Dublin, Ireland.'}, {'name': 'Qiang Wang', 'org': 'Mental Health Centre and Psychiatric Laboratory, West China Hospital, Sichuan University, Chendu, Sichuan, China.'}, {'name': 'Dieter B Wildenauer', 'org': 'School of Psychiatry and Clinical Neurosciences, The University of Western Australia, Perth, Western Australia, Australia.'}, {'name': 'Hualin Simon Xi', 'org': 'Computational Sciences CoE, Pfizer Worldwide Research and Development, Cambridge, Massachusetts, USA.'}, {'name': 'Xuebin Zheng  Jianjun Liu', 'org': 'Human Genetics, Genome Institute of Singapore, A*STAR, Singapore, Singapore.'}, {'name': 'Xuebin Zheng  Jianjun Liu', 'org': 'Human Genetics, Genome Institute of Singapore, A*STAR, Singapore, Singapore.'}, {'name': 'Douglas H R Blackwood', 'org': 'Division of Psychiatry, University of Edinburgh, Edinburgh, UK.'}, {'name': 'Elvira Bramon', 'org': 'University College London, London, UK.'}, {'name': 'Joseph D Buxbaum', 'org': 'Department of Neuroscience, Icahn School of Medicine at Mount Sinai, New York, New York, USA.'}, {'name': 'Sven Cichon  Thomas W Mühleisen', 'org': 'Institute of Neuroscience and Medicine (INM-1), Research Center Juelich, Juelich, Germany.'}, {'name': 'Sven Cichon  Thomas W Mühleisen', 'org': 'Institute of Neuroscience and Medicine (INM-1), Research Center Juelich, Juelich, Germany.'}, {'name': 'Ariel Darvasi', 'org': 'Department of Genetics, The Hebrew University of Jerusalem, Jerusalem, Israel.'}, {'name': 'Enrico Domenici', 'org': 'Neuroscience Discovery and Translational Area, Pharma Research and Early Development, F. Hoffman-La Roche, Basel, Switzerland.'}, {'name': 'Assen V Jablensky', 'org': 'School of Psychiatry and Clinical Neurosciences, The University of Western Australia, Perth, Western Australia, Australia.'}, {'name': 'Assen V Jablensky', 'org': 'Centre for Clinical Research in Neuropsychiatry, School of Psychiatry and Clinical Neurosciences, The University of Western Australia, Medical Research Foundation Building, Perth, Western Australia, Australia.'}, {'name': 'Assen V Jablensky', 'org': 'The Perkins Institute for Medical Research, The University of Western Australia, Perth, Western Australia, Australia.'}, {'name': 'Todd Lencz  Anil K Malhotra', 'org': 'The Zucker Hillside Hospital, Glen Oaks, New York, USA.'}, {'name': 'Todd Lencz  Anil K Malhotra', 'org': 'The Zucker Hillside Hospital, Glen Oaks, New York, USA.'}, {'name': 'Todd Lencz  Anil K Malhotra', 'org': 'The Feinstein Institute for Medical Research, Manhasset, New York, USA.'}, {'name': 'Todd Lencz  Anil K Malhotra', 'org': 'The Feinstein Institute for Medical Research, Manhasset, New York, USA.'}, {'name': 'Todd Lencz  Anil K Malhotra', 'org': 'The Hofstra North Shore–Long Island Jewish School of Medicine, Hempstead, New York, USA.'}, {'name': 'Todd Lencz  Anil K Malhotra', 'org': 'The Hofstra North Shore–Long Island Jewish School of Medicine, Hempstead, New York, USA.'}, {'name': 'Jianjun Liu', 'org': 'Saw Swee Hock School of Public Health, National University of Singapore, Singapore, Singapore.'}, {'name': 'Bryan J Mowry', 'org': 'Queensland Brain Institute, The University of Queensland, Brisbane, Queensland, Australia.'}, {'name': 'Danielle Posthuma', 'org': 'Department of Functional Genomics, Center for Neurogenomics and Cognitive Research, Neuroscience Campus Amsterdam, VU University, Amsterdam, the Netherlands.'}, {'name': 'Danielle Posthuma', 'org': 'Department of Complex Trait Genetics, Neuroscience Campus Amsterdam, VU University Medical Center Amsterdam, Amsterdam, the Netherlands.'}, {'name': 'Danielle Posthuma', 'org': 'Department of Child and Adolescent Psychiatry, Erasmus University Medical Centre, Rotterdam, the Netherlands.'}, {'name': 'David St Clair', 'org': 'University of Aberdeen, Institute of Medical Sciences, Aberdeen, UK.'}, {'name': 'Daniel R Weinberger', 'org': 'Departments of Psychiatry, Neurology, Neuroscience and Institute of Genetic Medicine, Johns Hopkins School of Medicine, Baltimore, Maryland, USA.'}, {'name': 'Sylvane Desrivières Tianye Jia Christine Macare  Gunter Schumann', 'org': "MRC-SGDP Centre, Institute of Psychiatry, King's College London, London, UK."}, {'name': 'Sylvane Desrivières Tianye Jia Christine Macare  Gunter Schumann', 'org': "MRC-SGDP Centre, Institute of Psychiatry, King's College London, London, UK."}, {'name': 'Sylvane Desrivières Tianye Jia Christine Macare  Gunter Schumann', 'org': "MRC-SGDP Centre, Institute of Psychiatry, King's College London, London, UK."}, {'name': 'Sylvane Desrivières Tianye Jia Christine Macare  Gunter Schumann', 'org': "MRC-SGDP Centre, Institute of Psychiatry, King's College London, London, UK."}, {'name': 'Neda Jahanshad  Christopher R K Ching', 'org': 'Imaging Genetics Center, Institute for Neuroimaging & Informatics, Keck School of Medicine of the University of Southern California, Los Angeles, USA.'}, {'name': 'Neda Jahanshad  Christopher R K Ching', 'org': 'Imaging Genetics Center, Institute for Neuroimaging & Informatics, Keck School of Medicine of the University of Southern California, Los Angeles, USA.'}, {'name': 'Roberto Toro', 'org': 'Institut Pasteur, Paris, France.'}, {'name': 'Katharina Wittfeld  Wolfgang Hoffmann', 'org': 'German Center for Neurodegenerative Diseases (DZNE) Rostock/Greifswald, Greifswald, Germany.'}, {'name': 'Katharina Wittfeld  Wolfgang Hoffmann', 'org': 'German Center for Neurodegenerative Diseases (DZNE) Rostock/Greifswald, Greifswald, Germany.'}, {'name': 'Katharina Wittfeld Deborah Janowitz  Hans J Grabe', 'org': 'Department of Psychiatry, University Medicine Greifswald, Greifswald, Germany.'}, {'name': 'Katharina Wittfeld Deborah Janowitz  Hans J Grabe', 'org': 'Department of Psychiatry, University Medicine Greifswald, Greifswald, Germany.'}, {'name': 'Katharina Wittfeld Deborah Janowitz  Hans J Grabe', 'org': 'Department of Psychiatry, University Medicine Greifswald, Greifswald, Germany.'}, {'name': 'Lucija Abramovic Marc M Bohlken Marco P Boks Kristel R van Eijk Neeltje E M van Haren Rachel M Brouwer  Hilleke E Hulshoff Pol', 'org': 'Brain Center Rudolf Magnus, Department of Psychiatry, UMC Utrecht, Utrecht, the Netherlands.'}, {'name': 'Lucija Abramovic Marc M Bohlken Marco P Boks Kristel R van Eijk Neeltje E M van Haren Rachel M Brouwer  Hilleke E Hulshoff Pol', 'org': 'Brain Center Rudolf Magnus, Department of Psychiatry, UMC Utrecht, Utrecht, the Netherlands.'}, {'name': 'Lucija Abramovic Marc M Bohlken Marco P Boks Kristel R van Eijk Neeltje E M van Haren Rachel M Brouwer  Hilleke E Hulshoff Pol', 'org': 'Brain Center Rudolf Magnus, Department of Psychiatry, UMC Utrecht, Utrecht, the Netherlands.'}, {'name': 'Lucija Abramovic Marc M Bohlken Marco P Boks Kristel R van Eijk Neeltje E M van Haren Rachel M Brouwer  Hilleke E Hulshoff Pol', 'org': 'Brain Center Rudolf Magnus, Department of Psychiatry, UMC Utrecht, Utrecht, the Netherlands.'}, {'name': 'Lucija Abramovic Marc M Bohlken Marco P Boks Kristel R van Eijk Neeltje E M van Haren Rachel M Brouwer  Hilleke E Hulshoff Pol', 'org': 'Brain Center Rudolf Magnus, Department of Psychiatry, UMC Utrecht, Utrecht, the Netherlands.'}, {'name': 'Lucija Abramovic Marc M Bohlken Marco P Boks Kristel R van Eijk Neeltje E M van Haren Rachel M Brouwer  Hilleke E Hulshoff Pol', 'org': 'Brain Center Rudolf Magnus, Department of Psychiatry, UMC Utrecht, Utrecht, the Netherlands.'}, {'name': 'Lucija Abramovic Marc M Bohlken Marco P Boks Kristel R van Eijk Neeltje E M van Haren Rachel M Brouwer  Hilleke E Hulshoff Pol', 'org': 'Brain Center Rudolf Magnus, Department of Psychiatry, UMC Utrecht, Utrecht, the Netherlands.'}, {'name': 'Micael Andersson Alireza Salami Lars G Nilsson  Lars Nyberg', 'org': 'Umeå Centre for Functional Brain Imaging (UFBI), Umeå University, Umeå, Sweden.'}, {'name': 'Micael Andersson Alireza Salami Lars G Nilsson  Lars Nyberg', 'org': 'Umeå Centre for Functional Brain Imaging (UFBI), Umeå University, Umeå, Sweden.'}, {'name': 'Micael Andersson Alireza Salami Lars G Nilsson  Lars Nyberg', 'org': 'Umeå Centre for Functional Brain Imaging (UFBI), Umeå University, Umeå, Sweden.'}, {'name': 'Micael Andersson Alireza Salami Lars G Nilsson  Lars Nyberg', 'org': 'Umeå Centre for Functional Brain Imaging (UFBI), Umeå University, Umeå, Sweden.'}, {'name': 'Benjamin S Aribisala Natalie A Royle Mark E Bastin  Joanna M Wardlaw', 'org': 'Brain Research Imaging Centre, University of Edinburgh, Edinburgh, UK.'}, {'name': 'Benjamin S Aribisala Natalie A Royle Mark E Bastin  Joanna M Wardlaw', 'org': 'Brain Research Imaging Centre, University of Edinburgh, Edinburgh, UK.'}, {'name': 'Benjamin S Aribisala Natalie A Royle Mark E Bastin  Joanna M Wardlaw', 'org': 'Brain Research Imaging Centre, University of Edinburgh, Edinburgh, UK.'}, {'name': 'Benjamin S Aribisala Natalie A Royle Mark E Bastin  Joanna M Wardlaw', 'org': 'Brain Research Imaging Centre, University of Edinburgh, Edinburgh, UK.'}, {'name': 'Benjamin S Aribisala', 'org': 'Department of Computer Science, Lagos State University, Lagos, Nigeria.'}, {'name': 'Benjamin S Aribisala Natalie A Royle Mark E Bastin  Joanna M Wardlaw', 'org': 'Scottish Imaging Network, A Platform for Scientific Excellence (SINAPSE) Collaboration, Department of Neuroimaging Sciences, University of Edinburgh, Edinburgh, UK.'}, {'name': 'Benjamin S Aribisala Natalie A Royle Mark E Bastin  Joanna M Wardlaw', 'org': 'Scottish Imaging Network, A Platform for Scientific Excellence (SINAPSE) Collaboration, Department of Neuroimaging Sciences, University of Edinburgh, Edinburgh, UK.'}, {'name': 'Benjamin S Aribisala Natalie A Royle Mark E Bastin  Joanna M Wardlaw', 'org': 'Scottish Imaging Network, A Platform for Scientific Excellence (SINAPSE) Collaboration, Department of Neuroimaging Sciences, University of Edinburgh, Edinburgh, UK.'}, {'name': 'Benjamin S Aribisala Natalie A Royle Mark E Bastin  Joanna M Wardlaw', 'org': 'Scottish Imaging Network, A Platform for Scientific Excellence (SINAPSE) Collaboration, Department of Neuroimaging Sciences, University of Edinburgh, Edinburgh, UK.'}, {'name': 'Manon Bernard Jean Shin  Zdenka Pausova', 'org': 'Hospital for Sick Children, University of Toronto, Toronto, Ontario, Canada.'}, {'name': 'Manon Bernard Jean Shin  Zdenka Pausova', 'org': 'Hospital for Sick Children, University of Toronto, Toronto, Ontario, Canada.'}, {'name': 'Manon Bernard Jean Shin  Zdenka Pausova', 'org': 'Hospital for Sick Children, University of Toronto, Toronto, Ontario, Canada.'}, {'name': 'M Mallar Chakravarty', 'org': 'Cerebral Imaging Centre, Douglas Mental Health University Institute, Montreal, Quebec, Canada.'}, {'name': 'M Mallar Chakravarty', 'org': 'Department of Psychiatry and Biomedical Engineering, McGill University, Montreal, Quebec, Canada.'}, {'name': "Anouk den Braber Iryna O Fedko Jouke-Jan Hottenga Dennis van 't Ent Dorret I Boomsma  Eco J C de Geus", 'org': 'Biological Psychology, Neuroscience Campus Amsterdam, VU University & VU Medical Center, Amsterdam, the Netherlands.'}, {'name': "Anouk den Braber Iryna O Fedko Jouke-Jan Hottenga Dennis van 't Ent Dorret I Boomsma  Eco J C de Geus", 'org': 'Biological Psychology, Neuroscience Campus Amsterdam, VU University & VU Medical Center, Amsterdam, the Netherlands.'}, {'name': "Anouk den Braber Iryna O Fedko Jouke-Jan Hottenga Dennis van 't Ent Dorret I Boomsma  Eco J C de Geus", 'org': 'Biological Psychology, Neuroscience Campus Amsterdam, VU University & VU Medical Center, Amsterdam, the Netherlands.'}, {'name': "Anouk den Braber Iryna O Fedko Jouke-Jan Hottenga Dennis van 't Ent Dorret I Boomsma  Eco J C de Geus", 'org': 'Biological Psychology, Neuroscience Campus Amsterdam, VU University & VU Medical Center, Amsterdam, the Netherlands.'}, {'name': "Anouk den Braber Iryna O Fedko Jouke-Jan Hottenga Dennis van 't Ent Dorret I Boomsma  Eco J C de Geus", 'org': 'Biological Psychology, Neuroscience Campus Amsterdam, VU University & VU Medical Center, Amsterdam, the Netherlands.'}, {'name': "Anouk den Braber Iryna O Fedko Jouke-Jan Hottenga Dennis van 't Ent Dorret I Boomsma  Eco J C de Geus", 'org': 'Biological Psychology, Neuroscience Campus Amsterdam, VU University & VU Medical Center, Amsterdam, the Netherlands.'}, {'name': 'Sudheer Giddaluru Vidar M Steen  Stephanie LeHellard', 'org': 'NORMENT – KG Jebsen Centre for Psychosis Research, Department of Clinical Science, University of Bergen, Bergen, Norway.'}, {'name': 'Sudheer Giddaluru Vidar M Steen  Stephanie LeHellard', 'org': 'NORMENT – KG Jebsen Centre for Psychosis Research, Department of Clinical Science, University of Bergen, Bergen, Norway.'}, {'name': 'Sudheer Giddaluru Vidar M Steen  Stephanie LeHellard', 'org': 'NORMENT – KG Jebsen Centre for Psychosis Research, Department of Clinical Science, University of Bergen, Bergen, Norway.'}, {'name': 'Sudheer Giddaluru Vidar M Steen  Stephanie LeHellard', 'org': 'Dr Einar Martens Research Group for Biological Psychiatry, Center for Medical Genetics and Molecular Medicine, Haukeland University Hospital, Bergen, Norway.'}, {'name': 'Sudheer Giddaluru Vidar M Steen  Stephanie LeHellard', 'org': 'Dr Einar Martens Research Group for Biological Psychiatry, Center for Medical Genetics and Molecular Medicine, Haukeland University Hospital, Bergen, Norway.'}, {'name': 'Sudheer Giddaluru Vidar M Steen  Stephanie LeHellard', 'org': 'Dr Einar Martens Research Group for Biological Psychiatry, Center for Medical Genetics and Molecular Medicine, Haukeland University Hospital, Bergen, Norway.'}, {'name': 'Tulio Guadalupe', 'org': 'Language and Genetics Department, Max Planck Institute for Psycholinguistics, Nijmegen, the Netherlands.'}, {'name': 'Tulio Guadalupe', 'org': 'International Max Planck Research School for Language Sciences, Nijmegen, the Netherlands.'}, {'name': 'Girma Woldehawariat Allison C Nugent Xinmin Liu Wayne C Drevets  Dara M Cannon', 'org': 'National Institute of Mental Health Intramural Research Program, Bethesda, Maryland, USA.'}, {'name': 'Girma Woldehawariat Allison C Nugent Xinmin Liu Wayne C Drevets  Dara M Cannon', 'org': 'National Institute of Mental Health Intramural Research Program, Bethesda, Maryland, USA.'}, {'name': 'Girma Woldehawariat Allison C Nugent Xinmin Liu Wayne C Drevets  Dara M Cannon', 'org': 'National Institute of Mental Health Intramural Research Program, Bethesda, Maryland, USA.'}, {'name': 'Girma Woldehawariat Allison C Nugent Xinmin Liu Wayne C Drevets  Dara M Cannon', 'org': 'National Institute of Mental Health Intramural Research Program, Bethesda, Maryland, USA.'}, {'name': 'Girma Woldehawariat Allison C Nugent Xinmin Liu Wayne C Drevets  Dara M Cannon', 'org': 'National Institute of Mental Health Intramural Research Program, Bethesda, Maryland, USA.'}, {'name': 'Loes M Olde Loohuis', 'org': 'Center for Neurobehavioral Genetics, University of California, Los Angeles, California, USA.'}, {'name': 'Michelle Luciano David C M Liewald Lorna M Lopez Natalie A Royle Mark E Bastin Ian J Deary  Joanna M Wardlaw', 'org': 'Centre for Cognitive Ageing and Cognitive Epidemiology, Psychology, University of Edinburgh, Edinburgh, UK.'}, {'name': 'Michelle Luciano David C M Liewald Lorna M Lopez Natalie A Royle Mark E Bastin Ian J Deary  Joanna M Wardlaw', 'org': 'Centre for Cognitive Ageing and Cognitive Epidemiology, Psychology, University of Edinburgh, Edinburgh, UK.'}, {'name': 'Michelle Luciano David C M Liewald Lorna M Lopez Natalie A Royle Mark E Bastin Ian J Deary  Joanna M Wardlaw', 'org': 'Centre for Cognitive Ageing and Cognitive Epidemiology, Psychology, University of Edinburgh, Edinburgh, UK.'}, {'name': 'Michelle Luciano David C M Liewald Lorna M Lopez Natalie A Royle Mark E Bastin Ian J Deary  Joanna M Wardlaw', 'org': 'Centre for Cognitive Ageing and Cognitive Epidemiology, Psychology, University of Edinburgh, Edinburgh, UK.'}, {'name': 'Michelle Luciano David C M Liewald Lorna M Lopez Natalie A Royle Mark E Bastin Ian J Deary  Joanna M Wardlaw', 'org': 'Centre for Cognitive Ageing and Cognitive Epidemiology, Psychology, University of Edinburgh, Edinburgh, UK.'}, {'name': 'Michelle Luciano David C M Liewald Lorna M Lopez Natalie A Royle Mark E Bastin Ian J Deary  Joanna M Wardlaw', 'org': 'Centre for Cognitive Ageing and Cognitive Epidemiology, Psychology, University of Edinburgh, Edinburgh, UK.'}, {'name': 'Michelle Luciano David C M Liewald Lorna M Lopez Natalie A Royle Mark E Bastin Ian J Deary  Joanna M Wardlaw', 'org': 'Centre for Cognitive Ageing and Cognitive Epidemiology, Psychology, University of Edinburgh, Edinburgh, UK.'}, {'name': 'Yuri Milaneschi Lianne Schmaal Dick J Veltman  Brenda W J H Penninx', 'org': 'Department of Psychiatry, Neuroscience Campus Amsterdam, VU University Medical Center, Amsterdam, the Netherlands.'}, {'name': 'Yuri Milaneschi Lianne Schmaal Dick J Veltman  Brenda W J H Penninx', 'org': 'Department of Psychiatry, Neuroscience Campus Amsterdam, VU University Medical Center, Amsterdam, the Netherlands.'}, {'name': 'Yuri Milaneschi Lianne Schmaal Dick J Veltman  Brenda W J H Penninx', 'org': 'Department of Psychiatry, Neuroscience Campus Amsterdam, VU University Medical Center, Amsterdam, the Netherlands.'}, {'name': 'Yuri Milaneschi Lianne Schmaal Dick J Veltman  Brenda W J H Penninx', 'org': 'Department of Psychiatry, Neuroscience Campus Amsterdam, VU University Medical Center, Amsterdam, the Netherlands.'}, {'name': 'Adaikalavan Ramasamy Mina Ryten  Michael E Weale', 'org': "Department of Medical and Molecular Genetics, King's College London, London, UK."}, {'name': 'Adaikalavan Ramasamy Mina Ryten  Michael E Weale', 'org': "Department of Medical and Molecular Genetics, King's College London, London, UK."}, {'name': 'Adaikalavan Ramasamy Mina Ryten  Michael E Weale', 'org': "Department of Medical and Molecular Genetics, King's College London, London, UK."}, {'name': 'Adaikalavan Ramasamy Daniah Trabzuni Raphael Gibbs Sebastian Guelfi Dena G Hernandez John Hardy  Mina Ryten', 'org': 'Reta Lila Weston Institute and Department of Molecular Neuroscience, UCL Institute of Neurology, London, UK.'}, {'name': 'Adaikalavan Ramasamy Daniah Trabzuni Raphael Gibbs Sebastian Guelfi Dena G Hernandez John Hardy  Mina Ryten', 'org': 'Reta Lila Weston Institute and Department of Molecular Neuroscience, UCL Institute of Neurology, London, UK.'}, {'name': 'Adaikalavan Ramasamy Daniah Trabzuni Raphael Gibbs Sebastian Guelfi Dena G Hernandez John Hardy  Mina Ryten', 'org': 'Reta Lila Weston Institute and Department of Molecular Neuroscience, UCL Institute of Neurology, London, UK.'}, {'name': 'Adaikalavan Ramasamy Daniah Trabzuni Raphael Gibbs Sebastian Guelfi Dena G Hernandez John Hardy  Mina Ryten', 'org': 'Reta Lila Weston Institute and Department of Molecular Neuroscience, UCL Institute of Neurology, London, UK.'}, {'name': 'Adaikalavan Ramasamy Daniah Trabzuni Raphael Gibbs Sebastian Guelfi Dena G Hernandez John Hardy  Mina Ryten', 'org': 'Reta Lila Weston Institute and Department of Molecular Neuroscience, UCL Institute of Neurology, London, UK.'}, {'name': 'Adaikalavan Ramasamy Daniah Trabzuni Raphael Gibbs Sebastian Guelfi Dena G Hernandez John Hardy  Mina Ryten', 'org': 'Reta Lila Weston Institute and Department of Molecular Neuroscience, UCL Institute of Neurology, London, UK.'}, {'name': 'Adaikalavan Ramasamy Daniah Trabzuni Raphael Gibbs Sebastian Guelfi Dena G Hernandez John Hardy  Mina Ryten', 'org': 'Reta Lila Weston Institute and Department of Molecular Neuroscience, UCL Institute of Neurology, London, UK.'}, {'name': 'Philipp G Sämann David Höhn Benno Pütz Christiane Wolf Michael Czisch  Florian Holsboer', 'org': 'Max Planck Institute of Psychiatry, Munich, Germany.'}, {'name': 'Philipp G Sämann David Höhn Benno Pütz Christiane Wolf Michael Czisch  Florian Holsboer', 'org': 'Max Planck Institute of Psychiatry, Munich, Germany.'}, {'name': 'Philipp G Sämann David Höhn Benno Pütz Christiane Wolf Michael Czisch  Florian Holsboer', 'org': 'Max Planck Institute of Psychiatry, Munich, Germany.'}, {'name': 'Philipp G Sämann David Höhn Benno Pütz Christiane Wolf Michael Czisch  Florian Holsboer', 'org': 'Max Planck Institute of Psychiatry, Munich, Germany.'}, {'name': 'Philipp G Sämann David Höhn Benno Pütz Christiane Wolf Michael Czisch  Florian Holsboer', 'org': 'Max Planck Institute of Psychiatry, Munich, Germany.'}, {'name': 'Philipp G Sämann David Höhn Benno Pütz Christiane Wolf Michael Czisch  Florian Holsboer', 'org': 'Max Planck Institute of Psychiatry, Munich, Germany.'}, {'name': 'Andrew J Schork', 'org': 'Multimodal Imaging Laboratory, Department of Neurosciences, University of California, San Diego, California, USA.'}, {'name': 'Andrew J Schork', 'org': 'Department of Cognitive Sciences, University of California, San Diego, California, USA.'}, {'name': 'Alexander Teumer Wolfgang Hoffmann  Henry Völzke', 'org': 'Institute for Community Medicine, University Medicine Greifswald, Greifswald, Germany.'}, {'name': 'Alexander Teumer Wolfgang Hoffmann  Henry Völzke', 'org': 'Institute for Community Medicine, University Medicine Greifswald, Greifswald, Germany.'}, {'name': 'Alexander Teumer Wolfgang Hoffmann  Henry Völzke', 'org': 'Institute for Community Medicine, University Medicine Greifswald, Greifswald, Germany.'}, {'name': 'Lars T Westlye  Thomas Espeseth', 'org': 'NORMENT – KG Jebsen Centre, Division of Mental Health and Addiction, Oslo University Hospital, Oslo, Norway.'}, {'name': 'Lars T Westlye  Thomas Espeseth', 'org': 'NORMENT – KG Jebsen Centre, Division of Mental Health and Addiction, Oslo University Hospital, Oslo, Norway.'}, {'name': 'Lars T Westlye Ivar Reinvang  Thomas Espeseth', 'org': 'NORMENT – KG Jebsen Centre, Department of Psychology, University of Oslo, Oslo, Norway.'}, {'name': 'Lars T Westlye Ivar Reinvang  Thomas Espeseth', 'org': 'NORMENT – KG Jebsen Centre, Department of Psychology, University of Oslo, Oslo, Norway.'}, {'name': 'Lars T Westlye Ivar Reinvang  Thomas Espeseth', 'org': 'NORMENT – KG Jebsen Centre, Department of Psychology, University of Oslo, Oslo, Norway.'}, {'name': 'Christopher D Whelan Saud Alhusaini Gianpiero L Cavalleri  Norman Delanty', 'org': 'Molecular and Cellular Therapeutics, The Royal College of Surgeons, Dublin, 2, Ireland.'}, {'name': 'Christopher D Whelan Saud Alhusaini Gianpiero L Cavalleri  Norman Delanty', 'org': 'Molecular and Cellular Therapeutics, The Royal College of Surgeons, Dublin, 2, Ireland.'}, {'name': 'Christopher D Whelan Saud Alhusaini Gianpiero L Cavalleri  Norman Delanty', 'org': 'Molecular and Cellular Therapeutics, The Royal College of Surgeons, Dublin, 2, Ireland.'}, {'name': 'Christopher D Whelan Saud Alhusaini Gianpiero L Cavalleri  Norman Delanty', 'org': 'Molecular and Cellular Therapeutics, The Royal College of Surgeons, Dublin, 2, Ireland.'}, {'name': 'Anderson M Winkler', 'org': 'The Oxford Center for Functional MRI of the Brain, Nuffield Department of Clinical Neurosciences, Oxford University, Oxford, UK.'}, {'name': 'Saud Alhusaini', 'org': 'Department of Neurology and Neurosurgery, Montreal Neurological Institute, McGill University, Montreal, Quebec, Canada.'}, {'name': 'Dalia Kasperaviciute Mar Matarin  Sanjay M Sisodiya', 'org': 'UCL Institute of Neurology, University College London, and Epilepsy Society, London, UK.'}, {'name': 'Dalia Kasperaviciute Mar Matarin  Sanjay M Sisodiya', 'org': 'UCL Institute of Neurology, University College London, and Epilepsy Society, London, UK.'}, {'name': 'Dalia Kasperaviciute Mar Matarin  Sanjay M Sisodiya', 'org': 'UCL Institute of Neurology, University College London, and Epilepsy Society, London, UK.'}, {'name': 'Dalia Kasperaviciute', 'org': 'Department of Medicine, Imperial College London, London, UK.'}, {'name': 'David R McKay  David C Glahn', 'org': 'Department of Psychiatry, Yale University, New Haven, Connecticut, USA.'}, {'name': 'David R McKay  David C Glahn', 'org': 'Department of Psychiatry, Yale University, New Haven, Connecticut, USA.'}, {'name': 'David R McKay  David C Glahn', 'org': 'Olin Neuropsychiatric Research Center, Hartford, Connecticut, USA.'}, {'name': 'David R McKay  David C Glahn', 'org': 'Olin Neuropsychiatric Research Center, Hartford, Connecticut, USA.'}, {'name': 'Daniah Trabzuni', 'org': 'Department of Genetics, King Faisal Specialist Hospital and Research Centre, Riyadh, Saudi Arabia.'}, {'name': 'Laura Almasy Melanie A Carless Joanne E Curran Ravi Duggirala Thomas D Dyer Harald H H Göring Jack W Kent Jr  John Blangero', 'org': 'Texas Biomedical Research Institute, San Antonio, Texas, USA.'}, {'name': 'Laura Almasy Melanie A Carless Joanne E Curran Ravi Duggirala Thomas D Dyer Harald H H Göring Jack W Kent Jr  John Blangero', 'org': 'Texas Biomedical Research Institute, San Antonio, Texas, USA.'}, {'name': 'Laura Almasy Melanie A Carless Joanne E Curran Ravi Duggirala Thomas D Dyer Harald H H Göring Jack W Kent Jr  John Blangero', 'org': 'Texas Biomedical Research Institute, San Antonio, Texas, USA.'}, {'name': 'Laura Almasy Melanie A Carless Joanne E Curran Ravi Duggirala Thomas D Dyer Harald H H Göring Jack W Kent Jr  John Blangero', 'org': 'Texas Biomedical Research Institute, San Antonio, Texas, USA.'}, {'name': 'Laura Almasy Melanie A Carless Joanne E Curran Ravi Duggirala Thomas D Dyer Harald H H Göring Jack W Kent Jr  John Blangero', 'org': 'Texas Biomedical Research Institute, San Antonio, Texas, USA.'}, {'name': 'Laura Almasy Melanie A Carless Joanne E Curran Ravi Duggirala Thomas D Dyer Harald H H Göring Jack W Kent Jr  John Blangero', 'org': 'Texas Biomedical Research Institute, San Antonio, Texas, USA.'}, {'name': 'Laura Almasy Melanie A Carless Joanne E Curran Ravi Duggirala Thomas D Dyer Harald H H Göring Jack W Kent Jr  John Blangero', 'org': 'Texas Biomedical Research Institute, San Antonio, Texas, USA.'}, {'name': 'Laura Almasy Melanie A Carless Joanne E Curran Ravi Duggirala Thomas D Dyer Harald H H Göring Jack W Kent Jr  John Blangero', 'org': 'Texas Biomedical Research Institute, San Antonio, Texas, USA.'}, {'name': 'David Ames', 'org': 'National Ageing Research Institute, Royal Melbourne Hospital, Melbourne, Victoria, Australia.'}, {'name': 'David Ames', 'org': 'Academic Unit for Psychiatry of Old Age, University of Melbourne, Melbourne, Victoria, Australia.'}, {'name': 'Sampath Arepalli Allissa Dillman Raphael Gibbs Dena G Hernandez Michael A Nalls Bryan Traynor Mark R Cookson  Andrew Singleton', 'org': 'Laboratory of Neurogenetics, National Institute on Aging, National Institutes of Health, Bethesda, Maryland, USA.'}, {'name': 'Sampath Arepalli Allissa Dillman Raphael Gibbs Dena G Hernandez Michael A Nalls Bryan Traynor Mark R Cookson  Andrew Singleton', 'org': 'Laboratory of Neurogenetics, National Institute on Aging, National Institutes of Health, Bethesda, Maryland, USA.'}, {'name': 'Sampath Arepalli Allissa Dillman Raphael Gibbs Dena G Hernandez Michael A Nalls Bryan Traynor Mark R Cookson  Andrew Singleton', 'org': 'Laboratory of Neurogenetics, National Institute on Aging, National Institutes of Health, Bethesda, Maryland, USA.'}, {'name': 'Sampath Arepalli Allissa Dillman Raphael Gibbs Dena G Hernandez Michael A Nalls Bryan Traynor Mark R Cookson  Andrew Singleton', 'org': 'Laboratory of Neurogenetics, National Institute on Aging, National Institutes of Health, Bethesda, Maryland, USA.'}, {'name': 'Sampath Arepalli Allissa Dillman Raphael Gibbs Dena G Hernandez Michael A Nalls Bryan Traynor Mark R Cookson  Andrew Singleton', 'org': 'Laboratory of Neurogenetics, National Institute on Aging, National Institutes of Health, Bethesda, Maryland, USA.'}, {'name': 'Sampath Arepalli Allissa Dillman Raphael Gibbs Dena G Hernandez Michael A Nalls Bryan Traynor Mark R Cookson  Andrew Singleton', 'org': 'Laboratory of Neurogenetics, National Institute on Aging, National Institutes of Health, Bethesda, Maryland, USA.'}, {'name': 'Sampath Arepalli Allissa Dillman Raphael Gibbs Dena G Hernandez Michael A Nalls Bryan Traynor Mark R Cookson  Andrew Singleton', 'org': 'Laboratory of Neurogenetics, National Institute on Aging, National Institutes of Health, Bethesda, Maryland, USA.'}, {'name': 'Sampath Arepalli Allissa Dillman Raphael Gibbs Dena G Hernandez Michael A Nalls Bryan Traynor Mark R Cookson  Andrew Singleton', 'org': 'Laboratory of Neurogenetics, National Institute on Aging, National Institutes of Health, Bethesda, Maryland, USA.'}, {'name': 'Mark E Bastin Maria C Valdés Hernández  Joanna M Wardlaw', 'org': 'Centre for Clinical Brain Sciences, University of Edinburgh, Edinburgh, UK.'}, {'name': 'Mark E Bastin Maria C Valdés Hernández  Joanna M Wardlaw', 'org': 'Centre for Clinical Brain Sciences, University of Edinburgh, Edinburgh, UK.'}, {'name': 'Mark E Bastin Maria C Valdés Hernández  Joanna M Wardlaw', 'org': 'Centre for Clinical Brain Sciences, University of Edinburgh, Edinburgh, UK.'}, {'name': 'Kazima B Bulayeva', 'org': 'NI Vavilov Institute of General Genetics, Russian Academy of Sciences, Moscow, Russia.'}, {'name': 'Luigi Ferrucci', 'org': 'Clinical Research Branch, National Institute on Aging, Baltimore, Maryland, USA.'}, {'name': 'Peter T Fox  Rene L Olvera', 'org': 'University of Texas Health Science Center, San Antonio, USA.'}, {'name': 'Peter T Fox  Rene L Olvera', 'org': 'University of Texas Health Science Center, San Antonio, USA.'}, {'name': 'Masaki Fukunaga', 'org': 'Biofunctional Imaging, Immunology Frontier Research Center, Osaka University, Osaka, Japan.'}, {'name': 'Robert C Green', 'org': 'Harvard Medical School, Cambridge, Massachusetts, USA.'}, {'name': 'Robert C Green', 'org': "Division of Genetics, Department of Medicine, Brigham and Women's Hospital, Boston, Massachusetts, USA."}, {'name': 'Catharina A Hartman  Pieter J Hoekstra', 'org': 'Department of Psychiatry, University of Groningen, University Medical Center Groningen, Groningen, the Netherlands.'}, {'name': 'Catharina A Hartman  Pieter J Hoekstra', 'org': 'Department of Psychiatry, University of Groningen, University Medical Center Groningen, Groningen, the Netherlands.'}, {'name': 'Katrin Hegenscheid  Norbert Hosten', 'org': 'Institute of Diagnostic Radiology and Neuroradiology, University Medicine Greifswald, Greifswald, Germany.'}, {'name': 'Katrin Hegenscheid  Norbert Hosten', 'org': 'Institute of Diagnostic Radiology and Neuroradiology, University Medicine Greifswald, Greifswald, Germany.'}, {'name': 'Dirk J Heslenfeld', 'org': 'Department of Psychology, VU University Amsterdam, Amsterdam, the Netherlands.'}, {'name': 'Georg Homuth', 'org': 'Interfaculty Institute for Genetics and Functional Genomics, University Medicine Greifswald, Greifswald, Germany.'}, {'name': 'Clifford R Jack Jr', 'org': 'Radiology, Mayo Clinic, Rochester, Minnesota, USA.'}, {'name': 'Mark Jenkinson', 'org': 'FMRIB Centre, University of Oxford, Oxford, UK.'}, {'name': 'Robert Johnson  Ronald H Zielke', 'org': 'NICHD Brain and Tissue Bank for Developmental Disorders, University of Maryland Medical School, Baltimore, Maryland, USA.'}, {'name': 'Robert Johnson  Ronald H Zielke', 'org': 'NICHD Brain and Tissue Bank for Developmental Disorders, University of Maryland Medical School, Baltimore, Maryland, USA.'}, {'name': 'Ryota Kanai', 'org': 'School of Psychology, University of Sussex, Brighton, UK.'}, {'name': 'Ryota Kanai', 'org': 'Institute of Cognitive Neuroscience, University College London, London, UK.'}, {'name': 'Peter Kochunov', 'org': 'Department of Psychiatry, University of Maryland, Catonsville, Maryland, USA.'}, {'name': 'John B Kwok  Peter R Schofield', 'org': 'Neuroscience Research Australia, Sydney, New South Wales, Australia.'}, {'name': 'John B Kwok  Peter R Schofield', 'org': 'Neuroscience Research Australia, Sydney, New South Wales, Australia.'}, {'name': 'John B Kwok  Peter R Schofield', 'org': 'School of Medical Sciences, University of New South Wales, Sydney, New South Wales, Australia.'}, {'name': 'John B Kwok  Peter R Schofield', 'org': 'School of Medical Sciences, University of New South Wales, Sydney, New South Wales, Australia.'}, {'name': 'Xinmin Liu', 'org': 'Columbia University Medical Center, New York, USA.'}, {'name': 'Dan L Longo', 'org': 'Lymphocyte Cell Biology Unit, Laboratory of Immunology, National Institute on Aging, National Institutes of Health, Baltimore, Maryland, USA.'}, {'name': 'Katie L McMahon', 'org': 'Centre for Advanced Imaging, University of Queensland, Brisbane, Queensland, Australia.'}, {'name': 'Eva Meisenzahl', 'org': 'Ludwig-Maximilians-Universität, Munich, Germany.'}, {'name': 'Kazutaka Ohi', 'org': 'Department of Psychiatry, Osaka University Graduate School of Medicine, Osaka, Japan.'}, {'name': 'G Bruce Pike', 'org': 'Department of Neurology, University of Calgary, Calgary, Canada.'}, {'name': 'G Bruce Pike', 'org': 'Department of Clinical Neuroscience, University of Calgary, Calgary, Canada.'}, {'name': 'Steven G Potkin', 'org': 'Psychiatry and Human Behavior, University of California, Irvine, California, USA.'}, {'name': 'Glenn D Rosen', 'org': 'Beth Israel Deaconess Medical Center, Boston, Massachusetts, USA.'}, {'name': 'Glenn D Rosen', 'org': 'Department of Neurology, Harvard Medical School, Boston, Massachusetts, USA.'}, {'name': 'Colin Smith', 'org': 'Department of Neuropathology, MRC Sudden Death Brain Bank Project, University of Edinburgh, Edinburgh, UK.'}, {'name': 'Arthur W Toga', 'org': 'Laboratory of Neuro Imaging, Institute for Neuroimaging and Informatics, Keck School of Medicine of the University of Southern California, Los Angeles, California, USA.'}, {'name': 'Juan Troncoso', 'org': 'Brain Resource Center, Johns Hopkins University, Baltimore, Maryland, USA.'}, {'name': 'Marcel van der Brug', 'org': 'The Scripps Research Institute, Jupiter, Florida, USA.'}, {'name': 'Nic J A van der Wee', 'org': 'Leiden University Medical Center, Leiden, the Netherlands.'}, {'name': 'Marie-Jose van Tol', 'org': 'Neuroimaging Centre, University of Groningen, University Medical Center Groningen, Groningen, the Netherlands.'}, {'name': 'Thomas H Wassink', 'org': 'Department of Psychiatry, Carver College of Medicine, University of Iowa, Iowa City, lowa, USA.'}, {'name': 'Eric Westman', 'org': 'Department of Neurobiology, Care Sciences and Society, Karolinska Institutet, Stockholm, Sweden.'}, {'name': 'Alan Zonderman', 'org': 'Research Resources Branch, National Institute on Aging, National Institutes of Health, Bethesda, Maryland, USA.'}, {'name': 'David G Ashbrook  Reinmar Hager', 'org': 'Faculty of Life Sciences, University of Manchester, Manchester, UK.'}, {'name': 'David G Ashbrook  Reinmar Hager', 'org': 'Faculty of Life Sciences, University of Manchester, Manchester, UK.'}, {'name': 'Lu Lu  Robert W Williams', 'org': 'Center for Integrative and Translational Genomics, University of Tennessee Health Science Center, Memphis, Tennessee, USA.'}, {'name': 'Lu Lu  Robert W Williams', 'org': 'Center for Integrative and Translational Genomics, University of Tennessee Health Science Center, Memphis, Tennessee, USA.'}, {'name': 'Lu Lu  Robert W Williams', 'org': 'Department of Anatomy and Neurobiology, University of Tennessee Health Science Center, Memphis, Tennessee, USA.'}, {'name': 'Lu Lu  Robert W Williams', 'org': 'Department of Anatomy and Neurobiology, University of Tennessee Health Science Center, Memphis, Tennessee, USA.'}, {'name': 'Vince D Calhoun', 'org': 'The Mind Research Network & LBERI, Albuquerque, New Mexico, USA.'}, {'name': 'Vince D Calhoun', 'org': 'Department of ECE, University of New Mexico, Albuquerque, New Mexico, USA.'}, {'name': 'Anders M Dale', 'org': 'Center for Translational Imaging and Personalized Medicine, University of California, San Diego, California, USA.'}, {'name': 'Anders M Dale', 'org': 'Departments of Neurosciences, Radiology, Psychiatry, and Cognitive Science, University of California, San Diego, California, USA.'}, {'name': 'Gareth E Davies', 'org': 'Avera Institute for Human Genetics, Sioux Falls, South Dakota, USA.'}, {'name': 'Norman Delanty', 'org': 'Neurology Division, Beaumont Hospital, Dublin, Ireland.'}, {'name': 'Chantal Depondt  Massimo Pandolfo', 'org': 'Department of Neurology, Hopital Erasme, Universite Libre de Bruxelles, Brussels, Belgium.'}, {'name': 'Chantal Depondt  Massimo Pandolfo', 'org': 'Department of Neurology, Hopital Erasme, Universite Libre de Bruxelles, Brussels, Belgium.'}, {'name': 'Wayne C Drevets', 'org': 'Janssen Research & Development, Johnson & Johnson, Raritan, New Jersey, USA.'}, {'name': 'Beng-Choon Ho', 'org': 'Department of Psychiatry, University of Iowa, Iowa City. lowa, USA.'}, {'name': 'Matthias Nauck', 'org': 'Institute of Clinical Chemistry and Laboratory Medicine, University Medicine Greifswald, Greifswald, Germany.'}, {'name': 'Michael W Weiner', 'org': 'Center for Imaging of Neurodegenerative Disease, San Francisco Veterans Affairs Medical Center, University of California, San Francisco, USA.'}, {'name': 'Tonya White', 'org': 'Department of Child Psychiatry, Erasmus University Medical Centre, Rotterdam, the Netherlands.'}, {'name': 'Tonya White Hieab H H Adams  M Arfan Ikram', 'org': 'Department of Radiology, Erasmus University Medical Centre, Rotterdam, the Netherlands.'}, {'name': 'Tonya White Hieab H H Adams  M Arfan Ikram', 'org': 'Department of Radiology, Erasmus University Medical Centre, Rotterdam, the Netherlands.'}, {'name': 'Tonya White Hieab H H Adams  M Arfan Ikram', 'org': 'Department of Radiology, Erasmus University Medical Centre, Rotterdam, the Netherlands.'}, {'name': 'Dara M Cannon', 'org': 'Clinical Neuroimaging Laboratory, College of Medicine, Nursing and Health Sciences, National University of Ireland Galway, Galway, Ireland.'}, {'name': 'Hans J Grabe', 'org': 'Department of Psychiatry and Psychotherapy, HELIOS Hospital Stralsund, Stralsund, Germany.'}, {'name': 'Ryota Hashimoto', 'org': "Molecular Research Center for Children's Mental Development, United Graduate School of Child Development, Osaka University, Osaka, Japan."}, {'name': 'Iwona Kloszewska', 'org': 'Medical University of Lodz, Lodz, Poland.'}, {'name': 'Patrizia Mecocci', 'org': 'Section of Gerontology and Geriatrics, Department of Medicine, University of Perugia, Perugia, Italy.'}, {'name': 'Tomas Paus', 'org': 'Rotman Research Institute, University of Toronto, Toronto, Ontario, Canada.'}, {'name': 'Tomas Paus', 'org': 'Departments of Psychology and Psychiatry, University of Toronto, Ontario, Canada.'}, {'name': 'Zdenka Pausova', 'org': 'Departments of Physiology and Nutritional Sciences, University of Toronto, Ontario, Canada.'}, {'name': 'Andy Simmons', 'org': "Department of Neuroimaging, Institute of Psychiatry, King's College London, London, UK."}, {'name': 'Andy Simmons', 'org': "Biomedical Research Centre for Mental Health, King's College London, London, UK."}, {'name': 'Andy Simmons', 'org': "Biomedical Research Unit for Dementia, King's College London, London, UK."}, {'name': 'Hilkka Soininen', 'org': 'Institute of Clinical Medicine, Neurology, University of Eastern Finland, Kuopio, Finland.'}, {'name': 'Hilkka Soininen', 'org': 'Neurocentre Neurology, Kuopio University Hospital, Kuopio, Finland.'}, {'name': 'Hieab H H Adams Sven J van der Lee Cornelia M van Duijn  M Arfan Ikram', 'org': 'Department of Epidemiology, Erasmus University Medical Centre, Rotterdam, the Netherlands.'}, {'name': 'Hieab H H Adams Sven J van der Lee Cornelia M van Duijn  M Arfan Ikram', 'org': 'Department of Epidemiology, Erasmus University Medical Centre, Rotterdam, the Netherlands.'}, {'name': 'Hieab H H Adams Sven J van der Lee Cornelia M van Duijn  M Arfan Ikram', 'org': 'Department of Epidemiology, Erasmus University Medical Centre, Rotterdam, the Netherlands.'}, {'name': 'Hieab H H Adams Sven J van der Lee Cornelia M van Duijn  M Arfan Ikram', 'org': 'Department of Epidemiology, Erasmus University Medical Centre, Rotterdam, the Netherlands.'}, {'name': 'Lenore J Launer', 'org': 'Laboratory of Epidemiology and Population Sciences, Intramural Research Program, National Institute on Aging, Bethesda, Maryland, USA.'}, {'name': 'Stephan Seiler  Reinhold Schmidt', 'org': 'Department of Neurology, Clinical Division of Neurogeriatrics, Medical University Graz, Graz, Austria.'}, {'name': 'Stephan Seiler  Reinhold Schmidt', 'org': 'Department of Neurology, Clinical Division of Neurogeriatrics, Medical University Graz, Graz, Austria.'}, {'name': 'Ganesh Chauhan', 'org': 'INSERM U897, University of Bordeaux, Bordeaux, France.'}, {'name': 'Claudia L Satizabal  Sudha Seshadri', 'org': 'Department of Neurology, Boston University School of Medicine, Boston, Massachusetts, USA.'}, {'name': 'Claudia L Satizabal  Sudha Seshadri', 'org': 'Department of Neurology, Boston University School of Medicine, Boston, Massachusetts, USA.'}, {'name': 'Claudia L Satizabal  Sudha Seshadri', 'org': 'Framingham Heart Study, Framingham, Massachusetts, USA.'}, {'name': 'Claudia L Satizabal  Sudha Seshadri', 'org': 'Framingham Heart Study, Framingham, Massachusetts, USA.'}, {'name': 'James T Becker', 'org': 'Department of Neurology, School of Medicine, University of Pittsburgh, Pittsburgh, Pennsylvania, USA.'}, {'name': 'James T Becker', 'org': 'Department of Psychiatry, School of Medicine, University of Pittsburgh, Pittsburgh, Pennsylvania, USA.'}, {'name': 'James T Becker', 'org': 'Department of Psychology, School of Medicine, University of Pittsburgh, Pittsburgh, Pennsylvania, USA.'}, {'name': 'Lisa Yanek', 'org': 'General Internal Medicine, Johns Hopkins School of Medicine, Baltimore, Maryland, USA.'}, {'name': 'Maritza Ebling Bruce Fischl Douglas Greve  Louis N Vinke', 'org': 'Martinos Center for Biomedical Imaging, Massachusetts General Hospital, Charlestown, Massachusetts, USA.'}, {'name': 'Maritza Ebling Bruce Fischl Douglas Greve  Louis N Vinke', 'org': 'Martinos Center for Biomedical Imaging, Massachusetts General Hospital, Charlestown, Massachusetts, USA.'}, {'name': 'Maritza Ebling Bruce Fischl Douglas Greve  Louis N Vinke', 'org': 'Martinos Center for Biomedical Imaging, Massachusetts General Hospital, Charlestown, Massachusetts, USA.'}, {'name': 'Maritza Ebling Bruce Fischl Douglas Greve  Louis N Vinke', 'org': 'Martinos Center for Biomedical Imaging, Massachusetts General Hospital, Charlestown, Massachusetts, USA.'}, {'name': 'Maritza Ebling Bruce Fischl Douglas Greve  Louis N Vinke', 'org': 'Department of Radiology, Massachusetts General Hospital, Harvard Medical School, Boston, Massachusetts, USA.'}, {'name': 'Maritza Ebling Bruce Fischl Douglas Greve  Louis N Vinke', 'org': 'Department of Radiology, Massachusetts General Hospital, Harvard Medical School, Boston, Massachusetts, USA.'}, {'name': 'Maritza Ebling Bruce Fischl Douglas Greve  Louis N Vinke', 'org': 'Department of Radiology, Massachusetts General Hospital, Harvard Medical School, Boston, Massachusetts, USA.'}, {'name': 'Maritza Ebling Bruce Fischl Douglas Greve  Louis N Vinke', 'org': 'Department of Radiology, Massachusetts General Hospital, Harvard Medical School, Boston, Massachusetts, USA.'}, {'name': 'W T Longstreth', 'org': 'Department of Neurology University of Washington, Seattle, Washington, USA.'}, {'name': 'Helena Schmidt', 'org': 'Institute of Molecular Biology and Biochemistry, Medical University Graz, Graz, Austria.'}, {'name': 'Paul Nyquist', 'org': 'Department of Neurology, Johns Hopkins University School of Medicine, Baltimore, Maryland, USA.'}, {'name': 'Bernard Mazoyer', 'org': "UMR5296, Centre d'études scientifiques et techniques d'Aquitaine and University of Bordeaux, Bordeaux, France."}, {'name': 'Joshua C Bis', 'org': 'Cardiovascular Health Research Unit, Department of Medicine, University of Washington, Seattle, Washington, USA.'}, {'name': 'Vilmundur Gudnason', 'org': 'Icelandic Heart Association, Kopavogur, University of Iceland, Faculty of Medicine, Reykjavik, Iceland.'}], 'venue': 'nature neuroscience', 'keywords': ['Genetic markers', 'Magnetic resonance imaging', 'Schizophrenia', 'Genome-wide association studies'], 'year': 2016}]

[{'id': 'KDgsDUXr',
  'title': 'Genetic influences on schizophrenia and subcortical brain volumes: large-scale proof of concept',
  'authors': [{'name': 'Barbara Franke Kimm J E van Hulzen Alejandro Arias-Vasquez Janita Bralten Martine Hoogman Marieke Klein Marjolein M J van Donkelaar Marina M H Hakobjan Angelien J G A M Heister Remco R R Makkinje Marlies A M Naber Saskia S L van der Marel Jeanette C Mostert Han G Brunner  Hans van Bokhoven',
    'org': 'Department of Human Genetics, Radboud University Medical Center, Nijmegen, the Netherlands.'},
   {'name': 'Barbara Franke Kimm J E van Hulzen Alejandro Arias-Vasquez Janita Bralten Martine Hoogman Marieke Klein Marjolein M J van Donkelaar Marina M H Hakobjan Angelien J G A M Heister Remco R R Makkinje Marlies A M Naber Saskia S L van der Marel Jeanette C Mostert Han G Brunner  Hans van Bokhoven',
    'org': 'Department of Human Genetics, Radboud University Medical Center, Nijmegen, the Netherlands.'},
   {'name': 'Barbara Franke Kimm 

In [38]:
with open(path + '/na_v1_pub.json') as f:
    parser = ijson.parse(f)
    for x, y, z in parser:
        if y == 'map_key': break
    objects = ijson.items(parser, z)
    a = {z : objects.__next__()}
    print (x,y,z)
    for x, y, z in parser:
        if y == 'map_key': break
    objects = ijson.items(parser, z)
    b = {z : objects.__next__()}

 map_key 56PyrvLA


In [117]:
isinstance(None, pd.DataFrame)

False

In [118]:
db.db.escape??

In [124]:
def f(): return 1

In [123]:
class A:
    def f(this): return f()

In [125]:
A().f()

1

In [122]:
stdSqlData(b'11', db=db)

"'11'"

In [40]:
b

{'YlCMdSUx': {'authors': [{'name': 'Li Ma',
    'org': 'Changchun Inst. of Applied Chemistry'},
   {'name': 'Ziyang Liu', 'org': 'Changchun Inst. of Applied Chemistry'},
   {'name': 'Yaping Xu', 'org': 'Changchun Inst. of Applied Chemistry'},
   {'name': 'Weijie Wang', 'org': 'Changchun Inst. of Applied Chemistry'},
   {'name': 'Xinghua Guo', 'org': 'Changchun Inst. of Applied Chemistry'},
   {'name': 'Shuying Liu', 'org': 'Changchun Inst. of Applied Chemistry'}],
  'title': 'Gas-phase ion-molecule reactions of neutral C60 with alkyl methyl ethers under chemical ionization conditions',
  'abstract': 'Gas-phase ion-molecule reactions of buckminsterfullerene (C',
  'keywords': [''],
  'venue': 'Rapid Communications in Mass Spectrometry',
  'year': 1998,
  'id': 'YlCMdSUx'}}

In [30]:
with open(path + '/na_v1_pub.json') as f:
    parser = ijson.parse(f)
    i = 0
    for x, y, z in parser:
        print(x,y,z)
        i += 1
        if 100 < i: break

 start_map None
 map_key 56PyrvLA
56PyrvLA start_map None
56PyrvLA map_key authors
56PyrvLA.authors start_array None
56PyrvLA.authors.item start_map None
56PyrvLA.authors.item map_key name
56PyrvLA.authors.item.name string Qingya Wang
56PyrvLA.authors.item map_key org
56PyrvLA.authors.item.org string Jilin Univ
56PyrvLA.authors.item end_map None
56PyrvLA.authors.item start_map None
56PyrvLA.authors.item map_key name
56PyrvLA.authors.item.name string Li Ma
56PyrvLA.authors.item map_key org
56PyrvLA.authors.item.org string Jilin Univ
56PyrvLA.authors.item end_map None
56PyrvLA.authors.item start_map None
56PyrvLA.authors.item map_key name
56PyrvLA.authors.item.name string Fanghai Zhao
56PyrvLA.authors.item map_key org
56PyrvLA.authors.item.org string Jilin Univ
56PyrvLA.authors.item end_map None
56PyrvLA.authors.item start_map None
56PyrvLA.authors.item map_key name
56PyrvLA.authors.item.name string Wei Zheng
56PyrvLA.authors.item map_key org
56PyrvLA.authors.item.org string Jilin Univ
5

In [23]:
[*objects]

ValueError: I/O operation on closed file.

In [6]:
db.show()

schemas,Database
0,AMiner_whoiswho
1,academic_data
2,beihang
3,beihangEng
4,beihang_bigdata
5,beihang_electric
6,beihang_energy
7,information_schema
8,keywords
9,mysql


# 关闭

In [136]:
db.close()